# Automotive Predictive Maintenance Code

This notebook implements an AI-powered predictive maintenance system for vehicles using multi-agent architecture with LLM integration.

## Cell 1: Install Required Dependencies

Install necessary Python packages for LangChain, LLM integration, and graph-based agent orchestration.

In [18]:
!pip install langchain_openai
!pip install langchain_core
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.1/476.1 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.1.3
    Uninstalling langchain-core-1.1.3:
      Successfully uninstalled langchain-core-1.1.3


## Cell 2: Import Libraries

Import core libraries for data processing (NumPy, Pandas), visualization (Matplotlib, Seaborn), and utility modules for the application.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from typing import Dict, Optional
import random
import os

## Cell 3: Dataset Loading Utilities

Define utility functions to load vehicle telemetry data, retrieve latest sensor readings, and stream vehicle data for analysis.

In [2]:
"""
Load dataset
"""
import pandas as pd

def load_telemetry(filepath: str):
    df = pd.read_csv("/content/vehicle_telemetry.csv")

    # Basic cleanup
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

    return df

def get_latest_reading(df: pd.DataFrame, vehicle_id: str):
    vehicle_data = df[df['vehicle_id'] == vehicle_id]
    if vehicle_data.empty:
        return None
    return vehicle_data.sort_values('timestamp').iloc[-1].to_dict()

def get_vehicle_stream(df: pd.DataFrame, vehicle_id: str):
    vehicle_data = df[df['vehicle_id'] == vehicle_id].sort_values('timestamp')
    return vehicle_data.to_dict('records')

## Cell 4: Mock Telematics API

Implement a TelemetryAPI class that simulates real-time vehicle sensor data streaming, providing continuous telemetry readings from multiple vehicles for analysis.

In [6]:
"""
Mock Telematics API - Simulates real-time vehicle data streaming.
As per competition requirement: "Telematics API: Mock real-time sensor data feed"
"""

import pandas as pd



class TelemetryAPI:
    """
    Mock Telematics API that simulates real-time vehicle sensor data streaming.
    In production, this would connect to actual vehicle IoT devices.
    """

    def __init__(self, dataset_path: str):
        self.df = pd.read_csv("/content/vehicle_telemetry.csv")
        self.df['timestamp'] = pd.to_datetime(self.df['timestamp'])
        self.current_index = {}  # Track position for each vehicle

        print(f"[TelemetryAPI] Initialized with {len(self.df)} records")
        print(f"[TelemetryAPI] Monitoring {self.df['vehicle_id'].nunique()} vehicles")

    def stream_telemetry(self, vehicle_id: str) -> Optional[Dict]:
        """
        Stream real-time telemetry for a specific vehicle.
        Simulates continuous data feed from vehicle sensors.
        Args:
            vehicle_id: Vehicle identifier

        Returns:
            Dictionary with current sensor readings
        """
        # Get vehicle data
        vehicle_data = self.df[self.df['vehicle_id'] == vehicle_id].copy()

        if len(vehicle_data) == 0:
            return None

        vehicle_data = vehicle_data.sort_values('timestamp')

        # Get current index for this vehicle
        if vehicle_id not in self.current_index:
            self.current_index[vehicle_id] = 0

        idx = self.current_index[vehicle_id]

        if idx >= len(vehicle_data):
            # Reset to beginning (loop)
            idx = 0
            self.current_index[vehicle_id] = 0

        # Get current reading
        reading = vehicle_data.iloc[idx].to_dict()

        # Increment index for next call
        self.current_index[vehicle_id] += 1

        # Add API metadata
        reading['api_timestamp'] = datetime.now().isoformat()
        reading['data_source'] = 'telemetry_api'

        return reading

    def get_latest_reading(self, vehicle_id: str) -> Optional[Dict]:
        """
        Get the most recent telemetry reading for a vehicle.

        Args:
            vehicle_id: Vehicle identifier

        Returns:
            Latest sensor data
        """
        vehicle_data = self.df[self.df['vehicle_id'] == vehicle_id].copy()

        if len(vehicle_data) == 0:
            return None

        vehicle_data = vehicle_data.sort_values('timestamp', ascending=False)
        latest = vehicle_data.iloc[0].to_dict()

        latest['api_timestamp'] = datetime.now().isoformat()
        latest['data_source'] = 'telemetry_api'

        return latest

    def get_historical_data(self, vehicle_id: str, hours: int = 24) -> pd.DataFrame:
        """
        Get historical telemetry data for ML training/analysis.

        Args:
            vehicle_id: Vehicle identifier
            hours: Number of hours of history to retrieve

        Returns:
            DataFrame with historical readings
        """
        vehicle_data = self.df[self.df['vehicle_id'] == vehicle_id].copy()

        if len(vehicle_data) == 0:
            return pd.DataFrame()

        vehicle_data = vehicle_data.sort_values('timestamp', ascending=False)

        # For mock, return last N records
        n_records = min(hours * 6, len(vehicle_data))  # Assume 1 reading per 10 min

        return vehicle_data.head(n_records)

    def get_fleet_summary(self) -> Dict:
        """Get summary of entire fleet status"""

        latest_per_vehicle = self.df.sort_values('timestamp').groupby('vehicle_id').tail(1)

        summary = {
            "total_vehicles": self.df['vehicle_id'].nunique(),
            "vehicles_with_engine_risk": int(latest_per_vehicle['engine_failure_imminent'].sum()),
            "vehicles_with_brake_risk": int(latest_per_vehicle['brake_issue_imminent'].sum()),
            "vehicles_with_battery_risk": int(latest_per_vehicle['battery_issue_imminent'].sum()),
            "timestamp": datetime.now().isoformat()
        }

        return summary

    def simulate_real_time_stream(self, vehicle_id: str, duration_seconds: int = 60):
        """
        Simulate real-time streaming for demo purposes.
        Yields data points at regular intervals.
        """
        vehicle_data = self.df[self.df['vehicle_id'] == vehicle_id].copy()
        vehicle_data = vehicle_data.sort_values('timestamp')

        interval = duration_seconds / len(vehicle_data)

        for _, row in vehicle_data.iterrows():
            reading = row.to_dict()
            reading['api_timestamp'] = datetime.now().isoformat()

            yield reading

            time.sleep(interval)


# Test the API
if __name__ == "__main__":
    api = TelemetryAPI("/content/vehicle_telemetry.csv")

    # Get first vehicle
    vehicles = api.df['vehicle_id'].unique()
    test_vehicle = vehicles[0]



[TelemetryAPI] Initialized with 397 records
[TelemetryAPI] Monitoring 10 vehicles


## Cell 5: Base Agent Framework

Define the abstract BaseAgent class and AgentResponse model that form the foundation for all specialized AI agents in the system.

In [8]:
"""
Base class for all AI agents in the system.
Provides common functionality and structure.
"""

from abc import ABC, abstractmethod
from typing import Dict, Any, Optional
from datetime import datetime
from pydantic import BaseModel
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class AgentResponse(BaseModel):
    """Standard response format from any agent"""
    agent_name: str
    timestamp: datetime
    success: bool
    data: Dict[str, Any]
    error: Optional[str] = None
    next_action: Optional[str] = None


class BaseAgent(ABC):
    """
    Abstract base class for all AI agents.

    Each agent must implement:
    - process(): Main logic
    - get_prompt(): LLM prompt template
    """

    def __init__(self, agent_name: str):
        self.agent_name = agent_name
        self.logger = logging.getLogger(f"Agent.{agent_name}")
        self.execution_count = 0

    @abstractmethod
    def process(self, input_data: Dict[str, Any]) -> AgentResponse:
        """
        Main processing logic for the agent.
        Must be implemented by each agent.

        Args:
            input_data: Input data for processing

        Returns:
            AgentResponse with results
        """
        pass

    @abstractmethod
    def get_prompt(self) -> str:
        return ""


    def log_activity(self, message: str, level: str = "INFO"):
        """Log agent activity"""
        self.execution_count += 1

        if level == "INFO":
            self.logger.info(f"[{self.agent_name}] {message}")
        elif level == "WARNING":
            self.logger.warning(f"[{self.agent_name}] {message}")
        elif level == "ERROR":
            self.logger.error(f"[{self.agent_name}] {message}")

    def create_response(
        self,
        success: bool,
        data: Dict[str, Any],
        error: Optional[str] = None,
        next_action: Optional[str] = None
    ) -> AgentResponse:
        """Create standardized agent response"""
        return AgentResponse(
            agent_name=self.agent_name,
            timestamp=datetime.now(),
            success=success,
            data=data,
            error=error,
            next_action=next_action
        )

## Cell 6: Data Analysis Agent

Implement the first core agent that continuously analyzes streaming vehicle telematics, detects anomalies, and forecasts maintenance needs and service demand.

In [11]:
"""
    Data Analysis Agent - First agent in the predictive maintenance workflow.

    Competition Requirements:
    1. Continuously analyze streaming vehicle telematics
    2. Analyze sensor data + maintenance history
    3. Detect early warning signs
    4. Forecast likely maintenance needs
    5. Forecast service demand

    Key Features:
    - Real-time anomaly detection
    - Historical pattern analysis
    - Baseline comparison
    - Service demand forecasting
    - Multi-vehicle monitoring
    """
from typing import List
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')


class DataAnalysisAgentV2(BaseAgent):


    def __init__(self, telemetry_api, maintenance_history_df: pd.DataFrame):
        super().__init__(agent_name="DataAnalysisAgentV2")

        self.telemetry_api = telemetry_api
        self.maintenance_history = maintenance_history_df

        # Establish baselines from dataset
        self.baselines = self._establish_baselines()

        # Track vehicle states
        self.vehicle_states = defaultdict(dict)

        # Service demand tracker
        self.service_demand_forecast = {
            "daily": 0,
            "weekly": 0,
            "monthly": 0
        }

        self.log_activity("Data Analysis Agent V2 initialized")
        # Handle telemetry input consistently
        if isinstance(self.telemetry_api, dict):
            df = self.telemetry_api["raw_dataframe"]
        else:
            df = self.telemetry_api.df

        self.log_activity(f"Monitoring {df['vehicle_id'].nunique()} vehicles")

        self.log_activity(f"Loaded {len(self.maintenance_history)} maintenance records")

    def _establish_baselines(self) -> Dict:
        """
        Establish normal operating baselines from historical data.
        These are the "healthy" ranges for each sensor.
        """

        if isinstance(self.telemetry_api, dict):
            df = self.telemetry_api["raw_dataframe"]
        else:
            df = self.telemetry_api.df

        # Only use data where no failures are imminent (healthy state)
        healthy_data = df[
            (df['engine_failure_imminent'] == 0) &
            (df['brake_issue_imminent'] == 0) &
            (df['battery_issue_imminent'] == 0)
        ]

        if len(healthy_data) == 0:
            self.log_activity("Warning: No healthy data found, using all data for baselines", level="WARNING")
            healthy_data = df

        baselines = {
            # Engine metrics
            "engine_temp_c": {
                "mean": healthy_data['engine_temp_c'].mean(),
                "std": healthy_data['engine_temp_c'].std(),
                "min": healthy_data['engine_temp_c'].quantile(0.05),
                "max": healthy_data['engine_temp_c'].quantile(0.95),
                "critical_high": 110,
                "critical_low": 50
            },
            "oil_pressure_psi": {
                "mean": healthy_data['oil_pressure_psi'].mean(),
                "std": healthy_data['oil_pressure_psi'].std(),
                "min": healthy_data['oil_pressure_psi'].quantile(0.05),
                "max": healthy_data['oil_pressure_psi'].quantile(0.95),
                "critical_low": 20
            },
            "coolant_temp_c": {
                "mean": healthy_data['coolant_temp_c'].mean(),
                "std": healthy_data['coolant_temp_c'].std(),
                "min": healthy_data['coolant_temp_c'].quantile(0.05),
                "max": healthy_data['coolant_temp_c'].quantile(0.95),
                "critical_high": 105
            },
            "vibration_level": {
                "mean": healthy_data['vibration_level'].mean(),
                "std": healthy_data['vibration_level'].std(),
                "max": healthy_data['vibration_level'].quantile(0.95),
                "critical_high": healthy_data['vibration_level'].quantile(0.99)
            },

            # Brake metrics
            "brake_pad_wear_mm": {
                "mean": healthy_data['brake_pad_wear_mm'].mean(),
                "std": healthy_data['brake_pad_wear_mm'].std(),
                "min": healthy_data['brake_pad_wear_mm'].quantile(0.05),
                "critical_high": healthy_data['brake_pad_wear_mm'].quantile(0.90),  # High wear = bad
                "warning_high": healthy_data['brake_pad_wear_mm'].quantile(0.75)
            },
            "brake_fluid_level_psi": {
                "mean": healthy_data['brake_fluid_level_psi'].mean(),
                "std": healthy_data['brake_fluid_level_psi'].std(),
                "min": healthy_data['brake_fluid_level_psi'].quantile(0.05),
                "critical_low": healthy_data['brake_fluid_level_psi'].quantile(0.10)
            },
            "brake_temp_c": {
                "mean": healthy_data['brake_temp_c'].mean(),
                "std": healthy_data['brake_temp_c'].std(),
                "max": healthy_data['brake_temp_c'].quantile(0.95),
                "critical_high": healthy_data['brake_temp_c'].quantile(0.99)
            },

            # Battery metrics
            "battery_voltage_v": {
                "mean": healthy_data['battery_voltage_v'].mean(),
                "std": healthy_data['battery_voltage_v'].std(),
                "min": healthy_data['battery_voltage_v'].quantile(0.05),
                "max": healthy_data['battery_voltage_v'].quantile(0.95),
                "critical_low": 11.5,
                "warning_low": 12.0
            },
            "battery_health_percent": {
                "mean": healthy_data['battery_health_percent'].mean(),
                "std": healthy_data['battery_health_percent'].std(),
                "min": healthy_data['battery_health_percent'].quantile(0.05),
                "critical_low": 60,
                "warning_low": 75
            },
            "battery_temp_c": {
                "mean": healthy_data['battery_temp_c'].mean(),
                "std": healthy_data['battery_temp_c'].std(),
                "max": healthy_data['battery_temp_c'].quantile(0.95),
                "critical_high": healthy_data['battery_temp_c'].quantile(0.99)
            }
        }

        self.log_activity("Baselines established from healthy vehicle data")

        return baselines

    def analyze_telemetry_stream(self, vehicle_id: str) -> Dict[str, Any]:
        """
        Analyze real-time telemetry stream for a specific vehicle.
        This is the main function called continuously.

        Args:
            vehicle_id: Vehicle to analyze

        Returns:
            Analysis results with anomalies and recommendations
        """

        self.log_activity(f"Analyzing telemetry stream for vehicle: {vehicle_id}")

        # Step 1: Get latest telemetry from API
        telemetry = self.telemetry_api.get_latest_reading(vehicle_id)

        if telemetry is None:
            return {
                "vehicle_id": vehicle_id,
                "status": "error",
                "message": "No telemetry data available"
            }

        #Get historical data for trend analysis
        historical = self.telemetry_api.get_historical_data(vehicle_id, hours=24)

        #Get maintenance history
        maintenance_records = self._get_vehicle_maintenance_history(vehicle_id)

        #Detect anomalies
        anomalies = self._detect_anomalies(telemetry, historical)

        #Analyze degradation trends
        degradation = self._analyze_degradation_trends(historical)

        #Check maintenance history patterns
        historical_patterns = self._analyze_maintenance_patterns(maintenance_records, telemetry)

        #Calculate risk score
        risk_score = self._calculate_risk_score(anomalies, degradation, telemetry)

        # Determine priority
        priority = self._determine_priority(risk_score, anomalies, telemetry)

        #Forecast maintenance needs
        maintenance_forecast = self._forecast_maintenance_needs(
            telemetry, historical, maintenance_records, degradation
        )

        # Compile results
        analysis_result = {
            "vehicle_id": vehicle_id,
            "timestamp": telemetry.get('timestamp'),
            "analysis_timestamp": datetime.now().isoformat(),

            # Current state
            "current_telemetry": telemetry,

            # Anomalies detected
            "anomalies_detected": len(anomalies),
            "anomalies": anomalies,

            # Degradation trends
            "degradation_trends": degradation,

            # Historical patterns
            "maintenance_history_insights": historical_patterns,

            # Risk assessment
            "risk_score": risk_score,
            "priority": priority,
            "requires_immediate_action": priority in ["critical", "high"],

            # Forecasting
            "maintenance_forecast": maintenance_forecast,

            # Early warnings
            "early_warnings": self._generate_early_warnings(
                anomalies, degradation, maintenance_forecast
            )
        }

        # Update service demand forecast
        self._update_service_demand_forecast(analysis_result)

        self.log_activity(
            f"Analysis complete - Priority: {priority}, "
            f"Anomalies: {len(anomalies)}, Risk: {risk_score:.2f}"
        )

        return analysis_result

    def _detect_anomalies(self, telemetry: Dict, historical: pd.DataFrame) -> List[Dict]:
        """
        Detect anomalies by comparing current readings against baselines.
        """

        anomalies = []

        # Engine anomalies
        engine_temp = telemetry.get('engine_temp_c', 0)
        if engine_temp > self.baselines['engine_temp_c']['critical_high']:
            anomalies.append({
                "component": "engine",
                "metric": "engine_temp_c",
                "severity": "critical",
                "current_value": engine_temp,
                "threshold": self.baselines['engine_temp_c']['critical_high'],
                "message": f"Engine temperature critically high: {engine_temp}°C",
                "recommendation": "Immediate inspection required - potential overheating"
            })
        elif engine_temp > self.baselines['engine_temp_c']['max']:
            anomalies.append({
                "component": "engine",
                "metric": "engine_temp_c",
                "severity": "high",
                "current_value": engine_temp,
                "threshold": self.baselines['engine_temp_c']['max'],
                "message": f"Engine temperature above normal: {engine_temp}°C",
                "recommendation": "Check cooling system"
            })

        # Oil pressure
        oil_pressure = telemetry.get('oil_pressure_psi', 0)
        if oil_pressure < self.baselines['oil_pressure_psi']['critical_low']:
            anomalies.append({
                "component": "engine",
                "metric": "oil_pressure_psi",
                "severity": "critical",
                "current_value": oil_pressure,
                "threshold": self.baselines['oil_pressure_psi']['critical_low'],
                "message": f"Oil pressure critically low: {oil_pressure} PSI",
                "recommendation": "Stop vehicle immediately - check oil level"
            })

        # Brake pad wear
        brake_wear = telemetry.get('brake_pad_wear_mm', 0)
        if brake_wear > self.baselines['brake_pad_wear_mm']['critical_high']:
            anomalies.append({
                "component": "brake",
                "metric": "brake_pad_wear_mm",
                "severity": "critical",
                "current_value": brake_wear,
                "threshold": self.baselines['brake_pad_wear_mm']['critical_high'],
                "message": f"Brake pads critically worn: {brake_wear}mm",
                "recommendation": "Replace brake pads immediately"
            })
        elif brake_wear > self.baselines['brake_pad_wear_mm']['warning_high']:
            anomalies.append({
                "component": "brake",
                "metric": "brake_pad_wear_mm",
                "severity": "high",
                "current_value": brake_wear,
                "threshold": self.baselines['brake_pad_wear_mm']['warning_high'],
                "message": f"Brake pads wearing thin: {brake_wear}mm",
                "recommendation": "Schedule brake service soon"
            })

        # Battery health
        battery_health = telemetry.get('battery_health_percent', 100)
        if battery_health < self.baselines['battery_health_percent']['critical_low']:
            anomalies.append({
                "component": "battery",
                "metric": "battery_health_percent",
                "severity": "critical",
                "current_value": battery_health,
                "threshold": self.baselines['battery_health_percent']['critical_low'],
                "message": f"Battery health poor: {battery_health}%",
                "recommendation": "Replace battery soon to avoid breakdown"
            })
        elif battery_health < self.baselines['battery_health_percent']['warning_low']:
            anomalies.append({
                "component": "battery",
                "metric": "battery_health_percent",
                "severity": "medium",
                "current_value": battery_health,
                "threshold": self.baselines['battery_health_percent']['warning_low'],
                "message": f"Battery health declining: {battery_health}%",
                "recommendation": "Monitor battery, consider replacement"
            })

        # Battery voltage
        battery_voltage = telemetry.get('battery_voltage_v', 12)
        if battery_voltage < self.baselines['battery_voltage_v']['critical_low']:
            anomalies.append({
                "component": "battery",
                "metric": "battery_voltage_v",
                "severity": "high",
                "current_value": battery_voltage,
                "threshold": self.baselines['battery_voltage_v']['critical_low'],
                "message": f"Battery voltage low: {battery_voltage}V",
                "recommendation": "Check charging system"
            })

        # Vibration level
        vibration = telemetry.get('vibration_level', 0)
        if vibration > self.baselines['vibration_level']['critical_high']:
            anomalies.append({
                "component": "engine",
                "metric": "vibration_level",
                "severity": "high",
                "current_value": vibration,
                "threshold": self.baselines['vibration_level']['critical_high'],
                "message": f"Excessive vibration detected: {vibration}",
                "recommendation": "Check engine mounts and balance"
            })

        # ABS fault indicator
        if telemetry.get('abs_fault_indicator', 0) == 1:
            anomalies.append({
                "component": "brake",
                "metric": "abs_fault_indicator",
                "severity": "high",
                "current_value": 1,
                "threshold": 0,
                "message": "ABS fault indicator active",
                "recommendation": "Diagnose ABS system immediately"
            })

        return anomalies

    def _analyze_degradation_trends(self, historical: pd.DataFrame) -> Dict:
        """
        Analyze degradation trends from historical data.
        Identifies components that are degrading over time.
        """

        if len(historical) < 5:
            return {"trends": [], "message": "Insufficient historical data"}

        trends = []

        # Sort by timestamp
        historical = historical.sort_values('timestamp')

        # Analyze brake pad wear trend
        if 'brake_pad_wear_mm' in historical.columns:
            wear_trend = historical['brake_pad_wear_mm'].diff().mean()
            if wear_trend > 0.1:  # Increasing wear
                days_to_critical = (
                    (self.baselines['brake_pad_wear_mm']['critical_high'] -
                     historical['brake_pad_wear_mm'].iloc[-1]) / wear_trend
                )
                trends.append({
                    "component": "brake",
                    "metric": "brake_pad_wear_mm",
                    "trend": "increasing",
                    "rate": float(wear_trend),
                    "estimated_days_to_critical": max(0, int(days_to_critical)),
                    "current_value": float(historical['brake_pad_wear_mm'].iloc[-1])
                })

        # Analyze battery health trend
        if 'battery_health_percent' in historical.columns:
            health_trend = historical['battery_health_percent'].diff().mean()
            if health_trend < -0.5:  # Declining health
                days_to_critical = (
                    (historical['battery_health_percent'].iloc[-1] -
                     self.baselines['battery_health_percent']['critical_low']) / abs(health_trend)
                )
                trends.append({
                    "component": "battery",
                    "metric": "battery_health_percent",
                    "trend": "declining",
                    "rate": float(health_trend),
                    "estimated_days_to_critical": max(0, int(days_to_critical)),
                    "current_value": float(historical['battery_health_percent'].iloc[-1])
                })

        # Analyze engine temperature stability
        if 'engine_temp_c' in historical.columns:
            temp_std = historical['engine_temp_c'].std()
            if temp_std > self.baselines['engine_temp_c']['std'] * 1.5:
                trends.append({
                    "component": "engine",
                    "metric": "engine_temp_c",
                    "trend": "unstable",
                    "variability": float(temp_std),
                    "message": "Engine temperature fluctuating more than normal"
                })

        return {
            "trends_detected": len(trends),
            "trends": trends
        }

    def _get_vehicle_maintenance_history(self, vehicle_id: str) -> pd.DataFrame:
        """Get maintenance history for a specific vehicle"""

        if self.maintenance_history is None or len(self.maintenance_history) == 0:
            return pd.DataFrame()

        # Filter by vehicle_id if column exists
        if 'vehicle_id' in self.maintenance_history.columns:
            return self.maintenance_history[
                self.maintenance_history['vehicle_id'] == vehicle_id
            ].copy()

        return pd.DataFrame()

    def _analyze_maintenance_patterns(self, maintenance_records: pd.DataFrame,
                                     current_telemetry: Dict) -> Dict:
        """
        Analyze maintenance history to find patterns.
        Cross-reference with current telemetry.
        """

        if len(maintenance_records) == 0:
            return {
                "past_services": 0,
                "patterns": [],
                "message": "No maintenance history available"
            }

        patterns = []

        # Count services by type
        if 'service_type' in maintenance_records.columns:
            service_counts = maintenance_records['service_type'].value_counts().to_dict()

            # Check if brake services are frequent
            brake_services = maintenance_records[
                maintenance_records['service_type'].str.contains('brake', case=False, na=False)
            ]

            if len(brake_services) >= 2:
                patterns.append({
                    "pattern_type": "recurring_brake_service",
                    "frequency": len(brake_services),
                    "message": f"Vehicle has required {len(brake_services)} brake services",
                    "recommendation": "Monitor brake system closely"
                })

        # Check last service date
        if 'service_date' in maintenance_records.columns:
            last_service = pd.to_datetime(maintenance_records['service_date']).max()
            days_since_service = (datetime.now() - last_service).days

            if days_since_service > 180:  # 6 months
                patterns.append({
                    "pattern_type": "overdue_service",
                    "days_since_last_service": days_since_service,
                    "message": f"Last service was {days_since_service} days ago",
                    "recommendation": "Schedule routine maintenance"
                })

        return {
            "past_services": len(maintenance_records),
            "patterns_detected": len(patterns),
            "patterns": patterns
        }

    def _calculate_risk_score(self, anomalies: List[Dict],
                             degradation: Dict, telemetry: Dict) -> float:
        """
        Calculate overall risk score (0.0 - 1.0).

        Factors:
        - Number and severity of anomalies
        - Degradation trends
        - Dataset's imminent failure flags
        """

        risk_score = 0.0

        # Anomaly contribution
        severity_weights = {
            "critical": 0.35,
            "high": 0.25,
            "medium": 0.15,
            "low": 0.05
        }

        for anomaly in anomalies:
            risk_score += severity_weights.get(anomaly.get("severity", "low"), 0.05)

        # Degradation trend contribution
        for trend in degradation.get("trends", []):
            if trend.get("estimated_days_to_critical", 999) < 7:
                risk_score += 0.20
            elif trend.get("estimated_days_to_critical", 999) < 30:
                risk_score += 0.10

        # Dataset flags (if available)
        if telemetry.get('engine_failure_imminent', 0) == 1:
            risk_score += 0.30
        if telemetry.get('brake_issue_imminent', 0) == 1:
            risk_score += 0.30
        if telemetry.get('battery_issue_imminent', 0) == 1:
            risk_score += 0.25

        # Cap at 1.0
        return min(risk_score, 1.0)

    def _determine_priority(self, risk_score: float, anomalies: List[Dict],
                           telemetry: Dict) -> str:
        """
        Determine priority level.

        Returns: "critical", "high", "medium", "low", or "none"
        """

        # Check dataset flags first
        if (telemetry.get('engine_failure_imminent', 0) == 1 or
            any(a.get('severity') == 'critical' for a in anomalies)):
            return "critical"

        if risk_score >= 0.7:
            return "critical"
        elif risk_score >= 0.5:
            return "high"
        elif risk_score >= 0.3:
            return "medium"
        elif risk_score >= 0.1:
            return "low"
        else:
            return "none"

    def _forecast_maintenance_needs(self, telemetry: Dict, historical: pd.DataFrame,
                                   maintenance_records: pd.DataFrame,
                                   degradation: Dict) -> Dict:
        """
        Forecast when maintenance will be needed.
        This helps optimize service center capacity.
        """

        forecasts = []

        # Based on degradation trends
        for trend in degradation.get("trends", []):
            days_to_critical = trend.get("estimated_days_to_critical", 999)

            if days_to_critical < 90:
                forecasts.append({
                    "component": trend.get("component"),
                    "metric": trend.get("metric"),
                    "estimated_days": days_to_critical,
                    "urgency": "high" if days_to_critical < 7 else "medium",
                    "action_required": "service_scheduling"
                })

        # Based on mileage (if maintenance records show patterns)
        if len(maintenance_records) >= 2:
            # Simple forecast: average interval between services
            if 'odometer_reading' in maintenance_records.columns:
                avg_interval = maintenance_records['odometer_reading'].diff().mean()
                current_odo = telemetry.get('odometer_reading', 0)
                last_service_odo = maintenance_records['odometer_reading'].max()

                km_since_service = current_odo - last_service_odo

                if km_since_service > avg_interval * 0.8:
                    forecasts.append({
                        "component": "general",
                        "metric": "scheduled_maintenance",
                        "estimated_days": 30,  # Rough estimate
                        "urgency": "medium",
                        "action_required": "routine_service"
                    })

        return {
            "forecasts": forecasts,
            "total_forecasted_needs": len(forecasts)
        }

    def _generate_early_warnings(self, anomalies: List[Dict],
                                 degradation: Dict, forecast: Dict) -> List[str]:
        """Generate human-readable early warnings"""

        warnings = []

        # Critical anomalies
        critical_anomalies = [a for a in anomalies if a.get('severity') == 'critical']
        if critical_anomalies:
            for anom in critical_anomalies:
                warnings.append(f" CRITICAL: {anom.get('message')}")

        # Degradation trends
        for trend in degradation.get("trends", []):
            days = trend.get("estimated_days_to_critical", 999)
            if days < 7:
                warnings.append(
                    f" {trend.get('component').upper()}: "
                    f"{trend.get('metric')} reaching critical in ~{days} days"
                )

        # Forecast warnings
        for fc in forecast.get("forecasts", []):
            if fc.get("urgency") == "high":
                warnings.append(
                    f" {fc.get('component').upper()} service needed within "
                    f"{fc.get('estimated_days')} days"
                )

        return warnings

    def _update_service_demand_forecast(self, analysis_result: Dict):
        """
        Update service demand forecast across the fleet.
        Helps optimize service center capacity.
        """

        # If this vehicle needs service soon, update demand
        if analysis_result.get("requires_immediate_action"):
            self.service_demand_forecast["daily"] += 1
            self.service_demand_forecast["weekly"] += 1
            self.service_demand_forecast["monthly"] += 1

        # Based on forecasts
        for forecast in analysis_result.get("maintenance_forecast", {}).get("forecasts", []):
            days = forecast.get("estimated_days", 999)

            if days <= 1:
                self.service_demand_forecast["daily"] += 1
            if days <= 7:
                self.service_demand_forecast["weekly"] += 1
            if days <= 30:
                self.service_demand_forecast["monthly"] += 1

    def get_service_demand_forecast(self) -> Dict:
        """
        Get current service demand forecast.
        Used by service centers for capacity planning.
        """
        return {
            "forecast_timestamp": datetime.now().isoformat(),
            "estimated_demand": self.service_demand_forecast,
            "recommendation": self._generate_capacity_recommendation()
        }

    def _generate_capacity_recommendation(self) -> str:
        """Generate capacity planning recommendation"""

        daily = self.service_demand_forecast["daily"]

        if daily >= 10:
            return "High demand expected - schedule additional technicians"
        elif daily >= 5:
            return "Moderate demand - maintain current capacity"
        else:
            return "Low demand - standard staffing sufficient"

    def process(self, input_data: Dict[str, Any]) -> AgentResponse:
        """
        Main processing entry point for LangGraph integration.

        Args:
            input_data: Must contain "vehicle_id"

        Returns:
            AgentResponse with analysis results
        """

        vehicle_id = input_data.get("vehicle_id")

        if not vehicle_id:
            return self.create_response(
                success=False,
                data={},
                error="vehicle_id is required"
            )

        try:
            # Run complete analysis
            analysis = self.analyze_telemetry_stream(vehicle_id)

            return self.create_response(
                success=True,
                data=analysis,
                next_action="diagnosis" if analysis.get("requires_immediate_action") else "monitor"
            )

        except Exception as e:
            self.log_activity(f"Analysis failed: {str(e)}", level="ERROR")
            return self.create_response(
                success=False,
                data={},
                error=str(e)
            )

    def get_prompt(self) -> str:
        """Not used - this agent is rule-based with statistical analysis"""
        return "Rule-based data analysis agent"

In [12]:
"""
Machine Learning models for failure prediction.
"""
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import joblib
from datetime import datetime, timedelta


class FailurePredictionModel:
    """
    ML Model for predicting vehicle component failures.
    Trains on competition dataset features.
    """

    def __init__(self):
        self.engine_model = None
        self.brake_model = None
        self.battery_model = None
        self.scaler = StandardScaler()
        self.feature_columns = []

    def prepare_features(self, df: pd.DataFrame) -> tuple:
        """
        Prepare features from the competition dataset.

        Features used:
        - Engine: engine_temp_c, oil_pressure_psi, coolant_temp_c, vibration_level
        - Brake: brake_pad_wear_mm, brake_fluid_level_psi, brake_temp_c, brake_pedal_pos_percent
        - Battery: battery_voltage_v, battery_current_a, battery_temp_c, battery_health_percent
        """

        # Define features
        self.feature_columns = [
            'odometer_reading', 'engine_temp_c', 'oil_pressure_psi', 'coolant_temp_c',
            'fuel_level_percent', 'vibration_level', 'brake_fluid_level_psi',
            'brake_pad_wear_mm', 'brake_temp_c', 'brake_pedal_pos_percent',
            'battery_voltage_v', 'battery_current_a', 'battery_temp_c',
            'alternator_output_v', 'battery_charge_percent', 'battery_health_percent',
            'vehicle_speed_kph', 'ambient_temp_c'
        ]

        # Extract features
        X = df[self.feature_columns].copy()

        # Handle missing values
        X = X.fillna(X.mean())

        # Extract targets
        y_engine = df['engine_failure_imminent'].astype(int)
        y_brake = df['brake_issue_imminent'].astype(int)
        y_battery = df['battery_issue_imminent'].astype(int)

        return X, y_engine, y_brake, y_battery

    def train(self, df: pd.DataFrame):
        """Train all three failure prediction models"""

        print("[ML] Preparing training data...")
        X, y_engine, y_brake, y_battery = self.prepare_features(df)

        # Scale features
        X_scaled = self.scaler.fit_transform(X)

        # Train Engine Failure Model
        print("[ML] Training Engine Failure Model...")
        self.engine_model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            class_weight='balanced'
        )

        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y_engine, test_size=0.2, random_state=42
        )

        self.engine_model.fit(X_train, y_train)
        y_pred = self.engine_model.predict(X_test)

        print(f"  Engine Model Accuracy: {accuracy_score(y_test, y_pred):.2%}")

        # Train Brake Issue Model
        print("[ML] Training Brake Issue Model...")
        self.brake_model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            class_weight='balanced'
        )

        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y_brake, test_size=0.2, random_state=42
        )

        self.brake_model.fit(X_train, y_train)
        y_pred = self.brake_model.predict(X_test)

        print(f"  Brake Model Accuracy: {accuracy_score(y_test, y_pred):.2%}")

        # Train Battery Issue Model
        print("[ML] Training Battery Issue Model...")
        self.battery_model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            class_weight='balanced'
        )

        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y_battery, test_size=0.2, random_state=42
        )

        self.battery_model.fit(X_train, y_train)
        y_pred = self.battery_model.predict(X_test)

        print(f"  Battery Model Accuracy: {accuracy_score(y_test, y_pred):.2%}")

        print("[ML] All models trained successfully!")

    def predict(self, telemetry_data: Dict) -> Dict:
        """
        Predict failures from current telemetry reading.

        Returns:
            Dictionary with predictions for each component
        """

        # Prepare features
        features = []
        for col in self.feature_columns:
            features.append(telemetry_data.get(col, 0))

        X = np.array(features).reshape(1, -1)
        X_scaled = self.scaler.transform(X)

        # Get predictions and probabilities
        engine_pred = self.engine_model.predict(X_scaled)[0]
        engine_prob = self.engine_model.predict_proba(X_scaled)[0][1]

        brake_pred = self.brake_model.predict(X_scaled)[0]
        brake_prob = self.brake_model.predict_proba(X_scaled)[0][1]

        battery_pred = self.battery_model.predict(X_scaled)[0]
        battery_prob = self.battery_model.predict_proba(X_scaled)[0][1]

        return {
            "engine_failure_risk": bool(engine_pred),
            "engine_failure_probability": float(engine_prob),
            "brake_issue_risk": bool(brake_pred),
            "brake_issue_probability": float(brake_prob),
            "battery_issue_risk": bool(battery_pred),
            "battery_issue_probability": float(battery_prob),
            "overall_risk_score": float((engine_prob + brake_prob + battery_prob) / 3),
            "predicted_at": datetime.now().isoformat()
        }

    def save_models(self, path: str = "trained_models/"):
        """Save all trained models and artifacts needed by DiagnosisAgentV2"""
        import os
        os.makedirs(path, exist_ok=True)

        # Save models
        joblib.dump(self.engine_model, f"{path}/engine_model.pkl")
        joblib.dump(self.brake_model, f"{path}/brake_model.pkl")
        joblib.dump(self.battery_model, f"{path}/battery_model.pkl")

        # Save separate scalers (or reuse one if shared)
        joblib.dump(self.scaler, f"{path}/engine_scaler.pkl")
        joblib.dump(self.scaler, f"{path}/brake_scaler.pkl")
        joblib.dump(self.scaler, f"{path}/battery_scaler.pkl")

        # Save feature column list
        joblib.dump(self.feature_columns, f"{path}/feature_columns.pkl")

        # Save training metadata
        joblib.dump({
            "trained_at": datetime.now().isoformat(),
            "model_type": "RandomForest",
            "features_used": self.feature_columns
        }, f"{path}/metadata.pkl")

        print(f"[ML] All models and scalers saved to {path}")


    def load_models(self, path: str = "trained_models/"):
        """Load pre-trained models"""
        self.engine_model = joblib.load(f"{path}/engine_model.pkl")
        self.brake_model = joblib.load(f"{path}/brake_model.pkl")
        self.battery_model = joblib.load(f"{path}/battery_model.pkl")
        self.scaler = joblib.load(f"{path}/scaler.pkl")

        print(f"[ML] Models loaded from {path}")


# Train the models
if __name__ == "__main__":
    # Load dataset
    df = pd.read_csv("/content/vehicle_telemetry.csv")

    # Initialize and train
    model = FailurePredictionModel()
    model.train(df)

    # Save models
    model.save_models()

    print("\n ML Models trained and saved!")

[ML] Preparing training data...
[ML] Training Engine Failure Model...
  Engine Model Accuracy: 100.00%
[ML] Training Brake Issue Model...
  Brake Model Accuracy: 98.75%
[ML] Training Battery Issue Model...
  Battery Model Accuracy: 100.00%
[ML] All models trained successfully!
[ML] All models and scalers saved to trained_models/

 ML Models trained and saved!


In [15]:
os.environ["OPENAI_API_KEY"] = "API_KEY"


In [16]:
import os
print(os.getenv("OPENAI_API_KEY"))


API_KEY


In [17]:
llm_model = "gpt-4o-mini"  # or "gpt-4o" if you have access
temperature = 0.2

## Cell 7: Diagnostic Agent

Implement the diagnostic agent that interprets data analysis results and generates detailed diagnostic reports with root cause analysis.

In [20]:
"""
    Diagnosis Agent - Uses trained ML models to predict failures.

    Competition Requirements:
    1. Run predictive models to assess probable failures
    2. Assign priority levels
    3. Estimate time to failure
    4. Generate service recommendations

    Input: Analysis from Data Analysis Agent
    Output: Detailed diagnosis with ML predictions
  """

import uuid
from langchain_openai import ChatOpenAI



class DiagnosisAgentV2(BaseAgent):
    def __init__(self, models_dir: str = "trained_models/"):
        super().__init__(agent_name="DiagnosisAgentV2")

        self.models_dir = models_dir

        # Load ML models
        self._load_models()
        self.llm = ChatOpenAI(
            model=llm_model,
            temperature=temperature,
            api_key=os.getenv("OPENAI_API_KEY")
        )

        self.log_activity("Diagnosis Agent V2 initialized with ML models")

    def _load_models(self):
        """Load pre-trained ML models"""

        if not os.path.exists(self.models_dir):
            raise FileNotFoundError(
                f"Models directory not found: {self.models_dir}\n"
                f"Please train models first by running: python models/train_models.py"
            )

        self.log_activity(f"Loading ML models from {self.models_dir}...")

        try:
            # Load models
            self.engine_model = joblib.load(f"{self.models_dir}/engine_model.pkl")
            self.brake_model = joblib.load(f"{self.models_dir}/brake_model.pkl")
            self.battery_model = joblib.load(f"{self.models_dir}/battery_model.pkl")

            # Load scalers
            self.engine_scaler = joblib.load(f"{self.models_dir}/engine_scaler.pkl")
            self.brake_scaler = joblib.load(f"{self.models_dir}/brake_scaler.pkl")
            self.battery_scaler = joblib.load(f"{self.models_dir}/battery_scaler.pkl")

            # Load feature columns
            self.feature_columns = joblib.load(f"{self.models_dir}/feature_columns.pkl")

            # Load metadata
            self.metadata = joblib.load(f"{self.models_dir}/metadata.pkl")

            self.log_activity(f" Models loaded successfully")
            self.log_activity(f"   Trained on: {self.metadata.get('trained_at')}")
            self.log_activity(f"   Features: {len(self.feature_columns)}")

        except Exception as e:
            self.log_activity(f"Failed to load models: {str(e)}", level="ERROR")
            raise

    def predict_failures(self, telemetry_data: Dict) -> Dict:
        """
        Run ML models to predict failures for all components.

        Args:
            telemetry_data: Current sensor readings

        Returns:
            Predictions for engine, brake, and battery
        """

        # Prepare features
        features = []
        for col in self.feature_columns:
            value = telemetry_data.get(col, 0)
            features.append(value if value is not None else 0)

        X = np.array(features).reshape(1, -1)

        # Engine prediction
        X_engine = self.engine_scaler.transform(X)
        engine_pred = self.engine_model.predict(X_engine)[0]
        engine_proba = self.engine_model.predict_proba(X_engine)[0]

        # Brake prediction
        X_brake = self.brake_scaler.transform(X)
        brake_pred = self.brake_model.predict(X_brake)[0]
        brake_proba = self.brake_model.predict_proba(X_brake)[0]

        # Battery prediction
        X_battery = self.battery_scaler.transform(X)
        battery_pred = self.battery_model.predict(X_battery)[0]
        battery_proba = self.battery_model.predict_proba(X_battery)[0]

        return {
            "engine": {
                "failure_predicted": bool(engine_pred),
                "failure_probability": float(engine_proba[1]),
                "confidence": float(max(engine_proba)),
                "model_version": self.metadata.get("trained_at")
            },
            "brake": {
                "failure_predicted": bool(brake_pred),
                "failure_probability": float(brake_proba[1]),
                "confidence": float(max(brake_proba)),
                "model_version": self.metadata.get("trained_at")
            },
            "battery": {
                "failure_predicted": bool(battery_pred),
                "failure_probability": float(battery_proba[1]),
                "confidence": float(max(battery_proba)),
                "model_version": self.metadata.get("trained_at")
            }
        }

    def estimate_time_to_failure(self, component: str, current_telemetry: Dict,
                                 degradation_trends: List[Dict]) -> int:
        """
        Estimate days until failure.

        Uses:
        1. Degradation trends from Data Analysis Agent
        2. Current sensor values
        3. ML model confidence
        """

        # Check degradation trends first
        for trend in degradation_trends:
            if trend.get('component') == component:
                days = trend.get('estimated_days_to_critical', 30)
                return max(1, days)

        # Fallback estimation based on current values
        if component == "engine":
            temp = current_telemetry.get('engine_temp_c', 90)
            if temp > 105:
                return 3
            elif temp > 100:
                return 7
            else:
                return 30

        elif component == "brake":
            wear = current_telemetry.get('brake_pad_wear_mm', 0)
            if wear > 8:
                return 5
            elif wear > 6:
                return 14
            else:
                return 30

        elif component == "battery":
            health = current_telemetry.get('battery_health_percent', 100)
            if health < 60:
                return 7
            elif health < 75:
                return 21
            else:
                return 60

        return 30  # Default

    def assign_priority(self, ml_predictions: Dict, analysis_data: Dict) -> str:
        """
        Assign priority level based on ML predictions and analysis.

        Priority levels:
        - critical: Immediate action required (< 3 days)
        - high: Service needed soon (3-7 days)
        - medium: Schedule service (7-30 days)
        - low: Monitor (> 30 days)

        Returns: "critical", "high", "medium", or "low"
        """

        # Check ML predictions
        any_failure_predicted = (
            ml_predictions['engine']['failure_predicted'] or
            ml_predictions['brake']['failure_predicted'] or
            ml_predictions['battery']['failure_predicted']
        )

        # Check probabilities
        max_probability = max(
            ml_predictions['engine']['failure_probability'],
            ml_predictions['brake']['failure_probability'],
            ml_predictions['battery']['failure_probability']
        )

        # Check Data Analysis Agent's assessment
        data_risk_score = analysis_data.get('risk_score', 0)
        data_priority = analysis_data.get('priority', 'none')

        # Decision logic
        if data_priority == "critical" or any_failure_predicted:
            return "critical"

        if max_probability > 0.7 or data_risk_score > 0.7:
            return "critical"

        if max_probability > 0.5 or data_risk_score > 0.5:
            return "high"

        if max_probability > 0.3 or data_risk_score > 0.3:
            return "medium"

        return "low"

    def generate_service_recommendations(self, ml_predictions: Dict,
                                        telemetry_data: Dict,
                                        priority: str) -> List[Dict]:
        """
        Generate specific service recommendations based on predictions.
        """

        recommendations = []

        # Engine recommendations
        if ml_predictions['engine']['failure_predicted']:
            recommendations.append({
                "component": "engine",
                "service_type": "Engine Diagnostic & Repair",
                "urgency": "high",
                "description": "ML model predicts engine failure risk",
                "actions": [
                    "Comprehensive engine diagnostic scan",
                    "Check engine temperature regulation",
                    "Inspect oil pressure system",
                    "Test coolant system"
                ],
                "estimated_duration_hours": 3.0,
                "estimated_cost_min": 5000,
                "estimated_cost_max": 15000,
                "confidence": ml_predictions['engine']['failure_probability']
            })

        # Brake recommendations
        if ml_predictions['brake']['failure_predicted']:
            wear = telemetry_data.get('brake_pad_wear_mm', 0)
            recommendations.append({
                "component": "brake",
                "service_type": "Brake System Service",
                "urgency": "high" if wear > 7 else "medium",
                "description": f"ML model predicts brake issues (wear: {wear}mm)",
                "actions": [
                    "Replace brake pads" if wear > 6 else "Inspect brake pads",
                    "Check brake fluid level",
                    "Test brake system pressure",
                    "Inspect brake rotors"
                ],
                "estimated_duration_hours": 2.0,
                "estimated_cost_min": 4000,
                "estimated_cost_max": 8000,
                "confidence": ml_predictions['brake']['failure_probability']
            })

        # Battery recommendations
        if ml_predictions['battery']['failure_predicted']:
            health = telemetry_data.get('battery_health_percent', 100)
            recommendations.append({
                "component": "battery",
                "service_type": "Battery Service",
                "urgency": "high" if health < 60 else "medium",
                "description": f"ML model predicts battery issues (health: {health}%)",
                "actions": [
                    "Battery load test",
                    "Check charging system",
                    "Inspect alternator output",
                    "Replace battery if needed"
                ],
                "estimated_duration_hours": 1.5,
                "estimated_cost_min": 3000,
                "estimated_cost_max": 8000,
                "confidence": ml_predictions['battery']['failure_probability']
            })

        # If no ML predictions but high priority from data analysis
        if not recommendations and priority in ["high", "critical"]:
            recommendations.append({
                "component": "general",
                "service_type": "Comprehensive Inspection",
                "urgency": priority,
                "description": "Data analysis indicates potential issues",
                "actions": [
                    "Full vehicle diagnostic scan",
                    "Visual inspection of all systems",
                    "Sensor calibration check"
                ],
                "estimated_duration_hours": 2.0,
                "estimated_cost_min": 2000,
                "estimated_cost_max": 5000,
                "confidence": 0.7
            })

        return recommendations

    def process(self, input_data: Dict[str, Any]) -> AgentResponse:
        """
        Main diagnosis process.

        Args:
            input_data: Must contain:
                - analysis_result: Output from Data Analysis Agent
                - telemetry_data: Current sensor readings
                - vehicle_id: Vehicle identifier

        Returns:
            AgentResponse with complete diagnosis
        """

        try:
            self.log_activity("Starting ML-based diagnosis")

            # Extract input data
            analysis_result = input_data.get("analysis_result", {})
            telemetry_data = input_data.get("telemetry_data", {})
            vehicle_id = input_data.get("vehicle_id", "Unknown")

            if not telemetry_data:
                telemetry_data = analysis_result.get("current_telemetry", {})

            # Step 1: Run ML models
            self.log_activity("Running ML models for failure prediction...")
            ml_predictions = self.predict_failures(telemetry_data)

            # Step 2: Estimate time to failure for predicted failures
            time_to_failure_estimates = {}
            degradation_trends = analysis_result.get("degradation_trends", {}).get("trends", [])

            for component, prediction in ml_predictions.items():
                if prediction['failure_predicted']:
                    days = self.estimate_time_to_failure(
                        component, telemetry_data, degradation_trends
                    )
                    time_to_failure_estimates[component] = days

            # Step 3: Assign priority
            priority = self.assign_priority(ml_predictions, analysis_result)

            # Step 4: Generate service recommendations
            recommendations = self.generate_service_recommendations(
                ml_predictions, telemetry_data, priority
            )

            # Step 5: Calculate overall confidence
            confidence_scores = [
                ml_predictions['engine']['confidence'],
                ml_predictions['brake']['confidence'],
                ml_predictions['battery']['confidence']
            ]
            overall_confidence = sum(confidence_scores) / len(confidence_scores)

            # Compile diagnosis result
            diagnosis_result = {
                "diagnosis_id": f"DIAG-{uuid.uuid4().hex[:8]}",
                "vehicle_id": vehicle_id,
                "timestamp": datetime.now().isoformat(),

                # ML Predictions
                "ml_predictions": ml_predictions,

                # Priority assessment
                "priority": priority,
                "requires_immediate_service": priority in ["critical", "high"],

                # Time estimates
                "time_to_failure_estimates": time_to_failure_estimates,

                # Service recommendations
                "service_recommendations": recommendations,
                "total_recommendations": len(recommendations),

                # Confidence
                "overall_confidence": overall_confidence,

                # Summary
                "summary": self._generate_diagnosis_summary(
                    ml_predictions, priority, recommendations
                ),

                # Integration with Data Analysis
                "data_analysis_summary": {
                    "risk_score": analysis_result.get("risk_score", 0),
                    "anomalies_detected": analysis_result.get("anomalies_detected", 0),
                    "early_warnings": analysis_result.get("early_warnings", [])
                }
            }

            self.log_activity(
                f"Diagnosis complete - Priority: {priority}, "
                f"Recommendations: {len(recommendations)}, "
                f"Confidence: {overall_confidence:.2%}"
            )

            return self.create_response(
                success=True,
                data=diagnosis_result,
                next_action="customer_engagement" if diagnosis_result["requires_immediate_service"] else "monitor"
            )

        except Exception as e:
            self.log_activity(f"Diagnosis failed: {str(e)}", level="ERROR")
            return self.create_response(
                success=False,
                data={},
                error=str(e)
            )

    def _generate_diagnosis_summary(self, ml_predictions: Dict,
                                   priority: str, recommendations: List[Dict]) -> str:
        """Generate human-readable diagnosis summary"""

        issues = []

        if ml_predictions['engine']['failure_predicted']:
            prob = ml_predictions['engine']['failure_probability']
            issues.append(f"Engine failure risk ({prob:.0%} probability)")

        if ml_predictions['brake']['failure_predicted']:
            prob = ml_predictions['brake']['failure_probability']
            issues.append(f"Brake system issues ({prob:.0%} probability)")

        if ml_predictions['battery']['failure_predicted']:
            prob = ml_predictions['battery']['failure_probability']
            issues.append(f"Battery problems ({prob:.0%} probability)")

        if issues:
            summary = f"ML models predict: {', '.join(issues)}. "
            summary += f"Priority: {priority.upper()}. "
            summary += f"{len(recommendations)} service recommendation(s) generated."
        else:
            summary = f"No immediate failures predicted. Priority: {priority.upper()}. Vehicle monitoring continues."

        return summary

    def get_prompt(self) -> str:
        """Not used - this agent uses ML models"""
        return "ML-based diagnosis agent"


# Test script
if __name__ == "__main__":
    print("Testing Diagnosis Agent V2...")
    print("\nNote: Make sure ML models are trained first!")
    print("Run: python models/train_models.py")

Testing Diagnosis Agent V2...

Note: Make sure ML models are trained first!
Run: python models/train_models.py


In [21]:
"""
Voice Call Simulator - Simulates AI-powered voice calls to customers.
In production, this would integrate with Twilio.
"""

import time
from datetime import datetime
from typing import Dict, List, Optional
from enum import Enum
import random


class CallStatus(str, Enum):
    """Voice call status"""
    INITIATED = "initiated"
    RINGING = "ringing"
    ANSWERED = "answered"
    BUSY = "busy"
    NO_ANSWER = "no_answer"
    COMPLETED = "completed"
    FAILED = "failed"


class CustomerSentiment(str, Enum):
    """Customer sentiment during call"""
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    CONCERNED = "concerned"
    SKEPTICAL = "skeptical"
    FRUSTRATED = "frustrated"


class VoiceCallSimulator:
    """
    Simulates voice calls to customers.

    In production, this would:
    - Use Twilio/AWS Connect for actual calls
    - Integrate with Speech-to-Text (Deepgram/Google)
    - Use Text-to-Speech (ElevenLabs/Azure)
    - Real-time conversation AI
    """

    def __init__(self):
        self.call_history = []

    def initiate_call(self, customer_phone: str, customer_name: str,
                     call_script: str, diagnosis_summary: str) -> Dict:
        """
        Initiate a voice call to the customer.

        Args:
            customer_phone: Customer's phone number
            customer_name: Customer's name
            call_script: Prepared conversation script
            diagnosis_summary: Summary of vehicle issues

        Returns:
            Call result with conversation transcript
        """

        print(f"\n{'='*80}")
        print(f" INITIATING VOICE CALL")
        print(f"{'='*80}")
        print(f"To: {customer_name} ({customer_phone})")
        print(f"Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"\n{'='*80}")

        # Simulate call connection
        call_status = self._simulate_call_connection()

        if call_status not in [CallStatus.ANSWERED]:
            return self._handle_unanswered_call(customer_phone, customer_name, call_status)

        # Simulate conversation
        conversation = self._simulate_conversation(
            customer_name, call_script, diagnosis_summary
        )

        call_record = {
            "call_id": f"CALL-{datetime.now().strftime('%Y%m%d%H%M%S')}-{random.randint(1000,9999)}",
            "customer_phone": customer_phone,
            "customer_name": customer_name,
            "call_timestamp": datetime.now().isoformat(),
            "call_status": CallStatus.COMPLETED.value,
            "call_duration_seconds": conversation["duration_seconds"],
            "conversation_transcript": conversation["transcript"],
            "customer_response": conversation["customer_response"],
            "customer_sentiment": conversation["sentiment"].value,
            "outcome": conversation["outcome"],
            "appointment_interest": conversation["appointment_interest"],
            "follow_up_required": conversation["follow_up_required"]
        }

        self.call_history.append(call_record)

        return call_record

    def _simulate_call_connection(self) -> CallStatus:
        """Simulate call connection attempt"""

        print("Connecting call...", end="", flush=True)
        time.sleep(0.5)
        print(" 📡", end="", flush=True)
        time.sleep(0.5)
        print(" 🔊", end="", flush=True)
        time.sleep(0.5)

        # Simulate connection outcomes (weighted probabilities)
        outcomes = [
            CallStatus.ANSWERED,      # 70% chance
            CallStatus.ANSWERED,
            CallStatus.ANSWERED,
            CallStatus.ANSWERED,
            CallStatus.ANSWERED,
            CallStatus.ANSWERED,
            CallStatus.ANSWERED,
            CallStatus.NO_ANSWER,     # 20% chance
            CallStatus.NO_ANSWER,
            CallStatus.BUSY           # 10% chance
        ]

        status = random.choice(outcomes)

        if status == CallStatus.ANSWERED:
            print("  Connected!\n")
        elif status == CallStatus.NO_ANSWER:
            print("  No answer\n")
        else:
            print("  Line busy\n")

        return status

    def _handle_unanswered_call(self, phone: str, name: str, status: CallStatus) -> Dict:
        """Handle unanswered call scenarios"""

        print(f"\n Call not answered - Status: {status.value}")
        print(" Triggering fallback: SMS + App Notification")

        return {
            "call_id": f"CALL-{datetime.now().strftime('%Y%m%d%H%M%S')}-{random.randint(1000,9999)}",
            "customer_phone": phone,
            "customer_name": name,
            "call_timestamp": datetime.now().isoformat(),
            "call_status": status.value,
            "call_duration_seconds": 0,
            "conversation_transcript": [],
            "customer_response": "unreachable",
            "customer_sentiment": CustomerSentiment.NEUTRAL.value,
            "outcome": "no_contact",
            "appointment_interest": False,
            "follow_up_required": True,
            "fallback_triggered": True
        }

    def _simulate_conversation(self, customer_name: str, call_script: str,
                              diagnosis_summary: str) -> Dict:
        """
        Simulate full conversation between AI agent and customer.
        """

        transcript = []
        duration = 0

        # Opening
        transcript.append({
            "speaker": "AI_AGENT",
            "timestamp": duration,
            "text": f"Hello, {customer_name}! This is Maya from your vehicle service center. I hope you're doing well today. Do you have a couple of minutes to talk about your vehicle?",
            "tone": "friendly"
        })
        duration += 5

        # Customer response (simulated)
        customer_mood = random.choice([
            CustomerSentiment.POSITIVE,
            CustomerSentiment.NEUTRAL,
            CustomerSentiment.CONCERNED
        ])

        if customer_mood == CustomerSentiment.POSITIVE:
            transcript.append({
                "speaker": "CUSTOMER",
                "timestamp": duration,
                "text": "Hi! Yes, sure. What's this about?",
                "sentiment": "positive"
            })
        elif customer_mood == CustomerSentiment.NEUTRAL:
            transcript.append({
                "speaker": "CUSTOMER",
                "timestamp": duration,
                "text": "Okay... what is it?",
                "sentiment": "neutral"
            })
        else:
            transcript.append({
                "speaker": "CUSTOMER",
                "timestamp": duration,
                "text": "Is something wrong with my car?",
                "sentiment": "concerned"
            })

        duration += 3

        # Explain the issue
        transcript.append({
            "speaker": "AI_AGENT",
            "timestamp": duration,
            "text": f"Thank you for taking my call. Our smart monitoring system has been keeping an eye on your vehicle's health. {diagnosis_summary} I wanted to reach out proactively before this becomes a bigger problem.",
            "tone": "informative"
        })
        duration += 12

        # Customer reaction
        if "critical" in diagnosis_summary.lower() or "failure" in diagnosis_summary.lower():
            transcript.append({
                "speaker": "CUSTOMER",
                "timestamp": duration,
                "text": "Oh, that sounds serious. What should I do?",
                "sentiment": "concerned"
            })
            customer_interest = 0.85  # High interest
        else:
            transcript.append({
                "speaker": "CUSTOMER",
                "timestamp": duration,
                "text": "Hmm, but the car seems to be running fine...",
                "sentiment": "skeptical"
            })
            customer_interest = 0.50  # Medium interest

        duration += 4

        # Address concerns and provide solution
        transcript.append({
            "speaker": "AI_AGENT",
            "timestamp": duration,
            "text": "I understand your concern. That's exactly why we use predictive monitoring - to catch issues before you feel them. Let me explain what we found in simple terms... [explains using data]. The good news is that if we address this now, it's a straightforward service. If we wait, the cost could increase significantly, and you might face an unexpected breakdown.",
            "tone": "reassuring"
        })
        duration += 20

        # Value proposition
        transcript.append({
            "speaker": "AI_AGENT",
            "timestamp": duration,
            "text": "We have availability this Saturday at 2 PM at our service center, which is just 5 km from your location. The service will take about 2 hours, and we'll provide a comfortable waiting area with WiFi. We're also offering a complimentary 25-point vehicle health check worth ₹1,500. Would Saturday work for you?",
            "tone": "helpful"
        })
        duration += 15

        # Customer decision (based on issue severity and customer interest)
        decision_threshold = random.random()

        if decision_threshold < customer_interest:
            # Customer agrees
            transcript.append({
                "speaker": "CUSTOMER",
                "timestamp": duration,
                "text": "Okay, Saturday at 2 PM works for me. Better safe than sorry!",
                "sentiment": "positive"
            })
            outcome = "interested"
            appointment_interest = True
            follow_up = False
            final_sentiment = CustomerSentiment.POSITIVE

        elif decision_threshold < customer_interest + 0.25:
            # Customer wants to think
            transcript.append({
                "speaker": "CUSTOMER",
                "timestamp": duration,
                "text": "Let me check my schedule and I'll call back. Can you send me the details?",
                "sentiment": "neutral"
            })
            outcome = "callback"
            appointment_interest = False
            follow_up = True
            final_sentiment = CustomerSentiment.NEUTRAL

        else:
            # Customer declines
            transcript.append({
                "speaker": "CUSTOMER",
                "timestamp": duration,
                "text": "I think I'll wait and see. If something happens, I'll bring it in.",
                "sentiment": "skeptical"
            })
            outcome = "declined"
            appointment_interest = False
            follow_up = True
            final_sentiment = CustomerSentiment.SKEPTICAL

        duration += 4

        # Closing
        if appointment_interest:
            transcript.append({
                "speaker": "AI_AGENT",
                "timestamp": duration,
                "text": f"Perfect! I've reserved Saturday at 2 PM for you. You'll receive a confirmation SMS and app notification shortly with all the details. We'll also send you a reminder 24 hours before. Thank you for trusting us with your vehicle care, {customer_name}. Have a great day!",
                "tone": "friendly"
            })
        else:
            transcript.append({
                "speaker": "AI_AGENT",
                "timestamp": duration,
                "text": f"I completely understand, {customer_name}. I'll send you all the information via SMS and our app. If you change your mind or have any questions, please don't hesitate to reach out. Your vehicle's safety is our priority. Take care!",
                "tone": "understanding"
            })

        duration += 10

        # Print conversation
        self._print_conversation(transcript)

        return {
            "transcript": transcript,
            "duration_seconds": duration,
            "customer_response": outcome,
            "sentiment": final_sentiment,
            "outcome": outcome,
            "appointment_interest": appointment_interest,
            "follow_up_required": follow_up
        }

    def _print_conversation(self, transcript: List[Dict]):
        """Print conversation transcript in a readable format"""

        print("\n" + "─"*80)
        print(" CONVERSATION TRANSCRIPT")
        print("─"*80 + "\n")

        for entry in transcript:
            speaker = entry["speaker"]
            text = entry["text"]

            if speaker == "AI_AGENT":
                print(f" AI Agent (Maya):")
                print(f"   {text}")
                if "tone" in entry:
                    print(f"   [Tone: {entry['tone']}]")
            else:
                print(f"\n Customer:")
                print(f"   {text}")
                if "sentiment" in entry:
                    print(f"   [Sentiment: {entry['sentiment']}]")

            print()

        print("─"*80)

    def get_call_summary(self, call_record: Dict) -> str:
        """Generate human-readable call summary"""

        if call_record["call_status"] != "completed":
            return f"Call not completed - Status: {call_record['call_status']}"

        summary = f"Call Duration: {call_record['call_duration_seconds']} seconds\n"
        summary += f"Outcome: {call_record['outcome'].upper()}\n"
        summary += f"Customer Sentiment: {call_record['customer_sentiment'].upper()}\n"
        summary += f"Appointment Interest: {'Yes ' if call_record['appointment_interest'] else 'No '}\n"
        summary += f"Follow-up Required: {'Yes' if call_record['follow_up_required'] else 'No'}"

        return summary

In [22]:
"""
Mobile App Notification System.
Simulates push notifications and in-app messages.
"""

from datetime import datetime
from typing import Dict, List
from enum import Enum
import random


class NotificationType(str, Enum):
    """Types of notifications"""
    ALERT = "alert"              # Critical alerts
    REMINDER = "reminder"        # Appointment reminders
    INFO = "info"                # General information
    PROMOTION = "promotion"      # Service promotions


class NotificationPriority(str, Enum):
    """Notification priority levels"""
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"


class AppNotificationSystem:
    """
    Mobile app notification system.

    In production, this would integrate with:
    - Firebase Cloud Messaging (FCM)
    - Apple Push Notification Service (APNS)
    - OneSignal, Pusher, etc.
    """

    def __init__(self):
        self.notification_history = []

    def send_notification(self, customer_id: str, title: str, body: str,
                         notification_type: NotificationType,
                         priority: NotificationPriority,
                         data: Dict = None) -> Dict:
        """
        Send push notification to customer's mobile app.

        Args:
            customer_id: Customer identifier
            title: Notification title
            body: Notification message
            notification_type: Type of notification
            priority: Priority level
            data: Additional data payload

        Returns:
            Notification record
        """

        notification = {
            "notification_id": f"NOTIF-{datetime.now().strftime('%Y%m%d%H%M%S')}-{random.randint(1000,9999)}",
            "customer_id": customer_id,
            "timestamp": datetime.now().isoformat(),
            "type": notification_type.value,
            "priority": priority.value,
            "title": title,
            "body": body,
            "data": data or {},
            "delivery_status": "sent",
            "read_status": False
        }

        # Simulate notification delivery
        self._display_notification(notification)

        self.notification_history.append(notification)

        return notification

    def _display_notification(self, notification: Dict):
        """Display notification in console (simulation)"""

        priority_icons = {
            "high": "🔴",
            "medium": "🟡",
            "low": "🟢"
        }

        type_icons = {
            "alert": "⚠️",
            "reminder": "🔔",
            "info": "ℹ️",
            "promotion": "🎁"
        }

        priority_icon = priority_icons.get(notification["priority"], "⚪")
        type_icon = type_icons.get(notification["type"], "📱")

        print(f"\n{'='*80}")
        print(f"📱 MOBILE APP NOTIFICATION")
        print(f"{'='*80}")
        print(f"{type_icon} Type: {notification['type'].upper()} {priority_icon} Priority: {notification['priority'].upper()}")
        print(f"Time: {notification['timestamp']}")
        print(f"\n{notification['title']}")
        print(f"{notification['body']}")

        if notification["data"]:
            print(f"\nAdditional Info:")
            for key, value in notification["data"].items():
                print(f"  • {key}: {value}")

        print(f"\n{'='*80}\n")

    def send_vehicle_alert(self, customer_id: str, vehicle_id: str,
                          alert_message: str, severity: str) -> Dict:
        """Send vehicle health alert"""

        severity_map = {
            "critical": NotificationPriority.HIGH,
            "high": NotificationPriority.HIGH,
            "medium": NotificationPriority.MEDIUM,
            "low": NotificationPriority.LOW
        }

        priority = severity_map.get(severity, NotificationPriority.MEDIUM)

        return self.send_notification(
            customer_id=customer_id,
            title=f" Vehicle Alert - {severity.upper()}",
            body=alert_message,
            notification_type=NotificationType.ALERT,
            priority=priority,
            data={
                "vehicle_id": vehicle_id,
                "severity": severity,
                "action_required": True
            }
        )

    def send_appointment_reminder(self, customer_id: str, appointment_details: Dict) -> Dict:
        """Send appointment reminder"""

        return self.send_notification(
            customer_id=customer_id,
            title=" Upcoming Service Appointment",
            body=f"Reminder: Your service appointment is scheduled for {appointment_details.get('date')} at {appointment_details.get('time')}. Location: {appointment_details.get('location')}",
            notification_type=NotificationType.REMINDER,
            priority=NotificationPriority.MEDIUM,
            data=appointment_details
        )

    def send_service_recommendation(self, customer_id: str, vehicle_id: str,
                                   service_type: str, estimated_cost: float) -> Dict:
        """Send service recommendation notification"""

        return self.send_notification(
            customer_id=customer_id,
            title="🔧 Service Recommendation",
            body=f"Based on your vehicle's health data, we recommend: {service_type}. Estimated cost: ₹{estimated_cost:,.0f}. Tap to schedule an appointment.",
            notification_type=NotificationType.INFO,
            priority=NotificationPriority.MEDIUM,
            data={
                "vehicle_id": vehicle_id,
                "service_type": service_type,
                "estimated_cost": estimated_cost,
                "cta": "schedule_appointment"
            }
        )

    def send_sms_fallback(self, phone_number: str, message: str) -> Dict:
        """Send SMS as fallback when call fails"""

        sms_record = {
            "sms_id": f"SMS-{datetime.now().strftime('%Y%m%d%H%M%S')}-{random.randint(1000,9999)}",
            "phone_number": phone_number,
            "timestamp": datetime.now().isoformat(),
            "message": message,
            "delivery_status": "sent"
        }

        # Display SMS
        print(f"\n{'='*80}")
        print(f" SMS MESSAGE")
        print(f"{'='*80}")
        print(f"To: {phone_number}")
        print(f"Time: {sms_record['timestamp']}")
        print(f"\nMessage:")
        print(f"{message}")
        print(f"\n{'='*80}\n")

        return sms_record

In [23]:
"""
Customer Engagement Agent - Handles all customer communication.

Competition Requirements:
    1. Proactively contact vehicle owners
    2. Personalized maintenance recommendations
    3. Primarily via voice-based agents
    4. Mobile app notifications as secondary channel
    5. Persuasive communication
    6. Support multiple Indian languages
    7. Handle edge cases (declined appointments, urgent alerts)

Communication Channels:
    - Primary: Voice call (AI agent)
    - Secondary: Mobile app push notifications
    - Fallback: SMS
"""

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

class CustomerEngagementAgentV2(BaseAgent):


    def __init__(self):
        super().__init__(agent_name="CustomerEngagementAgentV2")

        self.llm = ChatOpenAI(
            model=llm_model,
            temperature=temperature,
            api_key=os.getenv("OPENAI_API_KEY")
        )

        # Initialize communication systems
        self.voice_call_system = VoiceCallSimulator()
        self.app_notification_system = AppNotificationSystem()

        # Language support
        self.supported_languages = ["English", "Hindi", "Tamil", "Telugu", "Kannada", "Marathi"]

        self.log_activity("Customer Engagement Agent V2 initialized")
        self.log_activity("Voice call system: Ready")
        self.log_activity("App notification system: Ready")
        self.log_activity(f"Language support: {', '.join(self.supported_languages)}")

    def get_prompt(self) -> ChatPromptTemplate:
        """Generate personalized call script"""

        template = """You are an empathetic AI customer service agent (Maya) for a vehicle maintenance service.

        TASK: Create a persuasive but caring phone call script in {language}.

        CUSTOMER INFO:
        - Name: {customer_name}
        - Language: {language}
        - Vehicle: {vehicle_model}
        - Mileage: {mileage} km

        DIAGNOSIS:
        {diagnosis_summary}

        PRIORITY: {priority}
        TIME TO FAILURE: {time_to_failure_days} days
        ESTIMATED COST: ₹{estimated_cost}
        POTENTIAL COST IF DELAYED: ₹{delayed_cost}

        EDGE CASE CONTEXT: {edge_case_context}

        CREATE A CONVERSATIONAL SCRIPT:

        1. WARM OPENING (Build trust, don't alarm)
          - Friendly greeting in {language}
          - Establish credibility
          - Show you care about their vehicle

        2. EXPLAIN THE SITUATION (Simple, clear language)
          - What we detected using predictive AI
          - Why it matters (safety/cost)
          - Use relatable analogy if helpful
          - Be transparent about severity

        3. VALUE PROPOSITION (Make it compelling)
          - Act now: ₹{estimated_cost} + {time_to_failure_days} days window
          - Wait later: ₹{delayed_cost} + potential breakdown
          - Safety aspect for family
          - Convenience factors (location, timing, complimentary services)

        4. HANDLE OBJECTIONS (Anticipate concerns)
          - "Car seems fine" → Explain predictive maintenance
          - "Too expensive" → Compare with future costs
          - "No time" → Offer flexible scheduling

        5. CALL TO ACTION (Make it easy)
          - Specific appointment options
          - Easy confirmation process
          - Reassurance and support

        IMPORTANT:
        - Keep it natural, conversational, under 200 words
        - Be persuasive but not pushy
        - Focus on being helpful and protective
        - If critical/urgent, emphasize safety without panic
        - Show empathy and understanding

        Script:"""

        return ChatPromptTemplate.from_template(template)

    def generate_call_script(self, customer_info: Dict, diagnosis_data: Dict,
                            vehicle_info: Dict, edge_case_type: str = None) -> str:
        """Generate personalized call script using LLM"""

        try:
            prompt = self.get_prompt()
            chain = prompt | self.llm

            # Extract recommendation details
            recommendations = diagnosis_data.get("service_recommendations", [])
            primary_rec = recommendations[0] if recommendations else {}

            time_estimates = diagnosis_data.get("time_to_failure_estimates", {})
            min_days = min(time_estimates.values()) if time_estimates else 14

            # Calculate delayed cost (estimate 2-3x increase)
            estimated_cost = primary_rec.get("estimated_cost_min", 5000)
            delayed_cost = int(estimated_cost * 2.5)

            # Edge case context
            edge_case_context = self._get_edge_case_context(
                edge_case_type, diagnosis_data.get("priority")
            )

            response = chain.invoke({
                "customer_name": customer_info.get("name", "Customer"),
                "language": customer_info.get("language", "English"),
                "vehicle_model": vehicle_info.get("model", "your vehicle"),
                "mileage": vehicle_info.get("mileage", 50000),
                "diagnosis_summary": diagnosis_data.get("summary", "We detected some maintenance needs"),
                "priority": diagnosis_data.get("priority", "medium"),
                "time_to_failure_days": min_days,
                "estimated_cost": estimated_cost,
                "delayed_cost": delayed_cost,
                "edge_case_context": edge_case_context
            })

            return response.content

        except Exception as e:
            self.log_activity(f"Script generation failed: {str(e)}", level="WARNING")
            return self._fallback_script(customer_info, diagnosis_data)

    def _get_edge_case_context(self, edge_case_type: str, priority: str) -> str:
        """Generate context for edge case scenarios"""

        if edge_case_type == "critical_urgent":
            return "CRITICAL: Immediate safety risk. Component may fail within 48 hours. Emphasize urgency and safety."
        elif edge_case_type == "fleet_vehicle":
            return "Fleet vehicle detected. Offer batch scheduling and fleet discount."
        elif edge_case_type == "previous_decline":
            return "Customer declined previously. Be extra empathetic, address previous concerns."
        elif edge_case_type == "recurring_defect":
            return "Recurring issue detected. Mention manufacturing feedback loop and permanent solution."
        elif priority == "critical":
            return "High priority issue. Emphasize proactive prevention over reactive breakdown."
        else:
            return "Standard maintenance recommendation. Focus on value and convenience."

    def _fallback_script(self, customer_info: Dict, diagnosis_data: Dict) -> str:
        """Fallback script if LLM fails"""

        name = customer_info.get("name", "Customer")
        summary = diagnosis_data.get("summary", "maintenance needs detected")
        language = customer_info.get("language", "English")

        # Basic multilingual greetings
        greetings = {
            "Hindi": f"नमस्ते {name}, मैं आपकी वाहन सेवा केंद्र से माया बोल रही हूं।",
            "Tamil": f"வணக்கம் {name}, நான் உங்கள் வாகன சேவை மையத்தில் இருந்து மாயா.",
            "Telugu": f"నమస్కారం {name}, నేను మీ వాహన సేవా కేంద్రం నుండి మాయా.",
            "Kannada": f"ನಮಸ್ಕಾರ {name}, ನಾನು ನಿಮ್ಮ ವಾಹನ ಸೇವಾ ಕೇಂದ್ರದಿಂದ ಮಾಯಾ.",
            "Marathi": f"नमस्कार {name}, मी तुमच्या वाहन सेवा केंद्राकडून माया आहे.",
            "English": f"Hello {name}, this is Maya from your vehicle service center."
        }

        greeting = greetings.get(language, greetings["English"])

        return f"""{greeting}

Our AI monitoring system detected: {summary}

I wanted to reach out proactively before this becomes a bigger issue. We have appointments available this week. Would you like to schedule a service to address this?

We're here to help ensure your vehicle stays safe and reliable."""

    def process(self, input_data: Dict[str, Any]) -> AgentResponse:
        """
        Main engagement process.

        Args:
            input_data: Must contain:
                - diagnosis_result: From Diagnosis Agent
                - customer_info: Customer details
                - vehicle_info: Vehicle details
                - edge_case_type: Optional edge case identifier

        Returns:
            AgentResponse with engagement results for Scheduling Agent
        """

        try:
            self.log_activity("="*80)
            self.log_activity("STARTING CUSTOMER ENGAGEMENT PROCESS")
            self.log_activity("="*80)

            # Extract data
            diagnosis_data = input_data.get("diagnosis_result", {})
            customer_info = input_data.get("customer_info", {})
            vehicle_info = input_data.get("vehicle_info", {})
            edge_case_type = input_data.get("edge_case_type", None)

            customer_id = customer_info.get("customer_id", "UNKNOWN")
            customer_name = customer_info.get("name", "Customer")
            customer_phone = customer_info.get("phone", "+91-XXXXXXXXXX")
            vehicle_id = vehicle_info.get("vehicle_id", "UNKNOWN")
            priority = diagnosis_data.get("priority", "medium")

            self.log_activity(f"Customer: {customer_name} ({customer_id})")
            self.log_activity(f"Vehicle: {vehicle_id}")
            self.log_activity(f"Priority: {priority.upper()}")
            if edge_case_type:
                self.log_activity(f"Edge Case: {edge_case_type}")

            # Step 1: Send initial app notification
            self.log_activity("\n[STEP 1] Sending initial app notification...")
            initial_notification = self._send_initial_notification(
                customer_id, vehicle_id, diagnosis_data, edge_case_type
            )

            # Step 2: Generate personalized call script
            self.log_activity("\n[STEP 2] Generating personalized call script...")
            call_script = self.generate_call_script(
                customer_info, diagnosis_data, vehicle_info, edge_case_type
            )
            self.log_activity(f"Script generated in {customer_info.get('language', 'English')}")

            # Step 3: Initiate voice call (PRIMARY CHANNEL)
            self.log_activity(f"\n[STEP 3] Initiating voice call to {customer_name}...")
            call_record = self.voice_call_system.initiate_call(
                customer_phone=customer_phone,
                customer_name=customer_name,
                call_script=call_script,
                diagnosis_summary=diagnosis_data.get("summary", "maintenance needed")
            )

            # Step 4: Handle call outcome with appropriate follow-up
            self.log_activity("\n[STEP 4] Processing call outcome...")
            outcome_data = self._handle_call_outcome(
                call_record, customer_id, vehicle_id, diagnosis_data,
                customer_info, edge_case_type
            )

            # Step 5: Compile engagement result for Scheduling Agent
            engagement_result = self._compile_engagement_result(
                call_record, outcome_data, customer_id, customer_info,
                vehicle_id, vehicle_info, diagnosis_data, call_script,
                initial_notification, edge_case_type
            )

            self.log_activity("\n" + "="*80)
            self.log_activity("ENGAGEMENT COMPLETE")
            self.log_activity(f"Outcome: {call_record['customer_response'].upper()}")
            self.log_activity(f"Appointment Interest: {'YES ' if engagement_result['appointment_interest'] else 'NO ❌'}")
            self.log_activity(f"Next Action: {engagement_result['next_action'].upper()}")
            self.log_activity("="*80)

            return self.create_response(
                success=True,
                data=engagement_result,
                next_action=engagement_result['next_action']
            )

        except Exception as e:
            self.log_activity(f"Engagement failed: {str(e)}", level="ERROR")
            return self.create_response(
                success=False,
                data={},
                error=str(e)
            )

    def _send_initial_notification(self, customer_id: str, vehicle_id: str,
                                   diagnosis_data: Dict, edge_case_type: str = None) -> Dict:
        """Send initial app notification before call"""

        priority = diagnosis_data.get("priority", "medium")

        # Critical/urgent cases
        if priority == "critical" or edge_case_type == "critical_urgent":
            message = " URGENT: Critical vehicle alert detected. We'll be calling you immediately to discuss safety concerns."
            notif_priority = NotificationPriority.HIGH

        # Recurring defect case
        elif edge_case_type == "recurring_defect":
            message = "🔧 Important: We've detected a recurring issue with your vehicle. Our team will call you to discuss a permanent solution."
            notif_priority = NotificationPriority.HIGH

        # Fleet vehicle case
        elif edge_case_type == "fleet_vehicle":
            message = " Fleet Vehicle Alert: Maintenance needed. We'll call to discuss convenient batch scheduling options."
            notif_priority = NotificationPriority.MEDIUM

        # Previous decline case
        elif edge_case_type == "previous_decline":
            message = " Follow-up: The previously identified issue needs attention. We'd like to discuss updated options with you."
            notif_priority = NotificationPriority.MEDIUM

        # Standard case
        else:
            message = "Our predictive monitoring system detected maintenance needs for your vehicle. Expect a call from us soon."
            notif_priority = NotificationPriority.MEDIUM

        return self.app_notification_system.send_notification(
            customer_id=customer_id,
            title="Vehicle Health Alert",
            body=message,
            notification_type=NotificationType.ALERT,
            priority=notif_priority,
            data={
                "vehicle_id": vehicle_id,
                "priority": priority,
                "edge_case": edge_case_type
            }
        )

    def _handle_call_outcome(self, call_record: Dict, customer_id: str,
                            vehicle_id: str, diagnosis_data: Dict,
                            customer_info: Dict, edge_case_type: str = None) -> Dict:
        """Handle different call outcomes with appropriate follow-up"""

        outcome_data = {
            "notifications": [],
            "sms_record": None,
            "follow_up_channel": None,
            "follow_up_scheduled": None,
            "escalation_required": False
        }

        call_status = call_record["call_status"]
        customer_response = call_record["customer_response"]
        priority = diagnosis_data.get("priority", "medium")

        # EDGE CASE 1: Call not answered (No answer / Busy)
        if call_status in ["no_answer", "busy"]:
            self.log_activity(f" Call not answered - Status: {call_status}")
            self.log_activity("Triggering fallback communication channels...")

            # Send SMS fallback
            sms_message = self._generate_sms_message(
                customer_info, diagnosis_data, edge_case_type
            )
            outcome_data["sms_record"] = self.app_notification_system.send_sms_fallback(
                call_record["customer_phone"], sms_message
            )

            # Send detailed app notification
            notif = self.app_notification_system.send_service_recommendation(
                customer_id=customer_id,
                vehicle_id=vehicle_id,
                service_type=diagnosis_data.get("summary", "Maintenance required"),
                estimated_cost=diagnosis_data.get("service_recommendations", [{}])[0].get("estimated_cost_min", 5000)
            )
            outcome_data["notifications"].append(notif)
            outcome_data["follow_up_channel"] = "sms_and_app"

            # Schedule retry based on priority
            if priority == "critical" or edge_case_type == "critical_urgent":
                outcome_data["follow_up_scheduled"] = "retry_in_2_hours"
                outcome_data["escalation_required"] = True
                self.log_activity(" CRITICAL: Scheduling immediate retry in 2 hours")
            else:
                outcome_data["follow_up_scheduled"] = "retry_in_24_hours"
                self.log_activity("Scheduling retry call in 24 hours")

        # EDGE CASE 2: Customer interested in appointment
        elif customer_response == "interested":
            self.log_activity(" Customer interested in appointment!")

            # Send confirmation notification
            notif = self.app_notification_system.send_notification(
                customer_id=customer_id,
                title=" Appointment Request Received",
                body="Great! We're processing your service appointment. You'll receive confirmation details shortly.",
                notification_type=NotificationType.INFO,
                priority=NotificationPriority.MEDIUM,
                data={
                    "vehicle_id": vehicle_id,
                    "status": "pending_scheduling",
                    "action": "await_confirmation"
                }
            )
            outcome_data["notifications"].append(notif)
            outcome_data["follow_up_channel"] = "app_notification"

        # EDGE CASE 3: Customer wants to think / callback
        elif customer_response == "callback":
            self.log_activity(" Customer wants time to decide")

            # Send summary notification with details
            recommendations = diagnosis_data.get("service_recommendations", [])
            primary_rec = recommendations[0] if recommendations else {}

            notif = self.app_notification_system.send_notification(
                customer_id=customer_id,
                title=" Service Details - As Requested",
                body=f"Service: {primary_rec.get('service_type', 'Maintenance')}. Estimated cost: ₹{primary_rec.get('estimated_cost_min', 5000):,}. Tap to schedule when ready.",
                notification_type=NotificationType.INFO,
                priority=NotificationPriority.MEDIUM,
                data={
                    "vehicle_id": vehicle_id,
                    "recommendations": recommendations,
                    "cta": "schedule_appointment"
                }
            )
            outcome_data["notifications"].append(notif)

            # Schedule follow-up based on urgency
            if priority == "critical":
                outcome_data["follow_up_scheduled"] = "follow_up_in_48_hours"
                outcome_data["follow_up_channel"] = "voice_call"
                self.log_activity("Critical issue - scheduling follow-up in 48 hours")
            else:
                outcome_data["follow_up_scheduled"] = "follow_up_in_7_days"
                outcome_data["follow_up_channel"] = "app_notification"
                self.log_activity("Scheduling gentle reminder in 7 days")

        # EDGE CASE 4: Customer declined appointment
        elif customer_response == "declined":
            self.log_activity(" Customer declined appointment")

            # Send educational notification (non-pushy)
            if priority == "critical" or edge_case_type == "critical_urgent":
                # For critical issues, send safety reminder
                notif = self.app_notification_system.send_notification(
                    customer_id=customer_id,
                    title=" Safety Reminder",
                    body="We respect your decision. However, this is a safety-critical issue. We'll check in again soon. Your safety is our priority.",
                    notification_type=NotificationType.INFO,
                    priority=NotificationPriority.HIGH,
                    data={
                        "vehicle_id": vehicle_id,
                        "issue_type": "safety_critical",
                        "declined": True
                    }
                )
                outcome_data["follow_up_scheduled"] = "follow_up_in_72_hours"
                outcome_data["escalation_required"] = True
                self.log_activity(" SAFETY CONCERN: Declined critical issue - escalation required")

            else:
                # For non-critical, gentle reminder
                notif = self.app_notification_system.send_notification(
                    customer_id=customer_id,
                    title=" We're Here When You Need Us",
                    body="No problem! We've saved the diagnosis details in your app. Feel free to schedule anytime.",
                    notification_type=NotificationType.INFO,
                    priority=NotificationPriority.LOW,
                    data={
                        "vehicle_id": vehicle_id,
                        "diagnosis_saved": True
                    }
                )
                outcome_data["follow_up_scheduled"] = "follow_up_in_30_days"
                self.log_activity("Customer declined - respecting decision, gentle follow-up in 30 days")

            outcome_data["notifications"].append(notif)
            outcome_data["follow_up_channel"] = "app_notification"

        # Handle edge case specific follow-ups
        if edge_case_type:
            self._handle_edge_case_follow_up(
                edge_case_type, outcome_data, customer_id, vehicle_id, call_record
            )

        return outcome_data

    def _handle_edge_case_follow_up(self, edge_case_type: str, outcome_data: Dict,
                                    customer_id: str, vehicle_id: str, call_record: Dict):
        """Handle specific edge case scenarios"""

        # Fleet vehicle - offer batch scheduling
        if edge_case_type == "fleet_vehicle":
            notif = self.app_notification_system.send_notification(
                customer_id=customer_id,
                title=" Fleet Service Options",
                body="As a fleet customer, we offer convenient batch scheduling and volume discounts. Contact your fleet manager for details.",
                notification_type=NotificationType.PROMOTION,
                priority=NotificationPriority.MEDIUM,
                data={
                    "vehicle_id": vehicle_id,
                    "fleet_benefits": True,
                    "batch_scheduling": True
                }
            )
            outcome_data["notifications"].append(notif)
            self.log_activity("Fleet vehicle detected - sent batch scheduling options")

        # Recurring defect - manufacturing feedback
        elif edge_case_type == "recurring_defect":
            notif = self.app_notification_system.send_notification(
                customer_id=customer_id,
                title="🔧 Permanent Solution Available",
                body="We've identified this as a recurring issue and have implemented a permanent fix. This service includes an upgraded component at no extra cost.",
                notification_type=NotificationType.INFO,
                priority=NotificationPriority.HIGH,
                data={
                    "vehicle_id": vehicle_id,
                    "recurring_defect": True,
                    "permanent_solution": True,
                    "no_extra_cost": True
                }
            )
            outcome_data["notifications"].append(notif)
            outcome_data["manufacturing_feedback_triggered"] = True
            self.log_activity("Recurring defect case - manufacturing feedback loop activated")

    def _generate_sms_message(self, customer_info: Dict, diagnosis_data: Dict,
                             edge_case_type: str = None) -> str:
        """Generate SMS fallback message"""

        name = customer_info.get("name", "Customer")
        priority = diagnosis_data.get("priority", "medium")

        if priority == "critical" or edge_case_type == "critical_urgent":
            return f"""URGENT - {name},

Your vehicle needs immediate attention. Our predictive system detected a critical issue: {diagnosis_data.get('summary', 'safety concern')}.

Please call us at 1800-XXX-XXXX or check our app for details.

- Your Vehicle Service Team"""

        else:
            return f"""Hi {name},

We tried calling about your vehicle maintenance. Our AI system detected: {diagnosis_data.get('summary', 'service needs')}.

Check our app for details or call us at 1800-XXX-XXXX to schedule.

- Your Vehicle Service Team"""

    def _compile_engagement_result(self, call_record: Dict, outcome_data: Dict,
                                   customer_id: str, customer_info: Dict,
                                   vehicle_id: str, vehicle_info: Dict,
                                   diagnosis_data: Dict, call_script: str,
                                   initial_notification: Dict,
                                   edge_case_type: str = None) -> Dict:
        """Compile comprehensive engagement result for downstream agents"""

        # Determine next action
        if call_record.get("appointment_interest", False):
            next_action = "schedule_appointment"
        elif outcome_data.get("escalation_required", False):
            next_action = "escalate_to_supervisor"
        elif call_record.get("follow_up_required", False):
            next_action = "schedule_follow_up"
        else:
            next_action = "close_engagement"

        return {
            # Identifiers
            "engagement_id": f"ENG-{uuid.uuid4().hex[:8]}",
            "vehicle_id": vehicle_id,
            "customer_id": customer_id,
            "timestamp": datetime.now().isoformat(),

            # Customer context (for scheduling agent)
            "customer_context": {
                "name": customer_info.get("name"),
                "phone": customer_info.get("phone"),
                "language": customer_info.get("language", "English"),
                "location": customer_info.get("location", "Unknown"),
                "preferred_service_center": customer_info.get("preferred_service_center")
            },

            # Vehicle context
            "vehicle_context": {
                "vehicle_id": vehicle_id,
                "model": vehicle_info.get("model"),
                "mileage": vehicle_info.get("mileage"),
                "last_service_date": vehicle_info.get("last_service_date")
            },

            # Diagnosis summary (for scheduling agent)
            "diagnosis_summary": {
                "priority": diagnosis_data.get("priority"),
                "summary": diagnosis_data.get("summary"),
                "service_recommendations": diagnosis_data.get("service_recommendations", []),
                "estimated_duration_hours": diagnosis_data.get("service_recommendations", [{}])[0].get("estimated_duration_hours", 2),
                "time_to_failure_days": min(diagnosis_data.get("time_to_failure_estimates", {30: 30}).values())
            },

            # Communication records
            "communication_records": {
                "primary_channel": "voice_call",
                "call_record": call_record,
                "initial_notification": initial_notification,
                "follow_up_notifications": outcome_data.get("notifications", []),
                "sms_record": outcome_data.get("sms_record"),
                "total_touchpoints": 1 + len(outcome_data.get("notifications", [])) + (1 if outcome_data.get("sms_record") else 0)
            },

            # Outcome analysis
            "outcome": {
                "call_status": call_record["call_status"],
                "customer_response": call_record["customer_response"],
                "customer_sentiment": call_record.get("customer_sentiment", "neutral"),
                "appointment_interest": call_record.get("appointment_interest", False),
                "conversation_duration_seconds": call_record.get("call_duration_seconds", 0)
            },

            # Next steps (structured for scheduling agent)
            "next_steps": {
                "action": next_action,
                "needs_scheduling": call_record.get("appointment_interest", False),
                "follow_up_required": call_record.get("follow_up_required", False),
                "follow_up_channel": outcome_data.get("follow_up_channel"),
                "follow_up_scheduled": outcome_data.get("follow_up_scheduled"),
                "escalation_required": outcome_data.get("escalation_required", False),
                "manufacturing_feedback": outcome_data.get("manufacturing_feedback_triggered", False)
            },

            # Edge case tracking
            "edge_case_info": {
                "type": edge_case_type,
                "handled": True,
                "special_actions_taken": self._get_edge_case_actions(edge_case_type)
            },

            # Scripts and summaries
            "scripts": {
                "generated_call_script": call_script,
                "conversation_summary": self.voice_call_system.get_call_summary(call_record),
                "language": customer_info.get("language", "English")
            },

            # Metadata for master agent
            "metadata": {
                "agent_name": self.agent_name,
                "engagement_type": "proactive_predictive",
                "success": call_record["call_status"] == "completed",
                "requires_human_intervention": outcome_data.get("escalation_required", False)
            }
        }

    def _get_edge_case_actions(self, edge_case_type: str) -> List[str]:
        """Document actions taken for edge cases"""

        actions_map = {
            "critical_urgent": [
                "Immediate notification sent",
                "Urgent callback scheduled within 2 hours",
                "Safety alert escalation triggered"
            ],
            "fleet_vehicle": [
                "Batch scheduling options provided",
                "Fleet discount information sent",
                "Fleet manager notification triggered"
            ],
            "previous_decline": [
                "Empathetic approach used",
                "Addressed previous concerns",
                "Updated cost/benefit analysis provided"
            ],
            "recurring_defect": [
                "Manufacturing feedback loop activated",
                "Permanent solution offered",
                "No-cost upgrade notification sent",
                "RCA/CAPA analysis triggered"
            ]
        }

        return actions_map.get(edge_case_type, ["Standard engagement flow"])

    def handle_multi_vehicle_fleet(self, fleet_data: Dict[str, Any]) -> Dict:
        """
        Handle edge case: Multi-vehicle fleet scheduling

        Args:
            fleet_data: Contains multiple vehicles needing service

        Returns:
            Batch engagement result
        """

        self.log_activity("\n" + "="*80)
        self.log_activity("FLEET ENGAGEMENT - BATCH PROCESSING")
        self.log_activity("="*80)

        fleet_results = {
            "engagement_id": f"FLEET-ENG-{uuid.uuid4().hex[:8]}",
            "fleet_id": fleet_data.get("fleet_id"),
            "fleet_owner": fleet_data.get("fleet_owner"),
            "timestamp": datetime.now().isoformat(),
            "vehicles_processed": [],
            "batch_scheduling_recommended": True,
            "total_estimated_cost": 0,
            "recommended_schedule": []
        }

        vehicles = fleet_data.get("vehicles", [])
        self.log_activity(f"Processing {len(vehicles)} fleet vehicles")

        for vehicle_data in vehicles:
            self.log_activity(f"\nProcessing vehicle: {vehicle_data.get('vehicle_id')}")

            # Process each vehicle
            result = self.process({
                "diagnosis_result": vehicle_data.get("diagnosis_result"),
                "customer_info": fleet_data.get("fleet_owner_info"),
                "vehicle_info": vehicle_data.get("vehicle_info"),
                "edge_case_type": "fleet_vehicle"
            })

            fleet_results["vehicles_processed"].append(result.data)

            # Aggregate costs
            if result.data.get("diagnosis_summary"):
                recommendations = result.data["diagnosis_summary"].get("service_recommendations", [])
                if recommendations:
                    fleet_results["total_estimated_cost"] += recommendations[0].get("estimated_cost_min", 0)

        # Send consolidated fleet notification
        customer_id = fleet_data.get("fleet_owner_info", {}).get("customer_id")

        fleet_notif = self.app_notification_system.send_notification(
            customer_id=customer_id,
            title=f"🚛 Fleet Maintenance Report - {len(vehicles)} Vehicles",
            body=f"We've analyzed your fleet. {len(vehicles)} vehicles need service. Total estimated cost: ₹{fleet_results['total_estimated_cost']:,}. Batch scheduling available with 15% fleet discount.",
            notification_type=NotificationType.INFO,
            priority=NotificationPriority.HIGH,
            data={
                "fleet_id": fleet_data.get("fleet_id"),
                "vehicles_count": len(vehicles),
                "batch_discount": "15%",
                "cta": "schedule_fleet_service"
            }
        )

        fleet_results["fleet_notification"] = fleet_notif
        fleet_results["fleet_discount_applicable"] = "15%"

        self.log_activity(f"\n✅ Fleet engagement complete - {len(vehicles)} vehicles processed")
        self.log_activity(f"Total cost: ₹{fleet_results['total_estimated_cost']:,} (before 15% fleet discount)")

        return fleet_results

    def handle_urgent_failure_alert(self, input_data: Dict[str, Any]) -> AgentResponse:
        """
        Handle edge case: Urgent/critical failure detected
        Requires immediate action within hours

        Args:
            input_data: Contains critical diagnosis with urgent timeline

        Returns:
            AgentResponse with escalation
        """

        self.log_activity("\n" + "🚨"*20)
        self.log_activity("CRITICAL URGENT FAILURE ALERT")
        self.log_activity("🚨"*20)

        # Mark as critical urgent edge case
        input_data["edge_case_type"] = "critical_urgent"

        # Standard engagement with urgency context
        result = self.process(input_data)

        # Additional urgent actions
        customer_id = input_data.get("customer_info", {}).get("customer_id")
        vehicle_id = input_data.get("vehicle_info", {}).get("vehicle_id")

        # Send multiple urgent notifications
        urgent_notif_1 = self.app_notification_system.send_notification(
            customer_id=customer_id,
            title="🚨 CRITICAL VEHICLE ALERT",
            body="IMMEDIATE ACTION REQUIRED: Your vehicle has a critical safety issue. DO NOT DRIVE until serviced. We're calling you now.",
            notification_type=NotificationType.ALERT,
            priority=NotificationPriority.HIGH,
            data={
                "vehicle_id": vehicle_id,
                "severity": "critical",
                "do_not_drive": True,
                "immediate_action": True
            }
        )

        # If call failed, escalate immediately
        if result.data.get("outcome", {}).get("call_status") != "completed":
            self.log_activity(" ESCALATION: Critical issue, customer unreachable")
            self.log_activity("Triggering supervisor intervention...")

            result.data["escalation"] = {
                "reason": "critical_failure_unreachable",
                "escalated_to": "supervisor",
                "escalation_time": datetime.now().isoformat(),
                "requires_manual_intervention": True
            }

        result.data["urgent_notifications"] = [urgent_notif_1]

        return result

    def handle_declined_appointment_follow_up(self, previous_engagement: Dict,
                                             days_since_decline: int) -> AgentResponse:
        """
        Handle edge case: Customer previously declined, follow-up required

        Args:
            previous_engagement: Previous engagement record
            days_since_decline: Days since customer declined

        Returns:
            Follow-up engagement result
        """

        self.log_activity("\n" + "="*80)
        self.log_activity("FOLLOW-UP: PREVIOUS DECLINE")
        self.log_activity("="*80)
        self.log_activity(f"Days since decline: {days_since_decline}")

        # Reconstruct input with context
        input_data = {
            "diagnosis_result": previous_engagement.get("diagnosis_summary"),
            "customer_info": previous_engagement.get("customer_context"),
            "vehicle_info": previous_engagement.get("vehicle_context"),
            "edge_case_type": "previous_decline",
            "previous_decline_info": {
                "decline_date": previous_engagement.get("timestamp"),
                "days_elapsed": days_since_decline,
                "previous_response": previous_engagement.get("outcome", {}).get("customer_response")
            }
        }

        # More empathetic, less pushy approach
        self.log_activity("Using empathetic follow-up approach...")

        result = self.process(input_data)

        # Track follow-up
        result.data["follow_up_context"] = {
            "is_follow_up": True,
            "follow_up_attempt": "second_contact",
            "previous_engagement_id": previous_engagement.get("engagement_id"),
            "approach": "empathetic_non_pushy"
        }

        return result

    def generate_engagement_report(self, engagement_results: List[Dict]) -> Dict:
        """
        Generate comprehensive engagement report for analytics

        Args:
            engagement_results: List of engagement records

        Returns:
            Analytics report
        """

        total_engagements = len(engagement_results)

        # Calculate metrics
        successful_calls = sum(1 for e in engagement_results
                              if e.get("outcome", {}).get("call_status") == "completed")

        appointments_scheduled = sum(1 for e in engagement_results
                                    if e.get("outcome", {}).get("appointment_interest", False))

        declined = sum(1 for e in engagement_results
                      if e.get("outcome", {}).get("customer_response") == "declined")

        unreachable = sum(1 for e in engagement_results
                         if e.get("outcome", {}).get("call_status") in ["no_answer", "busy"])

        # Sentiment analysis
        sentiment_counts = {
            "positive": 0,
            "neutral": 0,
            "concerned": 0,
            "skeptical": 0,
            "frustrated": 0
        }

        for e in engagement_results:
            sentiment = e.get("outcome", {}).get("customer_sentiment", "neutral")
            sentiment_counts[sentiment] = sentiment_counts.get(sentiment, 0) + 1

        # Edge cases handled
        edge_cases = {}
        for e in engagement_results:
            edge_type = e.get("edge_case_info", {}).get("type")
            if edge_type:
                edge_cases[edge_type] = edge_cases.get(edge_type, 0) + 1

        report = {
            "report_id": f"RPT-{uuid.uuid4().hex[:8]}",
            "generated_at": datetime.now().isoformat(),
            "period": "current_session",

            "summary": {
                "total_engagements": total_engagements,
                "successful_calls": successful_calls,
                "call_success_rate": f"{(successful_calls/total_engagements*100):.1f}%" if total_engagements > 0 else "0%",
                "appointments_scheduled": appointments_scheduled,
                "conversion_rate": f"{(appointments_scheduled/total_engagements*100):.1f}%" if total_engagements > 0 else "0%",
                "declined": declined,
                "unreachable": unreachable
            },

            "sentiment_analysis": sentiment_counts,

            "edge_cases_handled": edge_cases,

            "communication_channels": {
                "voice_calls": successful_calls,
                "sms_fallback": unreachable,
                "app_notifications": sum(len(e.get("communication_records", {}).get("follow_up_notifications", []))
                                        for e in engagement_results)
            },

            "follow_up_required": sum(1 for e in engagement_results
                                     if e.get("next_steps", {}).get("follow_up_required", False)),

            "escalations": sum(1 for e in engagement_results
                              if e.get("next_steps", {}).get("escalation_required", False)),

            "manufacturing_feedback_triggers": sum(1 for e in engagement_results
                                                   if e.get("next_steps", {}).get("manufacturing_feedback", False))
        }

        return report

    def get_engagement_analytics(self) -> Dict:
        """Get real-time engagement analytics"""

        return {
            "total_calls_made": len(self.voice_call_system.call_history),
            "total_notifications_sent": len(self.app_notification_system.notification_history),
            "channels_active": ["voice_call", "app_notification", "sms"],
            "languages_supported": self.supported_languages
        }

In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

class SchedulingAgent(BaseAgent):
    """
    Scheduling Agent - Manages appointments and service center coordination.

    Competition Requirements:
    1. Check service center capacity
    2. Propose appointment slots
    3. Confirm bookings with customers
    4. Optimize service center workload
    5. Handle customer preferences (location, timing)
    6. Coordinate multi-vehicle fleet scheduling

    Capabilities:
    - Real-time service center availability checking
    - Intelligent slot recommendation based on:
      * Service duration estimates
      * Priority levels
      * Customer location
      * Service center capacity
    - Automated booking confirmation
    - Rescheduling support
    - Fleet batch scheduling
    """

    def __init__(self):
        super().__init__(agent_name="SchedulingAgent")

        # Initialize LLM for intelligent scheduling decisions
        self.llm = ChatOpenAI(
            model=llm_model,
            temperature=temperature,
            api_key=os.getenv("OPENAI_API_KEY")
        )

        # Initialize notification system
        self.app_notification_system = AppNotificationSystem()

        # Mock service center database
        self.service_centers = self._initialize_service_centers()

        # Appointment database (in-memory for prototype)
        self.appointments = []
        self.appointment_counter = 1

        self.log_activity("Scheduling Agent initialized")
        self.log_activity(f"Service centers loaded: {len(self.service_centers)}")


    def get_prompt(self) -> str:
        # Dummy implementation – prototype does not use LLM
        return "NO_PROMPT_REQUIRED_FOR_SCHEDULING_AGENT"

    def _initialize_service_centers(self) -> List[Dict]:
        """Initialize mock service center database"""

        centers = [
            {
                "center_id": "SC001",
                "name": "Downtown Service Hub",
                "location": "Mumbai Central",
                "coordinates": {"lat": 19.0176, "lon": 72.8561},
                "capacity_per_day": 12,
                "working_hours": {"start": 9, "end": 18},
                "specializations": ["engine", "transmission", "electrical", "brake"],
                "rating": 4.8,
                "distance_from_customer": 5  # km
            },
            {
                "center_id": "SC002",
                "name": "Express Service Station",
                "location": "Andheri West",
                "coordinates": {"lat": 19.1136, "lon": 72.8697},
                "capacity_per_day": 8,
                "working_hours": {"start": 8, "end": 20},
                "specializations": ["quick_service", "oil_change", "tire", "battery"],
                "rating": 4.6,
                "distance_from_customer": 12  # km
            },
            {
                "center_id": "SC003",
                "name": "Premium Auto Care",
                "location": "Bandra East",
                "coordinates": {"lat": 19.0596, "lon": 72.8295},
                "capacity_per_day": 15,
                "working_hours": {"start": 9, "end": 19},
                "specializations": ["engine", "transmission", "suspension", "ac", "electrical"],
                "rating": 4.9,
                "distance_from_customer": 8  # km
            },
            {
                "center_id": "SC004",
                "name": "24/7 Emergency Service",
                "location": "Powai",
                "coordinates": {"lat": 19.1197, "lon": 72.9051},
                "capacity_per_day": 10,
                "working_hours": {"start": 0, "end": 24},  # 24/7
                "specializations": ["emergency", "towing", "all_services"],
                "rating": 4.7,
                "distance_from_customer": 15  # km
            },
            {
                "center_id": "SC005",
                "name": "Fleet Service Center",
                "location": "Thane Industrial Area",
                "coordinates": {"lat": 19.2183, "lon": 72.9781},
                "capacity_per_day": 25,  # Large capacity for fleet
                "working_hours": {"start": 7, "end": 21},
                "specializations": ["fleet", "bulk_service", "all_services"],
                "rating": 4.5,
                "distance_from_customer": 25  # km
            }
        ]

        return centers

    def process(self, input_data: Dict[str, Any]) -> AgentResponse:
        """
        Main scheduling process.

        Args:
            input_data: Must contain:
                - engagement_result: From Customer Engagement Agent
                - scheduling_preferences: Optional customer preferences

        Returns:
            AgentResponse with appointment details
        """

        try:
            self.log_activity("="*80)
            self.log_activity("STARTING APPOINTMENT SCHEDULING")
            self.log_activity("="*80)

            engagement_result = input_data.get("engagement_result", {})
            preferences = input_data.get("scheduling_preferences", {})

            # Extract key information
            customer_context = engagement_result.get("customer_context", {})
            vehicle_context = engagement_result.get("vehicle_context", {})
            diagnosis_summary = engagement_result.get("diagnosis_summary", {})
            next_steps = engagement_result.get("next_steps", {})

            customer_id = customer_context.get("customer_id", "UNKNOWN")
            customer_name = customer_context.get("name", "Customer")
            vehicle_id = vehicle_context.get("vehicle_id", "UNKNOWN")

            # Check if scheduling is needed
            if not next_steps.get("needs_scheduling", False):
                self.log_activity("Scheduling not required for this engagement")
                return self.create_response(
                    success=True,
                    data={"status": "no_scheduling_needed"},
                    next_action="complete"
                )

            self.log_activity(f"Customer: {customer_name} ({customer_id})")
            self.log_activity(f"Vehicle: {vehicle_id}")
            self.log_activity(f"Priority: {diagnosis_summary.get('priority', 'medium').upper()}")

            # Step 1: Find suitable service centers
            self.log_activity("\n[STEP 1] Finding suitable service centers...")
            suitable_centers = self._find_suitable_centers(
                diagnosis_summary, customer_context, preferences
            )

            # Step 2: Get available time slots
            self.log_activity("\n[STEP 2] Checking availability...")
            available_slots = self._get_available_slots(
                suitable_centers, diagnosis_summary, preferences
            )

            # Step 3: Recommend best slots using AI
            self.log_activity("\n[STEP 3] Generating intelligent recommendations...")
            recommended_slots = self._recommend_slots(
                available_slots, diagnosis_summary, customer_context, preferences
            )

            # Step 4: Create appointment (auto-book best slot)
            self.log_activity("\n[STEP 4] Creating appointment...")
            appointment = self._create_appointment(
                recommended_slots[0] if recommended_slots else None,
                customer_context, vehicle_context, diagnosis_summary,
                engagement_result.get("engagement_id")
            )

            # Step 5: Send confirmations
            self.log_activity("\n[STEP 5] Sending confirmations...")
            confirmation_records = self._send_confirmations(
                appointment, customer_id, customer_context, diagnosis_summary
            )

            # Compile scheduling result
            scheduling_result = {
                "scheduling_id": f"SCH-{uuid.uuid4().hex[:8]}",
                "appointment": appointment,
                "recommended_slots": recommended_slots[:3],  # Top 3 alternatives
                "suitable_centers": suitable_centers,
                "confirmation_records": confirmation_records,
                "scheduling_timestamp": datetime.now().isoformat(),
                "status": "confirmed" if appointment.get("status") == "confirmed" else "pending",
                "customer_id": customer_id,
                "vehicle_id": vehicle_id,
                "engagement_id": engagement_result.get("engagement_id")
            }

            self.log_activity("\n" + "="*80)
            self.log_activity("SCHEDULING COMPLETE")
            self.log_activity(f"Appointment ID: {appointment.get('appointment_id')}")
            self.log_activity(f"Service Center: {appointment.get('service_center_name')}")
            self.log_activity(f"Date/Time: {appointment.get('appointment_date')} at {appointment.get('appointment_time')}")
            self.log_activity("="*80)

            return self.create_response(
                success=True,
                data=scheduling_result,
                next_action="track_service"
            )

        except Exception as e:
            self.log_activity(f"Scheduling failed: {str(e)}", level="ERROR")
            return self.create_response(
                success=False,
                data={},
                error=str(e)
            )

    def _find_suitable_centers(self, diagnosis_summary: Dict,
                              customer_context: Dict,
                              preferences: Dict) -> List[Dict]:
        """Find service centers suitable for the required service"""

        priority = diagnosis_summary.get("priority", "medium")
        service_recommendations = diagnosis_summary.get("service_recommendations", [])

        # Determine required specialization
        required_specializations = set()
        for rec in service_recommendations:
            service_type = rec.get("service_type", "").lower()
            if "engine" in service_type:
                required_specializations.add("engine")
            elif "brake" in service_type:
                required_specializations.add("brake")
            elif "transmission" in service_type:
                required_specializations.add("transmission")
            elif "electrical" in service_type or "battery" in service_type:
                required_specializations.add("electrical")

        # Filter and score centers
        scored_centers = []
        for center in self.service_centers:
            score = 0

            # Check specialization match
            center_specs = set(center.get("specializations", []))
            if required_specializations.intersection(center_specs) or "all_services" in center_specs:
                score += 50

            # Priority-based selection
            if priority == "critical":
                if "emergency" in center_specs or center.get("working_hours", {}).get("end", 0) == 24:
                    score += 30  # Prefer 24/7 for critical

            # Distance score (closer is better)
            distance = center.get("distance_from_customer", 999)
            if distance <= 10:
                score += 20
            elif distance <= 20:
                score += 10

            # Rating score
            rating = center.get("rating", 0)
            score += rating * 5

            # Capacity score
            capacity = center.get("capacity_per_day", 0)
            score += min(capacity, 20)  # Cap at 20 points

            # Preferred center bonus
            if center.get("center_id") == customer_context.get("preferred_service_center"):
                score += 25

            scored_centers.append({
                **center,
                "suitability_score": score,
                "match_reason": self._get_match_reason(center, priority, distance)
            })

        # Sort by score
        scored_centers.sort(key=lambda x: x["suitability_score"], reverse=True)

        top_centers = scored_centers[:3]

        for center in top_centers:
            self.log_activity(
                f"  ✓ {center['name']} (Score: {center['suitability_score']}) - "
                f"{center['distance_from_customer']}km away"
            )

        return top_centers

    def _get_match_reason(self, center: Dict, priority: str, distance: float) -> str:
        """Generate human-readable match reason"""

        reasons = []

        if priority == "critical" and "emergency" in center.get("specializations", []):
            reasons.append("24/7 emergency service")

        if distance <= 5:
            reasons.append("closest to you")

        if center.get("rating", 0) >= 4.8:
            reasons.append("highest rated")

        if center.get("capacity_per_day", 0) >= 15:
            reasons.append("high capacity")

        return ", ".join(reasons) if reasons else "suitable for your service"

    def _get_available_slots(self, suitable_centers: List[Dict],
                            diagnosis_summary: Dict,
                            preferences: Dict) -> List[Dict]:
        """Get available appointment slots from service centers"""

        priority = diagnosis_summary.get("priority", "medium")
        estimated_duration = diagnosis_summary.get("estimated_duration_hours", 2)
        time_to_failure = diagnosis_summary.get("time_to_failure_days", 30)

        # Determine scheduling window based on priority
        if priority == "critical":
            max_days_ahead = min(2, time_to_failure)  # Within 2 days for critical
            self.log_activity(f"  Critical priority: Searching next {max_days_ahead} days")
        elif priority == "high":
            max_days_ahead = min(7, time_to_failure)
            self.log_activity(f"  High priority: Searching next {max_days_ahead} days")
        else:
            max_days_ahead = min(14, time_to_failure)
            self.log_activity(f"  Standard priority: Searching next {max_days_ahead} days")

        all_slots = []

        for center in suitable_centers[:2]:  # Check top 2 centers
            # Generate slots for each day
            for day_offset in range(max_days_ahead):
                slot_date = datetime.now() + timedelta(days=day_offset)

                # Skip if date is in customer's blackout
                if slot_date.strftime("%Y-%m-%d") in preferences.get("blackout_dates", []):
                    continue

                # Generate time slots
                working_hours = center.get("working_hours", {"start": 9, "end": 18})
                start_hour = working_hours["start"]
                end_hour = working_hours["end"]

                # Generate slots (every 2 hours for simplicity)
                for hour in range(start_hour, end_hour - int(estimated_duration), 2):
                    # Simulate availability (80% chance slot is available)
                    is_available = random.random() > 0.2

                    if is_available:
                        slot = {
                            "slot_id": f"SLOT-{uuid.uuid4().hex[:8]}",
                            "center_id": center["center_id"],
                            "center_name": center["name"],
                            "center_location": center["location"],
                            "center_rating": center["rating"],
                            "distance_km": center["distance_from_customer"],
                            "date": slot_date.strftime("%Y-%m-%d"),
                            "day_name": slot_date.strftime("%A"),
                            "time": f"{hour:02d}:00",
                            "estimated_duration_hours": estimated_duration,
                            "is_weekend": slot_date.weekday() >= 5,
                            "urgency_fit": self._calculate_urgency_fit(day_offset, priority),
                            "convenience_score": self._calculate_convenience_score(
                                hour, slot_date.weekday(), preferences
                            )
                        }
                        all_slots.append(slot)

        self.log_activity(f"  Found {len(all_slots)} available slots")

        return all_slots

    def _calculate_urgency_fit(self, day_offset: int, priority: str) -> float:
        """Calculate how well slot timing fits urgency"""

        if priority == "critical":
            return 1.0 if day_offset == 0 else 0.8 if day_offset == 1 else 0.5
        elif priority == "high":
            return 1.0 if day_offset <= 2 else 0.9 if day_offset <= 5 else 0.7
        else:
            return 1.0 if day_offset <= 7 else 0.8

    def _calculate_convenience_score(self, hour: int, weekday: int,
                                    preferences: Dict) -> float:
        """Calculate convenience score based on time and customer preferences"""

        score = 1.0

        # Preferred time preferences
        preferred_time = preferences.get("preferred_time", "any")

        if preferred_time == "morning" and 9 <= hour <= 12:
            score += 0.3
        elif preferred_time == "afternoon" and 13 <= hour <= 17:
            score += 0.3
        elif preferred_time == "evening" and 17 <= hour <= 20:
            score += 0.3

        # Weekend preference
        if preferences.get("prefer_weekend", False) and weekday >= 5:
            score += 0.2

        # Weekday preference
        if preferences.get("prefer_weekday", False) and weekday < 5:
            score += 0.2

        return min(score, 1.5)  # Cap at 1.5

    def _recommend_slots(self, available_slots: List[Dict],
                        diagnosis_summary: Dict,
                        customer_context: Dict,
                        preferences: Dict) -> List[Dict]:
        """Use AI to recommend best appointment slots"""

        if not available_slots:
            self.log_activity("No available slots found!")
            return []

        # Score each slot
        scored_slots = []
        for slot in available_slots:
            # Combined score
            score = (
                slot["urgency_fit"] * 40 +  # Urgency is most important
                slot["convenience_score"] * 30 +  # Then convenience
                (1 - slot["distance_km"] / 30) * 20 +  # Then distance
                slot["center_rating"] / 5 * 10  # Then rating
            )

            slot["recommendation_score"] = score
            slot["recommendation_reason"] = self._generate_recommendation_reason(slot)
            scored_slots.append(slot)

        # Sort by score
        scored_slots.sort(key=lambda x: x["recommendation_score"], reverse=True)

        top_slots = scored_slots[:5]

        self.log_activity(f"  Top recommendation: {top_slots[0]['day_name']}, "
                         f"{top_slots[0]['date']} at {top_slots[0]['time']} - "
                         f"{top_slots[0]['center_name']}")

        return top_slots

    def _generate_recommendation_reason(self, slot: Dict) -> str:
        """Generate reason for recommendation"""

        reasons = []

        if slot["urgency_fit"] >= 0.9:
            reasons.append("optimal timing for urgency")

        if slot["distance_km"] <= 5:
            reasons.append("closest location")

        if slot["convenience_score"] >= 1.2:
            reasons.append("matches your preferences")

        if slot["center_rating"] >= 4.8:
            reasons.append("highest rated center")

        if slot["is_weekend"]:
            reasons.append("weekend slot")

        return ", ".join(reasons) if reasons else "available slot"

    def _create_appointment(self, recommended_slot: Dict,
                           customer_context: Dict,
                           vehicle_context: Dict,
                           diagnosis_summary: Dict,
                           engagement_id: str) -> Dict:
        """Create and confirm appointment"""

        if not recommended_slot:
            self.log_activity("Cannot create appointment - no slots available")
            return {"status": "failed", "reason": "no_slots_available"}

        appointment = {
            "appointment_id": f"APT-{self.appointment_counter:05d}",
            "status": "confirmed",
            "created_at": datetime.now().isoformat(),

            # Customer info
            "customer_id": customer_context.get("customer_id"),
            "customer_name": customer_context.get("name"),
            "customer_phone": customer_context.get("phone"),

            # Vehicle info
            "vehicle_id": vehicle_context.get("vehicle_id"),
            "vehicle_model": vehicle_context.get("model"),

            # Appointment details
            "service_center_id": recommended_slot["center_id"],
            "service_center_name": recommended_slot["center_name"],
            "service_center_location": recommended_slot["center_location"],
            "appointment_date": recommended_slot["date"],
            "appointment_time": recommended_slot["time"],
            "day_name": recommended_slot["day_name"],
            "estimated_duration_hours": recommended_slot["estimated_duration_hours"],
            "estimated_completion_time": self._calculate_completion_time(
                recommended_slot["date"], recommended_slot["time"],
                recommended_slot["estimated_duration_hours"]
            ),

            # Service details
            "priority": diagnosis_summary.get("priority"),
            "service_type": diagnosis_summary.get("service_recommendations", [{}])[0].get("service_type", "Maintenance"),
            "estimated_cost": diagnosis_summary.get("service_recommendations", [{}])[0].get("estimated_cost_min", 0),
            "service_recommendations": diagnosis_summary.get("service_recommendations", []),

            # References
            "engagement_id": engagement_id,
            "slot_id": recommended_slot["slot_id"],

            # Additional services
            "complimentary_services": [
                "25-point vehicle health check (₹1,500 value)",
                "Free car wash",
                "WiFi waiting area",
                "Complimentary refreshments"
            ],

            # Reminders scheduled
            "reminders_scheduled": {
                "24_hours_before": True,
                "2_hours_before": True,
                "on_arrival": True
            }
        }

        self.appointments.append(appointment)
        self.appointment_counter += 1

        self.log_activity(f"Appointment created: {appointment['appointment_id']}")

        return appointment

    def _calculate_completion_time(self, date_str: str, time_str: str,
                                   duration_hours: float) -> str:
        """Calculate estimated completion time"""

        dt = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M")
        completion = dt + timedelta(hours=duration_hours)
        return completion.strftime("%H:%M")

    def _send_confirmations(self, appointment: Dict, customer_id: str,
                           customer_context: Dict, diagnosis_summary: Dict) -> Dict:
        """Send appointment confirmations via multiple channels"""

        if appointment.get("status") != "confirmed":
            return {"status": "no_confirmations_sent"}

        confirmations = {}

        # 1. App notification - Detailed confirmation
        app_notif = self.app_notification_system.send_notification(
            customer_id=customer_id,
            title="Appointment Confirmed!",
            body=f"Your service appointment is confirmed for {appointment['day_name']}, {appointment['appointment_date']} at {appointment['appointment_time']}. "
                 f"Location: {appointment['service_center_name']}, {appointment['service_center_location']}. "
                 f"Duration: ~{appointment['estimated_duration_hours']} hours.",
            notification_type=NotificationType.INFO,
            priority=NotificationPriority.HIGH,
            data={
                "appointment_id": appointment["appointment_id"],
                "appointment_date": appointment["appointment_date"],
                "appointment_time": appointment["appointment_time"],
                "service_center_location": appointment["service_center_location"],
                "estimated_cost": appointment["estimated_cost"],
                "action": "view_appointment_details"
            }
        )
        confirmations["app_notification"] = app_notif

        # 2. SMS confirmation
        sms_message = f"""Appointment Confirmed

{customer_context.get('name')}, your service is booked:

 {appointment['day_name']}, {appointment['appointment_date']}
 {appointment['appointment_time']}
 {appointment['service_center_name']}
   {appointment['service_center_location']}

 Est. Cost: ₹{appointment['estimated_cost']:,}
 Duration: ~{appointment['estimated_duration_hours']} hrs

 Includes: Free health check (₹1,500 value)

Questions? Call 1800-XXX-XXXX
View details in app."""

        sms_record = self.app_notification_system.send_sms_fallback(
            customer_context.get("phone", "N/A"),
            sms_message
        )
        confirmations["sms"] = sms_record

        # 3. Schedule reminder notifications (metadata only, not sent yet)
        confirmations["reminders_scheduled"] = {
            "24_hour_reminder": {
                "scheduled_for": self._calculate_reminder_time(
                    appointment["appointment_date"],
                    appointment["appointment_time"],
                    hours_before=24
                ),
                "channel": "app_notification"
            },
            "2_hour_reminder": {
                "scheduled_for": self._calculate_reminder_time(
                    appointment["appointment_date"],
                    appointment["appointment_time"],
                    hours_before=2
                ),
                "channel": "sms_and_app"
            }
        }

        return confirmations

    def _calculate_reminder_time(self, date_str: str, time_str: str,
                                 hours_before: int) -> str:
        """Calculate when reminder should be sent"""

        dt = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M")
        reminder_time = dt - timedelta(hours=hours_before)
        return reminder_time.isoformat()

    def reschedule_appointment(self, appointment_id: str,
                              new_preferences: Dict) -> AgentResponse:
        """Handle appointment rescheduling"""

        self.log_activity(f"Rescheduling appointment: {appointment_id}")

        # Find existing appointment
        existing = None
        for apt in self.appointments:
            if apt["appointment_id"] == appointment_id:
                existing = apt
                break

        if not existing:
            return self.create_response(
                success=False,
                data={},
                error="Appointment not found"
            )

        # Mark old appointment as rescheduled
        existing["status"] = "rescheduled"
        existing["rescheduled_at"] = datetime.now().isoformat()

        # Create mock data for rescheduling
        mock_input = {
            "engagement_result": {
                "customer_context": {
                    "customer_id": existing["customer_id"],
                    "name": existing["customer_name"],
                    "phone": existing["customer_phone"]
                },
                "vehicle_context": {
                    "vehicle_id": existing["vehicle_id"],
                    "model": existing["vehicle_model"]
                },
                "diagnosis_summary": {
                    "priority": existing["priority"],
                    "service_recommendations": existing["service_recommendations"],
                    "estimated_duration_hours": existing["estimated_duration_hours"]
                },
                "next_steps": {
                    "needs_scheduling": True
                },
                "engagement_id": existing["engagement_id"]
            },
            "scheduling_preferences": new_preferences
        }

        # Process new scheduling
        result = self.process(mock_input)

        if result.success:
            result.data["rescheduled_from"] = appointment_id
            result.data["reason"] = new_preferences.get("reason", "Customer request")

        return result

    def handle_fleet_batch_scheduling(self, fleet_engagement_result: Dict) -> AgentResponse:
        """Handle batch scheduling for fleet vehicles"""

        self.log_activity("\n" + "="*80)
        self.log_activity("FLEET BATCH SCHEDULING")
        self.log_activity("="*80)

        fleet_id = fleet_engagement_result.get("fleet_id")
        vehicles_processed = fleet_engagement_result.get("vehicles_processed", [])

        self.log_activity(f"Fleet ID: {fleet_id}")
        self.log_activity(f"Vehicles to schedule: {len(vehicles_processed)}")

        # Find fleet service center
        fleet_center = None
        for center in self.service_centers:
            if "fleet" in center.get("specializations", []):
                fleet_center = center
                break

        if not fleet_center:
            fleet_center = self.service_centers[0]  # Fallback

        # Schedule all vehicles at fleet center
        batch_appointments = []
        current_date = datetime.now() + timedelta(days=1)  # Start tomorrow
        current_slot = 7  # 7 AM start

        for vehicle_engagement in vehicles_processed:
            # Create appointment for each vehicle
            appointment = {
                "appointment_id": f"FLEET-APT-{self.appointment_counter:05d}",
                "status": "confirmed",
                "created_at": datetime.now().isoformat(),
                "is_fleet_booking": True,
                "fleet_id": fleet_id,

                "customer_id": vehicle_engagement.get("customer_context", {}).get("customer_id"),
                "customer_name": vehicle_engagement.get("customer_context", {}).get("name"),
                "vehicle_id": vehicle_engagement.get("vehicle_context", {}).get("vehicle_id"),
                "vehicle_model": vehicle_engagement.get("vehicle_context", {}).get("model"),

                "service_center_id": fleet_center["center_id"],
                "service_center_name": fleet_center["name"],
                "service_center_location": fleet_center["location"],

                "appointment_date": current_date.strftime("%Y-%m-%d"),
                "appointment_time": f"{current_slot:02d}:00",
                "estimated_duration_hours": 2,

                "priority": vehicle_engagement.get("diagnosis_summary", {}).get("priority"),
                "service_type": "Fleet Maintenance",
                "estimated_cost": vehicle_engagement.get("diagnosis_summary", {}).get("service_recommendations", [{}])[0].get("estimated_cost_min", 0),

                "batch_sequence": len(batch_appointments) + 1,
                "fleet_discount": "15%"
            }

            batch_appointments.append(appointment)
            self.appointments.append(appointment)
            self.appointment_counter += 1

            # Move to next slot (2-hour intervals)
            current_slot += 2
            if current_slot >= 19:  # End of day
                current_date += timedelta(days=1)
                current_slot = 7

        # Send fleet confirmation
        customer_id = fleet_engagement_result.get("fleet_owner", {}).get("customer_id")

        fleet_notif = self.app_notification_system.send_notification(
            customer_id=customer_id,
            title=f"🚛 Fleet Service Scheduled - {len(batch_appointments)} Vehicles",
            body=f"All {len(batch_appointments)} vehicles have been scheduled at {fleet_center['name']}. First appointment: {batch_appointments[0]['appointment_date']} at {batch_appointments[0]['appointment_time']}. 15% fleet discount applied!",
            notification_type=NotificationType.INFO,
            priority=NotificationPriority.HIGH,
            data={
                "fleet_id": fleet_id,
                "total_vehicles": len(batch_appointments),
                "total_cost_before_discount": sum(a["estimated_cost"] for a in batch_appointments),
                "fleet_discount": "15%",
                "action": "view_fleet_schedule"
            }
        )

        self.log_activity(f"Fleet batch scheduled: {len(batch_appointments)} vehicles")

        return self.create_response(
            success=True,
            data={
                "scheduling_id": f"FLEET-SCH-{uuid.uuid4().hex[:8]}",
                "fleet_id": fleet_id,
                "batch_appointments": batch_appointments,
                "fleet_center": fleet_center,
                "fleet_notification": fleet_notif,
                "total_vehicles": len(batch_appointments),
                "schedule_start_date": batch_appointments[0]["appointment_date"],
                "schedule_end_date": batch_appointments[-1]["appointment_date"],
                "fleet_discount": "15%"
            },
            next_action="track_fleet_service"
        )

    def get_appointment_details(self, appointment_id: str) -> Dict:
        """Retrieve appointment details"""

        for apt in self.appointments:
            if apt["appointment_id"] == appointment_id:
                return apt

        return None

    def get_scheduling_analytics(self) -> Dict:
        """Get scheduling analytics"""

        total = len(self.appointments)
        confirmed = sum(1 for a in self.appointments if a["status"] == "confirmed")
        fleet = sum(1 for a in self.appointments if a.get("is_fleet_booking", False))

        # By priority
        by_priority = {}
        for apt in self.appointments:
            priority = apt.get("priority", "unknown")
            by_priority[priority] = by_priority.get(priority, 0) + 1

        # By service center
        by_center = {}
        for apt in self.appointments:
            center = apt.get("service_center_name", "Unknown")
            by_center[center] = by_center.get(center, 0) + 1

        return {
            "total_appointments": total,
            "confirmed_appointments": confirmed,
            "fleet_bookings": fleet,
            "by_priority": by_priority,
            "by_service_center": by_center,
            "service_centers_active": len(self.service_centers)
        }

In [26]:
class FeedbackAgent(BaseAgent):
    """
    Feedback Agent - Manages post-service feedback and customer satisfaction.

    Competition Requirements:
    1. Track service progress until completion
    2. Follow up for customer feedback
    3. Collect satisfaction ratings
    4. Identify issues for improvement
    5. Update vehicle maintenance records

    Capabilities:
    - Multi-channel feedback collection (app, SMS, voice)
    - Sentiment analysis of feedback
    - Issue categorization
    - Automated follow-up for low ratings
    - Manufacturing insights from recurring issues
    """

    def __init__(self):
        super().__init__(agent_name="FeedbackAgent")

        self.llm = ChatOpenAI(
            model=llm_model,
            temperature=temperature,
            api_key=os.getenv("OPENAI_API_KEY")
        )

        # Initialize notification system
        self.app_notification_system = AppNotificationSystem()

        # Feedback database (in-memory for prototype)
        self.feedback_records = []

        self.log_activity("Feedback Agent initialized")

    def process(self, input_data: Dict[str, Any]) -> AgentResponse:
        """
        Main feedback collection process.

        Args:
            input_data: Must contain:
                - appointment: Completed appointment details
                - service_completion: Service completion data

        Returns:
            AgentResponse with feedback data
        """

        try:
            self.log_activity("="*80)
            self.log_activity("STARTING FEEDBACK COLLECTION")
            self.log_activity("="*80)

            appointment = input_data.get("appointment", {})
            service_completion = input_data.get("service_completion", {})

            customer_id = appointment.get("customer_id", "UNKNOWN")
            customer_name = appointment.get("customer_name", "Customer")
            vehicle_id = appointment.get("vehicle_id", "UNKNOWN")
            appointment_id = appointment.get("appointment_id", "UNKNOWN")

            self.log_activity(f"Customer: {customer_name} ({customer_id})")
            self.log_activity(f"Vehicle: {vehicle_id}")
            self.log_activity(f"Appointment: {appointment_id}")

            # Step 1: Send feedback request
            self.log_activity("\n[STEP 1] Sending feedback request...")
            feedback_request = self._send_feedback_request(
                customer_id, appointment, service_completion
            )

            # Step 2: Simulate customer feedback (for demo)
            self.log_activity("\n[STEP 2] Collecting customer feedback...")
            customer_feedback = self._simulate_customer_feedback(
                appointment, service_completion
            )

            # Step 3: Analyze feedback with AI
            self.log_activity("\n[STEP 3] Analyzing feedback...")
            feedback_analysis = self._analyze_feedback(customer_feedback, appointment)

            # Step 4: Handle low ratings (if any)
            self.log_activity("\n[STEP 4] Processing feedback...")
            follow_up_actions = self._handle_feedback_response(
                feedback_analysis, customer_id, appointment
            )

            # Step 5: Update vehicle maintenance records
            self.log_activity("\n[STEP 5] Updating maintenance records...")
            maintenance_update = self._update_maintenance_records(
                vehicle_id, appointment, service_completion, feedback_analysis
            )

            # Compile feedback result
            feedback_result = {
                "feedback_id": f"FB-{uuid.uuid4().hex[:8]}",
                "appointment_id": appointment_id,
                "customer_id": customer_id,
                "vehicle_id": vehicle_id,
                "timestamp": datetime.now().isoformat(),

                # Feedback data
                "customer_feedback": customer_feedback,
                "feedback_analysis": feedback_analysis,
                "feedback_request": feedback_request,

                # Actions taken
                "follow_up_actions": follow_up_actions,
                "maintenance_record_updated": maintenance_update,

                # Aggregated metrics
                "overall_satisfaction": feedback_analysis.get("overall_rating"),
                "would_recommend": feedback_analysis.get("would_recommend"),
                "issues_identified": feedback_analysis.get("issues", []),
                "positive_highlights": feedback_analysis.get("highlights", []),

                # Manufacturing insights
                "manufacturing_feedback": feedback_analysis.get("manufacturing_insights", {})
            }

            # Store feedback
            self.feedback_records.append(feedback_result)

            self.log_activity("\n" + "="*80)
            self.log_activity("FEEDBACK COLLECTION COMPLETE")
            self.log_activity(f"Overall Rating: {feedback_analysis.get('overall_rating')}/5.0")
            self.log_activity(f"Sentiment: {feedback_analysis.get('sentiment', 'neutral').upper()}")
            self.log_activity("="*80)

            return self.create_response(
                success=True,
                data=feedback_result,
                next_action="complete"
            )

        except Exception as e:
            self.log_activity(f"Feedback collection failed: {str(e)}", level="ERROR")
            return self.create_response(
                success=False,
                data={},
                error=str(e)
            )

    def _send_feedback_request(self, customer_id: str, appointment: Dict,
                              service_completion: Dict) -> Dict:
        """Send feedback request to customer"""

        # App notification
        app_notif = self.app_notification_system.send_notification(
            customer_id=customer_id,
            title="📝 How was your service experience?",
            body=f"Hi {appointment.get('customer_name')}! Your service is complete. We'd love to hear your feedback. It takes just 2 minutes!",
            notification_type=NotificationType.INFO,
            priority=NotificationPriority.MEDIUM,
            data={
                "appointment_id": appointment.get("appointment_id"),
                "vehicle_id": appointment.get("vehicle_id"),
                "action": "open_feedback_form",
                "incentive": "Complete feedback to earn 500 loyalty points!"
            }
        )

        # SMS with feedback link
        sms_message = f""" Service Complete!

{appointment.get('customer_name')}, thank you for visiting {appointment.get('service_center_name')}!

 Share your experience (2 min):
https://feedback.link/{appointment.get('appointment_id')}

 Earn 500 loyalty points!

- Your Service Team"""

        sms_record = self.app_notification_system.send_sms_fallback(
            appointment.get("customer_phone", "N/A"),
            sms_message
        )

        return {
            "app_notification": app_notif,
            "sms": sms_record,
            "sent_at": datetime.now().isoformat(),
            "feedback_link": f"https://feedback.link/{appointment.get('appointment_id')}"
        }

    def _simulate_customer_feedback(self, appointment: Dict,
                                   service_completion: Dict) -> Dict:
        """Simulate customer feedback (for prototype)"""

        # Generate realistic feedback based on service quality
        service_quality = service_completion.get("quality_score", 0.85)

        # Overall rating (1-5)
        if service_quality >= 0.9:
            overall_rating = random.uniform(4.5, 5.0)
        elif service_quality >= 0.8:
            overall_rating = random.uniform(4.0, 4.7)
        elif service_quality >= 0.7:
            overall_rating = random.uniform(3.5, 4.2)
        else:
            overall_rating = random.uniform(2.5, 3.8)

        # Specific ratings
        feedback = {
            "submission_timestamp": datetime.now().isoformat(),
            "response_time_hours": random.uniform(2, 24),

            # Ratings (1-5 scale)
            "ratings": {
                "overall": round(overall_rating, 1),
                "service_quality": round(random.uniform(overall_rating - 0.3, overall_rating + 0.3), 1),
                "staff_behavior": round(random.uniform(overall_rating - 0.2, 5.0), 1),
                "timeliness": round(random.uniform(overall_rating - 0.5, overall_rating + 0.2), 1),
                "value_for_money": round(random.uniform(overall_rating - 0.4, overall_rating + 0.1), 1),
                "facility_cleanliness": round(random.uniform(4.0, 5.0), 1)
            },

            # Yes/No questions
            "would_recommend": overall_rating >= 4.0,
            "issue_resolved": random.choice([True, True, True, False]),  # 75% yes
            "waited_as_expected": random.choice([True, True, False]),  # 67% yes

            # Open-ended feedback
            "comments": self._generate_feedback_comment(overall_rating),
            "improvement_suggestions": self._generate_improvement_suggestion(overall_rating),

            # Additional context
            "visit_purpose": appointment.get("service_type"),
            "service_center": appointment.get("service_center_name")
        }

        return feedback

    def _generate_feedback_comment(self, rating: float) -> str:
        """Generate realistic feedback comments"""

        if rating >= 4.5:
            comments = [
                "Excellent service! The team was professional and explained everything clearly. Very satisfied.",
                "Great experience. Fixed the issue perfectly and quickly. Will definitely return.",
                "Outstanding service quality. Staff was courteous and the work was done well.",
                "Very impressed! Transparent pricing and quality work. Highly recommend!"
            ]
        elif rating >= 3.5:
            comments = [
                "Good service overall. Took a bit longer than expected but the quality was fine.",
                "Satisfied with the work. Staff was helpful. Could improve on waiting area comfort.",
                "Service was decent. Got the job done. Pricing was reasonable.",
                "Everything went well. Minor communication issues but overall positive experience."
            ]
        else:
            comments = [
                "Service took longer than promised. Had to follow up multiple times.",
                "Not very happy. Issue wasn't fully resolved and had to bring car back.",
                "Expected better service. Staff seemed rushed and didn't explain things properly.",
                "Disappointed. Cost was higher than estimated and quality was just okay."
            ]

        return random.choice(comments)

    def _generate_improvement_suggestion(self, rating: float) -> str:
        """Generate improvement suggestions"""

        if rating >= 4.5:
            return "Nothing major. Keep up the good work!"
        elif rating >= 3.5:
            suggestions = [
                "Better communication about service delays",
                "More comfortable waiting area",
                "Faster service turnaround time",
                "More transparent cost breakdown upfront"
            ]
        else:
            suggestions = [
                "Improve time estimates - actual time taken was much longer",
                "Better staff training on customer service",
                "More accurate cost estimates before starting work",
                "Follow-up on service quality to ensure issues are resolved"
            ]

        return random.choice(suggestions)

        # Add this inside SchedulingAgent class

    def get_prompt(self) -> str:
        """
        Prototype placeholder - No LLM required for scheduling in MVP.
        Required only to satisfy BaseAgent interface.
        """
        return "Scheduling agent does not require an LLM prompt in prototype mode."


        def _analyze_feedback(self, feedback: Dict, appointment: Dict) -> Dict:
            """Analyze feedback using AI"""

        try:
            prompt = ChatPromptTemplate.from_template("""You are analyzing customer feedback for a vehicle service center.

FEEDBACK DATA:
Overall Rating: {overall_rating}/5
Service Quality: {service_quality}/5
Staff Behavior: {staff_behavior}/5
Comments: {comments}
Improvement Suggestions: {suggestions}
Would Recommend: {would_recommend}

APPOINTMENT CONTEXT:
Service Type: {service_type}
Priority: {priority}
Service Center: {service_center}

ANALYZE:
1. Overall sentiment (positive/neutral/negative)
2. Key issues (if any)
3. Positive highlights
4. Manufacturing/quality insights (recurring issues that need escalation)
5. Action required (yes/no)

Respond in JSON format with these exact keys:
- sentiment: string
- overall_summary: string
- issues: list of strings
- highlights: list of strings
- manufacturing_insights: object with "has_insights" (bool) and "details" (string)
- action_required: bool
- action_type: string (null or "follow_up_low_rating" or "investigate_issue")""")

            chain = prompt | self.llm

            response = chain.invoke({
                "overall_rating": feedback["ratings"]["overall"],
                "service_quality": feedback["ratings"]["service_quality"],
                "staff_behavior": feedback["ratings"]["staff_behavior"],
                "comments": feedback.get("comments", ""),
                "suggestions": feedback.get("improvement_suggestions", ""),
                "would_recommend": feedback.get("would_recommend", False),
                "service_type": appointment.get("service_type", ""),
                "priority": appointment.get("priority", ""),
                "service_center": appointment.get("service_center_name", "")
            })

            # Parse AI response (simplified for demo)
            analysis = {
                "overall_rating": feedback["ratings"]["overall"],
                "sentiment": "positive" if feedback["ratings"]["overall"] >= 4.0 else "neutral" if feedback["ratings"]["overall"] >= 3.0 else "negative",
                "overall_summary": response.content[:200],
                "would_recommend": feedback.get("would_recommend", False),

                # Extracted insights
                "issues": [],
                "highlights": [],
                "manufacturing_insights": {
                    "has_insights": False,
                    "details": None
                },
                "action_required": feedback["ratings"]["overall"] < 3.5,
                "action_type": "follow_up_low_rating" if feedback["ratings"]["overall"] < 3.5 else None
            }

            # Extract issues from comments
            if "longer" in feedback.get("comments", "").lower() or "delay" in feedback.get("comments", "").lower():
                analysis["issues"].append("Service time exceeded expectations")

            if "not resolved" in feedback.get("comments", "").lower():
                analysis["issues"].append("Issue not fully resolved")
                analysis["manufacturing_insights"]["has_insights"] = True
                analysis["manufacturing_insights"]["details"] = "Recurring quality issue detected"

            if "cost" in feedback.get("comments", "").lower() or "price" in feedback.get("comments", "").lower():
                analysis["issues"].append("Cost concerns")

            # Extract highlights
            if "excellent" in feedback.get("comments", "").lower() or "great" in feedback.get("comments", "").lower():
                analysis["highlights"].append("Exceptional service quality praised")

            if "professional" in feedback.get("comments", "").lower() or "courteous" in feedback.get("comments", "").lower():
                analysis["highlights"].append("Professional staff behavior")

            return analysis

        except Exception as e:
            self.log_activity(f"AI analysis failed, using rule-based: {str(e)}", level="WARNING")

            # Fallback rule-based analysis
            return {
                "overall_rating": feedback["ratings"]["overall"],
                "sentiment": "positive" if feedback["ratings"]["overall"] >= 4.0 else "neutral",
                "overall_summary": "Customer feedback recorded",
                "would_recommend": feedback.get("would_recommend", False),
                "issues": [],
                "highlights": [],
                "manufacturing_insights": {"has_insights": False, "details": None},
                "action_required": False,
                "action_type": None
            }

    def _handle_feedback_response(self, feedback_analysis: Dict,
                                  customer_id: str, appointment: Dict) -> Dict:
        """Handle feedback and take appropriate actions"""

        actions = {
            "actions_taken": [],
            "notifications_sent": [],
            "escalations": []
        }

        # Low rating follow-up
        if feedback_analysis.get("overall_rating", 5) < 3.5:
            self.log_activity("Low rating detected - initiating follow-up")

            # Send apology and resolution offer
            notif = self.app_notification_system.send_notification(
                customer_id=customer_id,
                title="We're Sorry - Let Us Make It Right",
                body=f"Hi {appointment.get('customer_name')}, we noticed you weren't fully satisfied with your recent service. Our team lead will contact you within 24 hours to resolve any concerns. Your satisfaction is our priority.",
                notification_type=NotificationType.INFO,
                priority=NotificationPriority.HIGH,
                data={
                    "appointment_id": appointment.get("appointment_id"),
                    "action": "manager_callback_scheduled",
                    "compensation_offer": "20% discount on next service"
                }
            )

            actions["notifications_sent"].append(notif)
            actions["actions_taken"].append("Manager callback scheduled")
            actions["actions_taken"].append("Compensation offer: 20% discount on next service")

            # Escalate to supervisor
            actions["escalations"].append({
                "escalation_type": "low_satisfaction",
                "escalated_to": "service_center_manager",
                "escalated_at": datetime.now().isoformat(),
                "customer_id": customer_id,
                "appointment_id": appointment.get("appointment_id"),
                "rating": feedback_analysis.get("overall_rating")
            })

        # Issue not resolved
        if not feedback_analysis.get("would_recommend", True):
            self.log_activity("Customer would not recommend - investigation needed")
            actions["actions_taken"].append("Quality investigation initiated")

        # Manufacturing insights
        if feedback_analysis.get("manufacturing_insights", {}).get("has_insights", False):
            self.log_activity("  🔧 Manufacturing insights detected - flagging for RCA/CAPA")
            actions["actions_taken"].append("Manufacturing feedback flagged for RCA/CAPA")
            actions["escalations"].append({
                "escalation_type": "quality_issue",
                "escalated_to": "manufacturing_quality_team",
                "escalated_at": datetime.now().isoformat(),
                "details": feedback_analysis["manufacturing_insights"]["details"]
            })

        # Positive feedback recognition
        if feedback_analysis.get("overall_rating", 0) >= 4.5:
            self.log_activity("Excellent rating - recognizing service team")
            actions["actions_taken"].append("Service team recognition submitted")

            # Thank you notification
            notif = self.app_notification_system.send_notification(
                customer_id=customer_id,
                title="Thank You!",
                body=f"Thank you for your excellent feedback, {appointment.get('customer_name')}! We've awarded you 500 loyalty points. We look forward to serving you again!",
                notification_type=NotificationType.INFO,
                priority=NotificationPriority.MEDIUM,
                data={
                    "loyalty_points_earned": 500,
                    "total_loyalty_points": 1500,  # Mock total
                    "action": "view_loyalty_rewards"
                }
            )
            actions["notifications_sent"].append(notif)

        return actions

    def _update_maintenance_records(self, vehicle_id: str, appointment: Dict,
                                   service_completion: Dict,
                                   feedback_analysis: Dict) -> Dict:
        """Update vehicle maintenance records"""

        maintenance_update = {
            "vehicle_id": vehicle_id,
            "updated_at": datetime.now().isoformat(),
            "service_record": {
                "appointment_id": appointment.get("appointment_id"),
                "service_date": appointment.get("appointment_date"),
                "service_center": appointment.get("service_center_name"),
                "service_type": appointment.get("service_type"),
                "mileage_at_service": service_completion.get("mileage_at_service", 50000),
                "work_performed": service_completion.get("work_performed", []),
                "parts_replaced": service_completion.get("parts_replaced", []),
                "cost": appointment.get("estimated_cost"),
                "customer_satisfaction": feedback_analysis.get("overall_rating")
            },
            "next_service_due": {
                "date": (datetime.now() + timedelta(days=180)).strftime("%Y-%m-%d"),
                "mileage": service_completion.get("mileage_at_service", 50000) + 10000,
                "recommended_services": ["Oil change", "Tire rotation", "Brake inspection"]
            }
        }

        self.log_activity(f"Maintenance record updated for vehicle {vehicle_id}")

        return maintenance_update

    def generate_feedback_report(self, time_period_days: int = 30) -> Dict:
        """Generate comprehensive feedback analytics report"""

        cutoff_date = datetime.now() - timedelta(days=time_period_days)

        # Filter recent feedback
        recent_feedback = [
            fb for fb in self.feedback_records
            if datetime.fromisoformat(fb["timestamp"]) >= cutoff_date
        ]

        if not recent_feedback:
            return {"error": "No feedback data available for the specified period"}

        # Calculate metrics
        total_responses = len(recent_feedback)

        # Average ratings
        avg_overall = sum(fb["overall_satisfaction"] for fb in recent_feedback) / total_responses

        # Sentiment distribution
        sentiment_dist = {"positive": 0, "neutral": 0, "negative": 0}
        for fb in recent_feedback:
            sentiment = fb["feedback_analysis"].get("sentiment", "neutral")
            sentiment_dist[sentiment] = sentiment_dist.get(sentiment, 0) + 1

        # Recommendation rate
        would_recommend = sum(1 for fb in recent_feedback
                             if fb["feedback_analysis"].get("would_recommend", False))
        nps = (would_recommend / total_responses) * 100

        # Issues aggregation
        all_issues = []
        for fb in recent_feedback:
            all_issues.extend(fb.get("issues_identified", []))

        issue_counts = {}
        for issue in all_issues:
            issue_counts[issue] = issue_counts.get(issue, 0) + 1

        # Manufacturing feedback
        manufacturing_feedback_count = sum(
            1 for fb in recent_feedback
            if fb.get("manufacturing_feedback", {}).get("has_insights", False)
        )

        report = {
            "report_id": f"FB-RPT-{uuid.uuid4().hex[:8]}",
            "generated_at": datetime.now().isoformat(),
            "period_days": time_period_days,

            "summary": {
                "total_responses": total_responses,
                "response_rate": "78%",  # Mock
                "average_rating": round(avg_overall, 2),
                "nps_score": round(nps, 1),
                "would_recommend_percentage": round((would_recommend/total_responses)*100, 1)
            },

            "sentiment_distribution": sentiment_dist,

            "top_issues": sorted(issue_counts.items(), key=lambda x: x[1], reverse=True)[:5],

            "actions_summary": {
                "low_rating_follow_ups": sum(1 for fb in recent_feedback if fb["overall_satisfaction"] < 3.5),
                "manager_escalations": sum(len(fb.get("follow_up_actions", {}).get("escalations", []))
                                          for fb in recent_feedback),
                "manufacturing_feedbacks": manufacturing_feedback_count
            },

            "service_center_performance": self._calculate_center_performance(recent_feedback)
        }

        return report

    def _calculate_center_performance(self, feedback_records: List[Dict]) -> Dict:
        """Calculate performance by service center"""

        by_center = {}

        for fb in feedback_records:
            center = fb.get("customer_feedback", {}).get("service_center", "Unknown")

            if center not in by_center:
                by_center[center] = {
                    "total_feedback": 0,
                    "total_rating": 0,
                    "positive_count": 0
                }

            by_center[center]["total_feedback"] += 1
            by_center[center]["total_rating"] += fb["overall_satisfaction"]

            if fb["feedback_analysis"].get("sentiment") == "positive":
                by_center[center]["positive_count"] += 1

        # Calculate averages
        for center in by_center:
            total = by_center[center]["total_feedback"]
            by_center[center]["average_rating"] = round(by_center[center]["total_rating"] / total, 2)
            by_center[center]["satisfaction_rate"] = round((by_center[center]["positive_count"] / total) * 100, 1)

        return by_center

    def get_feedback_analytics(self) -> Dict:
        """Get real-time feedback analytics"""

        return {
            "total_feedback_collected": len(self.feedback_records),
            "average_response_time_hours": 12.5,  # Mock
            "latest_feedback_timestamp": self.feedback_records[-1]["timestamp"] if self.feedback_records else None
        }

In [27]:
class ManufacturingQualityInsightsModule(BaseAgent):
    """
    Manufacturing Quality Insights - RCA/CAPA analysis and feedback loop.

    Competition Requirements:
    1. Cross-reference predicted failures with historical data
    2. Identify recurring defects
    3. Perform root cause analysis (RCA)
    4. Generate corrective and preventive actions (CAPA)
    5. Feed insights back to manufacturing team
    6. Suggest design improvements

    Capabilities:
    - Pattern detection in failures
    - RCA report generation
    - CAPA recommendation
    - Manufacturing feedback loop
    - Quality trend analysis
    - Design improvement suggestions
    """

    def __init__(self, llm_model: str = "gpt-4o", temperature: float = 0.3):
        super().__init__(agent_name="ManufacturingQualityInsights")

        # Initialize LLM for intelligent analysis
        self.llm = ChatOpenAI(
            model=llm_model,
            temperature=temperature
        )

        # Mock historical CAPA/RCA database
        self.historical_rca_data = self._load_historical_rca_data()
        self.manufacturing_defect_records = self._load_defect_records()

        # Insights database
        self.insights_generated = []

        self.log_activity("Manufacturing Quality Insights Module initialized")
        self.log_activity(f"Historical RCA records loaded: {len(self.historical_rca_data)}")
        self.log_activity(f"Defect records loaded: {len(self.manufacturing_defect_records)}")


    def get_prompt(self) -> str:
        # Dummy implementation – prototype does not use LLM
        return "NO_PROMPT_REQUIRED_FOR_SCHEDULING_AGENT"

    def _load_historical_rca_data(self) -> List[Dict]:
        """Load mock historical RCA data"""

        return [
            {
                "rca_id": "RCA-001",
                "issue": "Engine oil leak",
                "vehicle_models": ["Model X 2022", "Model X 2023"],
                "root_cause": "Defective oil pan gasket from supplier batch #45211",
                "affected_units": 1250,
                "severity": "high",
                "date_identified": "2024-03-15",
                "capa_actions": [
                    "Replace gasket supplier",
                    "Implement incoming QC inspection",
                    "Retrofit affected vehicles"
                ],
                "status": "closed"
            },
            {
                "rca_id": "RCA-002",
                "issue": "Brake pad premature wear",
                "vehicle_models": ["Model Y 2023"],
                "root_cause": "Incorrect brake pad compound specification",
                "affected_units": 3400,
                "severity": "critical",
                "date_identified": "2024-05-20",
                "capa_actions": [
                    "Update brake pad specifications",
                    "Voluntary recall initiated",
                    "Enhanced brake testing protocol"
                ],
                "status": "in_progress"
            },
            {
                "rca_id": "RCA-003",
                "issue": "Transmission slipping",
                "vehicle_models": ["Model Z 2022"],
                "root_cause": "Software calibration issue in TCM",
                "affected_units": 890,
                "severity": "high",
                "date_identified": "2024-07-10",
                "capa_actions": [
                    "OTA software update released",
                    "Service bulletin issued",
                    "Extended transmission warranty"
                ],
                "status": "closed"
            },
            {
                "rca_id": "RCA-004",
                "issue": "Battery drain in parked state",
                "vehicle_models": ["Model X 2023", "Model Y 2023"],
                "root_cause": "Parasitic draw from infotainment system",
                "affected_units": 2100,
                "severity": "medium",
                "date_identified": "2024-09-05",
                "capa_actions": [
                    "Software patch deployed",
                    "Power management optimization",
                    "Customer notification campaign"
                ],
                "status": "in_progress"
            }
        ]

    def _load_defect_records(self) -> List[Dict]:
        """Load mock manufacturing defect records"""

        return [
            {
                "defect_id": "DEF-001",
                "component": "Engine Oil Pan Gasket",
                "defect_type": "Material defect",
                "supplier": "Supplier A",
                "batch_number": "45211",
                "defect_rate": "8.2%",
                "detection_stage": "Field failures",
                "financial_impact": "₹12,50,000"
            },
            {
                "defect_id": "DEF-002",
                "component": "Brake Pads",
                "defect_type": "Design specification error",
                "supplier": "In-house",
                "batch_number": "N/A",
                "defect_rate": "12.5%",
                "detection_stage": "Customer complaints",
                "financial_impact": "₹45,00,000"
            },
            {
                "defect_id": "DEF-003",
                "component": "Transmission Control Module",
                "defect_type": "Software bug",
                "supplier": "Supplier B",
                "batch_number": "SW-v2.3.1",
                "defect_rate": "5.1%",
                "detection_stage": "Post-delivery",
                "financial_impact": "₹8,90,000"
            }
        ]

    def process(self, input_data: Dict[str, Any]) -> AgentResponse:
        """
        Main RCA/CAPA analysis process.

        Args:
            input_data: Must contain:
                - diagnosis_results: Recent diagnosis from Diagnosis Agent
                - maintenance_history: Historical maintenance data
                - feedback_data: Customer feedback (optional)

        Returns:
            AgentResponse with manufacturing insights
        """

        try:
            self.log_activity("="*80)
            self.log_activity("MANUFACTURING QUALITY INSIGHTS ANALYSIS")
            self.log_activity("="*80)

            diagnosis_results = input_data.get("diagnosis_results", [])
            maintenance_history = input_data.get("maintenance_history", [])
            feedback_data = input_data.get("feedback_data", [])

            self.log_activity(f"Diagnosis records to analyze: {len(diagnosis_results)}")
            self.log_activity(f"Maintenance history records: {len(maintenance_history)}")
            self.log_activity(f"Feedback records: {len(feedback_data)}")

            #Identify patterns and recurring issues
            self.log_activity("\n[STEP 1] Detecting patterns and recurring issues...")
            patterns = self._detect_patterns(diagnosis_results, maintenance_history)

            #Cross-reference with historical RCA data
            self.log_activity("\n[STEP 2] Cross-referencing with historical RCA data...")
            rca_matches = self._cross_reference_rca(patterns)

            #Perform root cause analysis for new patterns
            self.log_activity("\n[STEP 3] Performing root cause analysis...")
            rca_analysis = self._perform_rca(patterns, rca_matches, feedback_data)

            #Generate CAPA recommendations
            self.log_activity("\n[STEP 4] Generating CAPA recommendations...")
            capa_recommendations = self._generate_capa(rca_analysis)

            #Create manufacturing feedback report
            self.log_activity("\n[STEP 5] Creating manufacturing feedback report...")
            manufacturing_feedback = self._create_manufacturing_feedback(
                patterns, rca_analysis, capa_recommendations
            )

            # Compile insights
            insights_result = {
                "insights_id": f"MFG-INSIGHTS-{uuid.uuid4().hex[:8]}",
                "generated_at": datetime.now().isoformat(),
                "analysis_scope": {
                    "diagnosis_records": len(diagnosis_results),
                    "maintenance_records": len(maintenance_history),
                    "feedback_records": len(feedback_data)
                },

                # Core analysis
                "patterns_detected": patterns,
                "rca_analysis": rca_analysis,
                "capa_recommendations": capa_recommendations,

                # Manufacturing feedback
                "manufacturing_feedback_report": manufacturing_feedback,

                # Actionable insights
                "priority_actions": self._extract_priority_actions(capa_recommendations),
                "estimated_impact": self._estimate_impact(patterns),

                # Follow-up required
                "requires_manufacturing_action": len(capa_recommendations) > 0,
                "requires_design_review": any(c.get("requires_design_change", False)
                                             for c in capa_recommendations)
            }

            # Store insights
            self.insights_generated.append(insights_result)

            self.log_activity("\n" + "="*80)
            self.log_activity("MANUFACTURING INSIGHTS GENERATED")
            self.log_activity(f"Patterns detected: {len(patterns)}")
            self.log_activity(f"CAPA recommendations: {len(capa_recommendations)}")
            self.log_activity(f"Priority actions: {len(insights_result['priority_actions'])}")
            self.log_activity("="*80)

            return self.create_response(
                success=True,
                data=insights_result,
                next_action="send_to_manufacturing"
            )

        except Exception as e:
            self.log_activity(f"Manufacturing insights generation failed: {str(e)}", level="ERROR")
            return self.create_response(
                success=False,
                data={},
                error=str(e)
            )

    def _detect_patterns(self, diagnosis_results: List[Dict],
                        maintenance_history: List[Dict]) -> List[Dict]:
        """Detect patterns and recurring issues"""

        # Aggregate issues by component
        component_issues = {}

        # From diagnosis
        for diag in diagnosis_results:
            for rec in diag.get("service_recommendations", []):
                component = rec.get("component", "Unknown")
                issue_type = rec.get("service_type", "Unknown")

                key = f"{component}::{issue_type}"
                if key not in component_issues:
                    component_issues[key] = {
                        "component": component,
                        "issue_type": issue_type,
                        "occurrences": 0,
                        "vehicles_affected": set(),
                        "severity_levels": [],
                        "first_detected": None,
                        "last_detected": None
                    }

                component_issues[key]["occurrences"] += 1
                component_issues[key]["vehicles_affected"].add(diag.get("vehicle_id"))
                component_issues[key]["severity_levels"].append(diag.get("priority", "medium"))

                detected_time = diag.get("timestamp")
                if not component_issues[key]["first_detected"]:
                    component_issues[key]["first_detected"] = detected_time
                component_issues[key]["last_detected"] = detected_time

        # From maintenance history
        for maint in maintenance_history:
            for issue in maint.get("issues_found", []):
                component = issue.get("component", "Unknown")
                issue_type = issue.get("type", "Unknown")

                key = f"{component}::{issue_type}"
                if key not in component_issues:
                    component_issues[key] = {
                        "component": component,
                        "issue_type": issue_type,
                        "occurrences": 0,
                        "vehicles_affected": set(),
                        "severity_levels": [],
                        "first_detected": None,
                        "last_detected": None
                    }

                component_issues[key]["occurrences"] += 1
                component_issues[key]["vehicles_affected"].add(maint.get("vehicle_id"))

        # Identify recurring patterns (occurrence > threshold)
        recurring_threshold = 3
        patterns = []

        for key, data in component_issues.items():
            if data["occurrences"] >= recurring_threshold:
                pattern = {
                    "pattern_id": f"PTN-{uuid.uuid4().hex[:8]}",
                    "component": data["component"],
                    "issue_type": data["issue_type"],
                    "occurrence_count": data["occurrences"],
                    "vehicles_affected_count": len(data["vehicles_affected"]),
                    "severity": max(data["severity_levels"], key=data["severity_levels"].count) if data["severity_levels"] else "medium",
                    "recurrence_rate": self._calculate_recurrence_rate(data),
                    "is_recurring_defect": data["occurrences"] >= recurring_threshold,
                    "time_span_days": self._calculate_time_span(
                        data["first_detected"], data["last_detected"]
                    )
                }
                patterns.append(pattern)

                self.log_activity(f"  🔍 Pattern detected: {pattern['component']} - {pattern['issue_type']} "
                                f"({pattern['occurrence_count']} occurrences)")

        return patterns

    def _calculate_recurrence_rate(self, data: Dict) -> float:
        """Calculate how frequently issue recurs"""

        if data["first_detected"] and data["last_detected"]:
            days_span = self._calculate_time_span(data["first_detected"], data["last_detected"])
            if days_span > 0:
                return round(data["occurrences"] / days_span, 3)

        return 0.0

    def _calculate_time_span(self, first: str, last: str) -> int:
        """Calculate days between first and last occurrence"""

        if not first or not last:
            return 0

        try:
            first_dt = datetime.fromisoformat(first)
            last_dt = datetime.fromisoformat(last)
            return (last_dt - first_dt).days
        except:
            return 0

    def _cross_reference_rca(self, patterns: List[Dict]) -> List[Dict]:
        """Cross-reference patterns with historical RCA data"""

        matches = []

        for pattern in patterns:
            for rca in self.historical_rca_data:
                # Check if pattern matches historical RCA
                if (pattern["component"].lower() in rca["issue"].lower() or
                    pattern["issue_type"].lower() in rca["issue"].lower()):

                    match = {
                        "pattern_id": pattern["pattern_id"],
                        "matched_rca": rca,
                        "match_confidence": "high",
                        "known_root_cause": rca["root_cause"],
                        "existing_capa": rca["capa_actions"],
                        "capa_status": rca["status"]
                    }
                    matches.append(match)

                    self.log_activity(f"RCA match found: {pattern['component']} matches {rca['rca_id']}")

        return matches

    def _perform_rca(self, patterns: List[Dict], rca_matches: List[Dict],
                    feedback_data: List[Dict]) -> List[Dict]:
        """Perform root cause analysis for patterns"""

        rca_analysis = []

        for pattern in patterns:
            # Check if there's already an RCA match
            existing_rca = next((m for m in rca_matches
                               if m["pattern_id"] == pattern["pattern_id"]), None)

            if existing_rca:
                # Use existing RCA
                analysis = {
                    "rca_id": f"RCA-REF-{pattern['pattern_id']}",
                    "pattern_id": pattern["pattern_id"],
                    "analysis_type": "historical_reference",
                    "root_cause": existing_rca["known_root_cause"],
                    "confidence": "high",
                    "supporting_data": {
                        "historical_rca": existing_rca["matched_rca"]["rca_id"],
                        "previous_affected_units": existing_rca["matched_rca"]["affected_units"],
                        "existing_capa_status": existing_rca["capa_status"]
                    }
                }
            else:
                # Perform new RCA analysis
                analysis = self._analyze_new_root_cause(pattern, feedback_data)

            rca_analysis.append(analysis)

        return rca_analysis

    def _analyze_new_root_cause(self, pattern: Dict, feedback_data: List[Dict]) -> Dict:
        """Analyze root cause for new pattern using AI"""

        try:
            prompt = ChatPromptTemplate.from_template("""You are a quality engineer performing root cause analysis.

PATTERN DETECTED:
Component: {component}
Issue Type: {issue_type}
Occurrence Count: {occurrences}
Vehicles Affected: {vehicles_affected}
Severity: {severity}

CUSTOMER FEEDBACK MENTIONS:
{feedback_mentions}

Perform a root cause analysis and provide:
1. Most likely root cause
2. Contributing factors
3. Whether this is design, manufacturing, or supplier related
4. Confidence level (low/medium/high)
5. Recommended investigation steps

Respond in JSON format with keys: root_cause, contributing_factors, cause_category, confidence, investigation_steps""")

            # Extract relevant feedback
            feedback_mentions = self._extract_relevant_feedback(pattern, feedback_data)

            chain = prompt | self.llm

            response = chain.invoke({
                "component": pattern["component"],
                "issue_type": pattern["issue_type"],
                "occurrences": pattern["occurrence_count"],
                "vehicles_affected": pattern["vehicles_affected_count"],
                "severity": pattern["severity"],
                "feedback_mentions": feedback_mentions if feedback_mentions else "No specific customer feedback available"
            })

            # Parse response (simplified)
            return {
                "rca_id": f"RCA-NEW-{pattern['pattern_id']}",
                "pattern_id": pattern["pattern_id"],
                "analysis_type": "new_analysis",
                "root_cause": "AI-identified root cause pending validation",
                "confidence": "medium",
                "ai_analysis": response.content[:500],
                "requires_further_investigation": True
            }

        except Exception as e:
            self.log_activity(f"AI RCA failed: {str(e)}", level="WARNING")

            # Fallback
            return {
                "rca_id": f"RCA-NEW-{pattern['pattern_id']}",
                "pattern_id": pattern["pattern_id"],
                "analysis_type": "preliminary",
                "root_cause": f"Recurring {pattern['issue_type']} in {pattern['component']} - requires investigation",
                "confidence": "low",
                "requires_further_investigation": True
            }

    def _extract_relevant_feedback(self, pattern: Dict, feedback_data: List[Dict]) -> str:
        """Extract customer feedback relevant to pattern"""

        relevant = []

        for fb in feedback_data:
            comments = fb.get("customer_feedback", {}).get("comments", "")

            if (pattern["component"].lower() in comments.lower() or
                pattern["issue_type"].lower() in comments.lower()):
                relevant.append(comments)

        return "; ".join(relevant[:3]) if relevant else ""

    def _generate_capa(self, rca_analysis: List[Dict]) -> List[Dict]:
        """Generate CAPA (Corrective and Preventive Actions) recommendations"""

        capa_recommendations = []

        for rca in rca_analysis:
            capa = {
                "capa_id": f"CAPA-{uuid.uuid4().hex[:8]}",
                "related_rca": rca["rca_id"],
                "generated_at": datetime.now().isoformat(),

                # Corrective actions (fix current issues)
                "corrective_actions": [],

                # Preventive actions (prevent future occurrences)
                "preventive_actions": [],

                # Implementation details
                "priority": "high" if rca.get("confidence") == "high" else "medium",
                "estimated_implementation_time": "2-4 weeks",
                "requires_design_change": False,
                "requires_supplier_action": False,
                "estimated_cost": "₹5,00,000 - ₹20,00,000"
            }

            # Generate action recommendations based on RCA type
            if rca.get("analysis_type") == "historical_reference":
                # Reference existing CAPA
                existing_capa = rca.get("supporting_data", {}).get("existing_capa_status")

                capa["corrective_actions"] = [
                    "Review and reimplement previous CAPA actions",
                    "Investigate why previous CAPA was insufficient",
                    "Conduct field inspection of affected vehicles"
                ]

                capa["preventive_actions"] = [
                    "Strengthen quality controls",
                    "Update inspection protocols",
                    "Enhance supplier monitoring"
                ]
            else:
                # New CAPA
                capa["corrective_actions"] = [
                    f"Immediate inspection of {rca.get('pattern_id')} affected units",
                    "Issue service bulletin to dealerships",
                    "Initiate customer notification campaign"
                ]

                capa["preventive_actions"] = [
                    "Root cause investigation team formation",
                    "Design review for component improvement",
                    "Supplier audit and qualification review",
                    "Enhanced incoming quality inspection"
                ]

                capa["requires_design_change"] = True
                capa["requires_supplier_action"] = True

            capa_recommendations.append(capa)

            self.log_activity(f"  📋 CAPA generated: {capa['capa_id']} "
                            f"({len(capa['corrective_actions'])} corrective, "
                            f"{len(capa['preventive_actions'])} preventive actions)")

        return capa_recommendations

    def _create_manufacturing_feedback(self, patterns: List[Dict],
                                      rca_analysis: List[Dict],
                                      capa_recommendations: List[Dict]) -> Dict:
        """Create comprehensive manufacturing feedback report"""

        report = {
            "report_id": f"MFG-FB-{uuid.uuid4().hex[:8]}",
            "generated_at": datetime.now().isoformat(),
            "report_type": "Quality Improvement Feedback",

            "executive_summary": {
                "total_patterns_detected": len(patterns),
                "recurring_defects_identified": sum(1 for p in patterns if p["is_recurring_defect"]),
                "rca_completed": len(rca_analysis),
                "capa_recommendations": len(capa_recommendations),
                "requires_immediate_action": sum(1 for c in capa_recommendations if c["priority"] == "high")
            },

            "critical_findings": [
                {
                    "finding": pattern,
                    "root_cause": next((r for r in rca_analysis if r["pattern_id"] == pattern["pattern_id"]), {}),
                    "recommended_capa": next((c for c in capa_recommendations
                                            if c["related_rca"].endswith(pattern["pattern_id"])), {})
                }
                for pattern in patterns if pattern["severity"] in ["high", "critical"]
            ],

            "design_improvement_suggestions": self._generate_design_suggestions(patterns, rca_analysis),

            "supplier_quality_issues": self._identify_supplier_issues(patterns, rca_analysis),

            "process_improvement_recommendations": [
                "Implement real-time quality monitoring",
                "Enhance predictive failure detection",
                "Strengthen supplier qualification process",
                "Increase test coverage for identified failure modes"
            ],

            "estimated_quality_cost_impact": self._calculate_quality_costs(patterns),

            "next_steps": [
                "Schedule manufacturing review meeting",
                "Initiate design review for affected components",
                "Conduct supplier audits where applicable",
                "Update quality control procedures",
                "Track CAPA implementation progress"
            ]
        }

        return report

    def _generate_design_suggestions(self, patterns: List[Dict],
                                    rca_analysis: List[Dict]) -> List[Dict]:
        """Generate design improvement suggestions"""

        suggestions = []

        for pattern in patterns:
            if pattern["is_recurring_defect"]:
                suggestion = {
                    "component": pattern["component"],
                    "current_issue": pattern["issue_type"],
                    "suggestion": f"Redesign {pattern['component']} to address recurring {pattern['issue_type']}",
                    "priority": "high" if pattern["severity"] == "critical" else "medium",
                    "expected_improvement": "50-80% reduction in failure rate"
                }
                suggestions.append(suggestion)

        return suggestions

    def _identify_supplier_issues(self, patterns: List[Dict],
                                  rca_analysis: List[Dict]) -> List[Dict]:
        """Identify supplier-related quality issues"""

        supplier_issues = []

        # Check against defect records
        for pattern in patterns:
            for defect in self.manufacturing_defect_records:
                if pattern["component"].lower() in defect["component"].lower():
                    supplier_issues.append({
                        "component": pattern["component"],
                        "supplier": defect["supplier"],
                        "defect_rate": defect["defect_rate"],
                        "recommended_action": "Supplier audit required" if defect["supplier"] != "In-house" else "Internal process review"
                    })

        return supplier_issues

    def _calculate_quality_costs(self, patterns: List[Dict]) -> Dict:
        """Calculate estimated quality costs"""

        # Mock cost calculation
        total_occurrences = sum(p["occurrence_count"] for p in patterns)
        avg_repair_cost = 8000  # ₹

        return {
            "total_incidents": total_occurrences,
            "estimated_repair_costs": f"₹{total_occurrences * avg_repair_cost:,}",
            "estimated_warranty_impact": f"₹{int(total_occurrences * avg_repair_cost * 1.5):,}",
            "potential_cost_avoidance_with_capa": f"₹{int(total_occurrences * avg_repair_cost * 0.7):,}"
        }

    def _extract_priority_actions(self, capa_recommendations: List[Dict]) -> List[Dict]:
        """Extract priority actions from CAPA"""

        priority_actions = []

        for capa in capa_recommendations:
            if capa["priority"] == "high":
                action = {
                    "action_id": f"ACT-{uuid.uuid4().hex[:8]}",
                    "capa_id": capa["capa_id"],
                    "action_type": "immediate",
                    "description": capa["corrective_actions"][0] if capa["corrective_actions"] else "Review required",
                    "owner": "Manufacturing Quality Team",
                    "deadline": (datetime.now() + timedelta(days=14)).strftime("%Y-%m-%d"),
                    "requires_design_review": capa["requires_design_change"]
                }
                priority_actions.append(action)

        return priority_actions

    def _estimate_impact(self, patterns: List[Dict]) -> Dict:
        """Estimate overall impact of identified patterns"""

        total_vehicles = sum(p["vehicles_affected_count"] for p in patterns)
        critical_patterns = sum(1 for p in patterns if p["severity"] in ["high", "critical"])

        return {
            "vehicles_potentially_affected": total_vehicles,
            "critical_patterns": critical_patterns,
            "estimated_recall_risk": "Low" if critical_patterns == 0 else "Medium" if critical_patterns <= 2 else "High",
            "quality_improvement_potential": "High - systematic issues identified",
            "customer_satisfaction_impact": "Implementing CAPA can improve satisfaction by 15-25%"
        }

    def generate_insights_summary(self) -> Dict:
        """Generate summary of all insights generated"""

        if not self.insights_generated:
            return {"message": "No insights generated yet"}

        total_patterns = sum(len(i["patterns_detected"]) for i in self.insights_generated)
        total_capa = sum(len(i["capa_recommendations"]) for i in self.insights_generated)

        return {
            "total_insights_generated": len(self.insights_generated),
            "total_patterns_detected": total_patterns,
            "total_capa_recommendations": total_capa,
            "latest_insight_date": self.insights_generated[-1]["generated_at"]
        }

## Multi-Agent Orchestration Framework

Implement the orchestration system that coordinates communication between all agents using a graph-based workflow.

In [29]:
"""
Master Agent - LangGraph Orchestrator
Competition Requirement:
"Master Agent (Main Orchestrator): Monitors vehicle health data streams and
overall conversational flow. Coordinates Worker Agents using LangGraph"
"""

from typing import Dict, Any, List, TypedDict
from datetime import datetime
import uuid

from langgraph.graph import StateGraph, END

class WorkflowState(TypedDict):
    workflow_id: str
    vehicle_id: str
    customer_id: str
    started_at: str
    current_stage: str

    # Inputs
    vehicle_info: Dict
    customer_info: Dict
    telemetry_data: Dict
    maintenance_history: List[Dict]
    scheduling_preferences: Dict

    # Outputs
    data_analysis_result: Dict
    diagnosis_result: Dict
    engagement_result: Dict
    scheduling_result: Dict
    service_completion: Dict
    feedback_result: Dict
    manufacturing_insights: Dict

    requires_maintenance: bool
    appointment_interest: bool
    workflow_status: str

    stages_completed: List[str]
    errors: List[str]

class MasterAgent(BaseAgent):
    def __init__(self, telemetry_api, maintenance_history_df):
        # Call BaseAgent constructor
        super().__init__(agent_name="MasterAgent")

        self.telemetry_api = telemetry_api
        self.maintenance_history_df = maintenance_history_df

        # Initialize worker agents here *after* super()
        self.data_analysis_agent = DataAnalysisAgentV2(telemetry_api, maintenance_history_df)
        self.diagnosis_agent = DiagnosisAgentV2()
        self.customer_engagement_agent = CustomerEngagementAgentV2()
        self.scheduling_agent = SchedulingAgent()
        self.feedback_agent = FeedbackAgent()
        self.manufacturing_insights = ManufacturingQualityInsightsModule()

        self.workflow_graph = self._build_workflow_graph()
        self.log_activity("Master Agent initialized ")

    def _build_workflow_graph(self):
        graph = StateGraph(WorkflowState)

        graph.add_node("init", self._init_workflow)
        graph.add_node("data_analysis", self._run_data_analysis)
        graph.add_node("diagnosis", self._run_diagnosis)
        graph.add_node("engagement", self._run_engagement)
        graph.add_node("scheduling", self._run_scheduling)
        graph.add_node("service", self._run_service)
        graph.add_node("feedback", self._run_feedback)
        graph.add_node("mfg_insights", self._run_manufacturing)
        graph.add_node("final", self._finalize_workflow)

        graph.set_entry_point("init")
        graph.add_edge("init", "data_analysis")
        graph.add_edge("data_analysis", "diagnosis")

        graph.add_conditional_edges(
            "diagnosis",
            self._should_engage,
            {"yes": "engagement", "no": "final"}
        )

        graph.add_conditional_edges(
            "engagement",
            self._should_schedule,
            {"yes": "scheduling", "no": "final"}
        )

        graph.add_edge("scheduling", "service")
        graph.add_edge("service", "feedback")

        graph.add_conditional_edges(
            "feedback",
            self._should_generate_insights,
            {"yes": "mfg_insights", "no": "final"}
        )

        graph.add_edge("mfg_insights", "final")
        graph.add_edge("final", END)

        return graph.compile()

    def _init_workflow(self, state: WorkflowState) -> WorkflowState:
        state["workflow_status"] = "running"
        state["stages_completed"] = ["initialized"]
        return state

    def _run_data_analysis(self, state: WorkflowState):
        result = self.data_analysis_agent.process({
            "telemetry_data": state["telemetry_data"],
            "vehicle_info": state["vehicle_info"],
            "maintenance_history": state["maintenance_history"]
        })
        state["data_analysis_result"] = result.data
        state["stages_completed"].append("data_analysis")
        return state

    def _run_diagnosis(self, state: WorkflowState):
        result = self.diagnosis_agent.process({
            "analysis_result": state["data_analysis_result"],
            "vehicle_info": state["vehicle_info"],
            "maintenance_history": state["maintenance_history"]
        })
        state["diagnosis_result"] = result.data
        state["requires_maintenance"] = result.data.get("requires_maintenance", True)
        state["stages_completed"].append("diagnosis")
        return state

    def _run_engagement(self, state: WorkflowState):
        result = self.customer_engagement_agent.process({
            "diagnosis_result": state["diagnosis_result"],
            "customer_info": state["customer_info"],
            "vehicle_info": state["vehicle_info"]
        })
        state["engagement_result"] = result.data
        state["appointment_interest"] = result.data.get("appointment_interest", False)
        state["stages_completed"].append("engagement")
        return state

    def _run_scheduling(self, state: WorkflowState):
        result = self.scheduling_agent.process({
            "engagement_result": state["engagement_result"],
            "scheduling_preferences": state.get("scheduling_preferences", {})
        })
        state["scheduling_result"] = result.data
        state["stages_completed"].append("scheduling")
        return state

    def _run_service(self, state: WorkflowState):
        appointment = state["scheduling_result"]["appointment"]
        state["service_completion"] = {
            "completed_at": datetime.now().isoformat(),
            "appointment_id": appointment["appointment_id"],
            "work_performed": state["diagnosis_result"].get("service_recommendations", [])
        }
        state["stages_completed"].append("service")
        return state

    def _run_feedback(self, state: WorkflowState):
        result = self.feedback_agent.process({
            "appointment": state["scheduling_result"]["appointment"],
            "service_completion": state["service_completion"]
        })
        state["feedback_result"] = result.data
        state["stages_completed"].append("feedback")
        return state

    def _run_manufacturing(self, state: WorkflowState):
        result = self.manufacturing_insights.process({
            "diagnosis_results": [state["diagnosis_result"]],
            "maintenance_history": state["maintenance_history"],
            "feedback_data": [state["feedback_result"]]
        })
        state["manufacturing_insights"] = result.data
        state["stages_completed"].append("manufacturing_insights")
        return state

    def _finalize_workflow(self, state: WorkflowState):
        state["workflow_status"] = "completed"
        state["current_stage"] = "finalized"
        return state

    def process(self, input_data):
        """
        MasterAgent does not process raw input like other agents.
        Redirect to main workflow.
        """
        return self.process_vehicle_predictive_maintenance(input_data)

    def get_prompt(self):
        """
        MasterAgent orchestrates other agents and does not use LLM prompts.
        Returning static placeholder.
        """
        return "MasterAgent orchestrates subordinate agents; no prompt required."


    def _should_engage(self, state: WorkflowState) -> str:
        return "yes" if state["requires_maintenance"] else "no"

    def _should_schedule(self, state: WorkflowState) -> str:
        return "yes" if state["appointment_interest"] else "no"

    def _should_generate_insights(self, state: WorkflowState) -> str:
        rating = state["feedback_result"].get("overall_satisfaction", 5)
        return "yes" if rating < 3.5 else "no"

    def process_vehicle_predictive_maintenance(self, data: Dict[str, Any]) -> AgentResponse:
        workflow_id = f"WF-{uuid.uuid4().hex[:8]}"

        initial_state = WorkflowState(
            workflow_id=workflow_id,
            vehicle_id=data["vehicle_info"]["vehicle_id"],
            customer_id=data["customer_info"]["customer_id"],
            started_at=datetime.now().isoformat(),
            current_stage="pending",

            vehicle_info=data["vehicle_info"],
            customer_info=data["customer_info"],
            telemetry_data=data["telemetry_data"],
            maintenance_history=data["maintenance_history"],
            scheduling_preferences=data.get("scheduling_preferences", {}),

            data_analysis_result={},
            diagnosis_result={},
            engagement_result={},
            scheduling_result={},
            service_completion={},
            feedback_result={},
            manufacturing_insights={},

            requires_maintenance=True,
            appointment_interest=False,
            workflow_status="pending",
            stages_completed=[],
            errors=[]
        )

        final_state = self.workflow_graph.invoke(initial_state)

        return self.create_response(
            success=True,
            data=final_state,
            next_action="complete"
        )



In [ ]:
"""
Complete System Demo - Automotive Predictive Maintenance with Agentic AI
"""

import pandas as pd
import json
from datetime import datetime, timedelta
from typing import Dict, List, Optional
import numpy as np

class DataLoader:
    def __init__(self, telemetry_csv: str, maintenance_csv: str, customer_csv: str):

        self.telemetry_df = pd.read_csv(telemetry_csv)
        self.maintenance_df = pd.read_csv(maintenance_csv)
        self.customer_df = pd.read_csv(customer_csv)

        # Get unique vehicles
        self.available_vehicles = self.telemetry_df['vehicle_id'].unique()
        print(f"\n Available vehicles: {len(self.available_vehicles)}")
        for i, vid in enumerate(self.available_vehicles[:5], 1):
            print(f"  {i}. {vid}")
        if len(self.available_vehicles) > 5:
            print(f"  ... and {len(self.available_vehicles) - 5} more")

    def get_vehicle_telemetry(self, vehicle_id: str = None, num_records: int = 100) -> Dict:
        """
        Get telemetry data for a specific vehicle.

        Returns structured telemetry with:
        - Latest readings
        - Historical data
        - Failure predictions from dataset
        """

        # If no vehicle_id, pick first one
        if vehicle_id is None:
            vehicle_id = self.available_vehicles[0]

        # Filter data for this vehicle
        vehicle_data = self.telemetry_df[self.telemetry_df['vehicle_id'] == vehicle_id].copy()

        if len(vehicle_data) == 0:
            print(f" No data found for {vehicle_id}, using first available vehicle")
            vehicle_id = self.available_vehicles[0]
            vehicle_data = self.telemetry_df[self.telemetry_df['vehicle_id'] == vehicle_id].copy()

        # Sort by timestamp
        if 'timestamp' in vehicle_data.columns:
            vehicle_data['timestamp'] = pd.to_datetime(vehicle_data['timestamp'], errors='coerce')
            vehicle_data = vehicle_data.sort_values('timestamp')

        # Get most recent records
        if len(vehicle_data) > num_records:
            vehicle_data = vehicle_data.tail(num_records)

        # Get latest reading
        latest = vehicle_data.iloc[-1].to_dict()

        # Structure telemetry data properly
        telemetry = {
            "vehicle_id": vehicle_id,
            "timestamp": datetime.now().isoformat(),
            "records_count": len(vehicle_data),

            # Latest readings structured by system
            "latest_readings": {
                "odometer_reading": float(latest.get('odometer_reading', 0)),

                "engine": {
                    "temp_c": float(latest.get('engine_temp_c', 0)),
                    "oil_pressure_psi": float(latest.get('oil_pressure_psi', 0)),
                    "coolant_temp_c": float(latest.get('coolant_temp_c', 0)),
                    "vibration_level": float(latest.get('vibration_level', 0)),
                    "failure_imminent": int(latest.get('engine_failure_imminent', 0))
                },

                "brakes": {
                    "fluid_level_psi": float(latest.get('brake_fluid_level_psi', 0)),
                    "pad_wear_mm": float(latest.get('brake_pad_wear_mm', 0)),
                    "temp_c": float(latest.get('brake_temp_c', 0)),
                    "abs_fault": int(latest.get('abs_fault_indicator', 0)),
                    "pedal_position_percent": float(latest.get('brake_pedal_pos_percent', 0)),
                    "issue_imminent": int(latest.get('brake_issue_imminent', 0))
                },

                "battery": {
                    "voltage_v": float(latest.get('battery_voltage_v', 0)),
                    "current_a": float(latest.get('battery_current_a', 0)),
                    "temp_c": float(latest.get('battery_temp_c', 0)),
                    "charge_percent": float(latest.get('battery_charge_percent', 0)),
                    "health_percent": float(latest.get('battery_health_percent', 0)),
                    "alternator_output_v": float(latest.get('alternator_output_v', 0)),
                    "issue_imminent": int(latest.get('battery_issue_imminent', 0))
                },

                "vehicle": {
                    "speed_kph": float(latest.get('vehicle_speed_kph', 0)),
                    "fuel_level_percent": float(latest.get('fuel_level_percent', 0)),
                    "ambient_temp_c": float(latest.get('ambient_temp_c', 0))
                },

                # Failure predictions from dataset
                "failure_predictions": {
                    "engine_failure_imminent": bool(latest.get('engine_failure_imminent', 0)),
                    "brake_issue_imminent": bool(latest.get('brake_issue_imminent', 0)),
                    "battery_issue_imminent": bool(latest.get('battery_issue_imminent', 0)),
                    "failure_date": str(latest.get('failure_date', '')),
                    "failure_type": str(latest.get('failure_type', ''))
                }
            },

            # Historical data for analysis
            "historical_records": vehicle_data.to_dict('records'),
            "raw_dataframe": vehicle_data,

            # Statistical summary
            "statistics": {
                "engine_temp_avg": float(vehicle_data['engine_temp_c'].mean()),
                "engine_temp_max": float(vehicle_data['engine_temp_c'].max()),
                "brake_pad_wear_min": float(vehicle_data['brake_pad_wear_mm'].min()),
                "battery_health_avg": float(vehicle_data['battery_health_percent'].mean()),
                "total_distance_km": float(vehicle_data['odometer_reading'].max() - vehicle_data['odometer_reading'].min())
            }
        }

        return telemetry

    def get_maintenance_history(self, vehicle_id: str) -> List[Dict]:
        """
        Get maintenance history for a specific vehicle.

        Returns list of maintenance records with proper structure.
        """

        # Filter for this vehicle
        vehicle_maint = self.maintenance_df[self.maintenance_df['vehicle_id'] == vehicle_id].copy()

        if len(vehicle_maint) == 0:
            # No maintenance history for this vehicle
            return []

        # Sort by service date
        if 'service_date' in vehicle_maint.columns:
            vehicle_maint['service_date'] = pd.to_datetime(vehicle_maint['service_date'], errors='coerce')
            vehicle_maint = vehicle_maint.sort_values('service_date', ascending=False)

        # Convert to structured records
        maintenance_records = []
        for _, row in vehicle_maint.iterrows():
            record = {
                "service_id": f"SVC-{row.get('vehicle_id', 'UNK')}-{len(maintenance_records)+1}",
                "service_date": str(row.get('service_date', '')),
                "service_type": str(row.get('service_type', '')),
                "odometer_km": int(row.get('odometer_km', 0)),
                "component": str(row.get('component', '')),
                "issue_description": str(row.get('issue_description', '')),
                "dtc_code": str(row.get('dtc_code', '')),
                "repair_action": str(row.get('repair_action', '')),
                "downtime_hours": int(row.get('downtime_hours', 0)),
                "workshop_city": str(row.get('workshop_city', '')),
                "under_warranty": str(row.get('under_warranty', ''))
            }
            maintenance_records.append(record)

        return maintenance_records

    def get_customer_info(self, vehicle_id: str) -> Dict:
        """
        Get customer information for a specific vehicle from customer database.
        """

        # Find customer record
        customer = self.customer_df[self.customer_df['vehicle_id'] == vehicle_id]

        if len(customer) == 0:
            # No customer record, generate mock
            return self._generate_mock_customer(vehicle_id)

        # Get first matching record
        cust_row = customer.iloc[0]

        # Map language based on location (simple heuristic)
        address = str(cust_row.get('Address', ''))
        language = self._infer_language_from_address(address)

        return {
            "customer_id": str(cust_row.get('Customer ID', f"CUST-{vehicle_id}")),
            "name": str(cust_row.get('Customer Name', 'Customer')),
            "phone": str(cust_row.get('Phone Number', '+91-00000-00000')),
            "email": str(cust_row.get('Email', 'customer@example.com')),
            "language": language,
            "location": address,
            "preferred_service_center": "SC001",  # Default

            # Additional customer data
            "vin": str(cust_row.get('VIN', '')),
            "license_plate": str(cust_row.get('License Plate', '')),
            "last_service_date": str(cust_row.get('Last Service Date', '')),
            "insurance_cover": str(cust_row.get('Insurance Cover', ''))
        }

    def get_vehicle_info(self, vehicle_id: str, telemetry_data: Dict) -> Dict:
        """
        Get vehicle information combining customer DB and telemetry.
        """

        # Get customer record for vehicle details
        customer = self.customer_df[self.customer_df['vehicle_id'] == vehicle_id]

        if len(customer) > 0:
            cust_row = customer.iloc[0]
            year = int(cust_row.get('Year', 2023))
            vin = str(cust_row.get('VIN', f"VIN-{vehicle_id}"))
            license_plate = str(cust_row.get('License Plate', f"MH-12-AB-{vehicle_id[-4:]}"))
            last_service = str(cust_row.get('Last Service Date', ''))
        else:
            year = 2023
            vin = f"VIN-{vehicle_id}"
            license_plate = f"MH-12-AB-{vehicle_id[-4:]}"
            last_service = ""

        # Get mileage from telemetry
        odometer = telemetry_data.get('latest_readings', {}).get('odometer_reading', 50000)

        # Infer model from vehicle_id pattern or use generic
        model = self._infer_vehicle_model(vehicle_id)

        return {
            "vehicle_id": vehicle_id,
            "model": model,
            "year": year,
            "vin": vin,
            "mileage": int(odometer),
            "registration_number": license_plate,
            "last_service_date": last_service
        }

    def _infer_vehicle_model(self, vehicle_id: str) -> str:
        """Infer vehicle model from ID"""

        models = {
            "V001": "Maruti Suzuki Swift 2023",
            "V002": "Hyundai Creta 2023",
            "V003": "Tata Nexon EV 2023",
            "V004": "Mahindra XUV700 2023",
            "V005": "Honda City 2023",
            "V006": "Kia Seltos 2023",
            "V007": "Toyota Innova Crysta 2023",
            "V008": "Maruti Suzuki Vitara Brezza 2023",
            "V009": "Hyundai Venue 2023",
            "V010": "Tata Harrier 2023"
        }

        return models.get(vehicle_id, "Unknown Model 2023")

    def _infer_language_from_address(self, address: str) -> str:
        """Infer preferred language from address"""

        address_lower = address.lower()

        if 'mumbai' in address_lower or 'pune' in address_lower or 'maharashtra' in address_lower:
            return "Marathi"
        elif 'delhi' in address_lower or 'ncr' in address_lower:
            return "Hindi"
        elif 'chennai' in address_lower or 'tamil nadu' in address_lower:
            return "Tamil"
        elif 'hyderabad' in address_lower or 'telangana' in address_lower:
            return "Telugu"
        elif 'bangalore' in address_lower or 'karnataka' in address_lower:
            return "Kannada"
        else:
            return "English"

    def _generate_mock_customer(self, vehicle_id: str) -> Dict:
        """Generate mock customer if not in database"""

        import random

        names = ["Rajesh Kumar", "Priya Sharma", "Arjun Reddy", "Lakshmi Iyer",
                 "Vikram Singh", "Anjali Desai", "Karthik Nair", "Sneha Patel"]

        return {
            "customer_id": f"CUST-{abs(hash(vehicle_id)) % 100000:05d}",
            "name": random.choice(names),
            "phone": f"+91-{random.randint(70000,99999)}-{random.randint(10000,99999)}",
            "email": f"customer{vehicle_id[-4:]}@example.com",
            "language": "English",
            "location": "Mumbai, Maharashtra",
            "preferred_service_center": "SC001"
        }

    def get_available_vehicles(self) -> List[str]:
        """Get list of all available vehicles"""
        return list(self.available_vehicles)

    def analyze_dataset_health(self):
        """Analyze and display dataset health/statistics"""

        print("\n" + "="*80)
        print("DATASET HEALTH ANALYSIS")
        print("="*80)

        # Telemetry analysis
        print("\n TELEMETRY DATA:")
        print(f"  Total records: {len(self.telemetry_df)}")
        print(f"  Unique vehicles: {self.telemetry_df['vehicle_id'].nunique()}")
        print(f"  Date range: {self.telemetry_df['timestamp'].min()} to {self.telemetry_df['timestamp'].max()}")

        # Failure indicators
        engine_failures = self.telemetry_df['engine_failure_imminent'].sum()
        brake_issues = self.telemetry_df['brake_issue_imminent'].sum()
        battery_issues = self.telemetry_df['battery_issue_imminent'].sum()

        print(f"\n FAILURE INDICATORS:")
        print(f"  Engine failures predicted: {engine_failures} records ({engine_failures/len(self.telemetry_df)*100:.1f}%)")
        print(f"  Brake issues predicted: {brake_issues} records ({brake_issues/len(self.telemetry_df)*100:.1f}%)")
        print(f"  Battery issues predicted: {battery_issues} records ({battery_issues/len(self.telemetry_df)*100:.1f}%)")

        # Maintenance analysis
        print(f"\n🔧 MAINTENANCE DATA:")
        print(f"  Total service records: {len(self.maintenance_df)}")
        print(f"  Vehicles with service history: {self.maintenance_df['vehicle_id'].nunique()}")
        print(f"  Service types: {', '.join(self.maintenance_df['service_type'].unique()[:5])}")

        # Customer analysis
        print(f"\n CUSTOMER DATA:")
        print(f"  Total customers: {len(self.customer_df)}")
        print(f"  Vehicles in customer DB: {self.customer_df['vehicle_id'].nunique()}")

        print("="*80)


def run_single_vehicle_demo(data_loader: DataLoader, vehicle_id: str = None):
    """
    Run complete demo for a single vehicle with real data.
    """

    print("\n" + "="*80)
    print("SINGLE VEHICLE PREDICTIVE MAINTENANCE DEMO")
    print("="*80)

    # Get vehicle list if no ID specified
    if vehicle_id is None:
        available = data_loader.get_available_vehicles()
        vehicle_id = available[0]
        print(f"\nNo vehicle specified, using: {vehicle_id}")

    # Load all data for vehicle
    print(f"\n Loading data for vehicle: {vehicle_id}")
    telemetry_data = data_loader.get_vehicle_telemetry(vehicle_id)
    maintenance_history = data_loader.get_maintenance_history(vehicle_id)
    customer_info = data_loader.get_customer_info(vehicle_id)
    vehicle_info = data_loader.get_vehicle_info(vehicle_id, telemetry_data)

    print(f"  ✓ Telemetry records: {telemetry_data['records_count']}")
    print(f"  ✓ Maintenance records: {len(maintenance_history)}")
    print(f"  ✓ Customer: {customer_info['name']}")
    print(f"  ✓ Vehicle: {vehicle_info['model']}")

    # Check for imminent failures
    failures = telemetry_data['latest_readings']['failure_predictions']
    if failures['engine_failure_imminent']:
        print(f"\n CRITICAL: Engine failure imminent!")
        print(f"   Predicted date: {failures['failure_date']}")
        edge_case = "critical_urgent"
    elif failures['brake_issue_imminent']:
        print(f"\n WARNING: Brake issue imminent!")
        edge_case = "critical_urgent"
    elif failures['battery_issue_imminent']:
        print(f"\n WARNING: Battery issue imminent!")
        edge_case = None
    else:
        print(f"\n No imminent failures detected")
        edge_case = None

    # Check for recurring issues in maintenance history
    if len(maintenance_history) > 0:
        components = [m['component'] for m in maintenance_history]
        if len(components) != len(set(components)):
            edge_case = "recurring_defect"
            print(f"🔧 Recurring defect detected in maintenance history")

    # Prepare complete vehicle data
    vehicle_data = {
        "vehicle_info": vehicle_info,
        "customer_info": customer_info,
        "telemetry_data": telemetry_data,
        "maintenance_history": maintenance_history,
        "scheduling_preferences": {
            "preferred_time": "morning",
            "prefer_weekend": False,
            "blackout_dates": []
        }
    }

    # Display summary
    print(f"\n" + "-"*80)
    print(f" Customer: {customer_info['name']}")
    print(f" Location: {customer_info['location']}")
    print(f" Language: {customer_info['language']}")
    print(f" Phone: {customer_info['phone']}")
    print(f"\n Vehicle: {vehicle_info['model']} ({vehicle_info['year']})")
    print(f" Mileage: {vehicle_info['mileage']:,} km")
    print(f" VIN: {vehicle_info['vin']}")
    print(f" License: {vehicle_info['registration_number']}")

    # Show key telemetry
    latest = telemetry_data['latest_readings']
    print(f"\n Current Telemetry:")
    print(f"  Engine Temp: {latest['engine']['temp_c']:.1f}°C")
    print(f"  Oil Pressure: {latest['engine']['oil_pressure_psi']:.1f} PSI")
    print(f"  Brake Pad Wear: {latest['brakes']['pad_wear_mm']:.1f} mm")
    print(f"  Battery Health: {latest['battery']['health_percent']:.1f}%")
    print(f"  Battery Voltage: {latest['battery']['voltage_v']:.1f}V")
    print("-"*80)

    # Initialize Master Agent
    print("\n Initializing Master Agent with LangGraph...")
    master_agent = MasterAgent(
        telemetry_api=telemetry_data,
        maintenance_history_df=maintenance_history
    )


    # Execute workflow
    print("\n Executing predictive maintenance workflow...")
    input("\nPress Enter to start workflow...")

    result = master_agent.process_vehicle_predictive_maintenance(vehicle_data)

    # Display results summary
    print("\n" + "="*80)
    print("WORKFLOW COMPLETE - SUMMARY")
    print("="*80)

    if result.success:
        final_state = result.data
        print(f"\n Status: {final_state.get('workflow_status', 'Unknown').upper()}")
        print(f" Completed Stages: {len(final_state.get('stages_completed', []))}")

        for stage in final_state.get('stages_completed', []):
            print(f"  ✓ {stage}")

        if final_state.get('errors'):
            print(f"\n Errors encountered: {len(final_state['errors'])}")

        if final_state.get('ueba_alerts'):
            print(f"\n UEBA alerts: {len(final_state['ueba_alerts'])}")
    else:
        print(f"\n Workflow failed: {result.error}")

    print("="*80)

    return result


def main():
    """Main demo execution with your actual CSV files"""

    print("="*80)
    print("AUTOMOTIVE PREDICTIVE MAINTENANCE - AGENTIC AI SYSTEM")
    print("Hero MotoCorp + Mahindra & Mahindra Challenge")
    print("="*80)

    TELEMETRY_CSV = "/content/vehicle_telemetry.csv"
    MAINTENANCE_CSV = "/content/maintainance_history_new - Sheet1.csv"
    CUSTOMER_CSV = "/content/Customer Database - Sheet2.csv"
    # ========================================

    try:
        # Load datasets
        data_loader = DataLoader(TELEMETRY_CSV, MAINTENANCE_CSV, CUSTOMER_CSV)

        # Analyze dataset health
        data_loader.analyze_dataset_health()

        # Demo menu
        print("\n" + "="*80)
        print("DEMO OPTIONS")
        print("="*80)
        print("1. Single Vehicle Demo (Auto-select)")
        print("2. Choose Specific Vehicle")
        print("3. Process All Vehicles (Batch)")
        print("4. Process Vehicles with Imminent Failures Only")
        print("="*80)

        choice = input("\nSelect option (1-4): ").strip() or "1"

        if choice == "1":
            # Auto-select first vehicle
            run_single_vehicle_demo(data_loader)

        elif choice == "2":
            # Choose specific vehicle
            vehicles = data_loader.get_available_vehicles()
            print("\nAvailable vehicles:")
            for i, vid in enumerate(vehicles, 1):
                print(f"  {i}. {vid}")

            idx = int(input("\nSelect vehicle number: ").strip()) - 1
            if 0 <= idx < len(vehicles):
                run_single_vehicle_demo(data_loader, vehicles[idx])
            else:
                print("Invalid selection")

        elif choice == "3":
            # Process all vehicles
            vehicles = data_loader.get_available_vehicles()
            print(f"\n Processing {len(vehicles)} vehicles...")

            for i, vid in enumerate(vehicles, 1):
                print(f"\n{'='*80}")
                print(f"VEHICLE {i}/{len(vehicles)}: {vid}")
                print(f"{'='*80}")
                run_single_vehicle_demo(data_loader, vid)

                if i < len(vehicles):
                    cont = input("\nContinue to next vehicle? (y/n): ").strip().lower()
                    if cont != 'y':
                        break

        elif choice == "4":
            # Process only vehicles with imminent failures
            vehicles = data_loader.get_available_vehicles()
            print(f"\n Scanning {len(vehicles)} vehicles for imminent failures...")

            critical_vehicles = []
            for vid in vehicles:
                telemetry = data_loader.get_vehicle_telemetry(vid, num_records=10)
                failures = telemetry['latest_readings']['failure_predictions']

                if (failures['engine_failure_imminent'] or
                    failures['brake_issue_imminent'] or
                    failures['battery_issue_imminent']):
                    critical_vehicles.append(vid)
                    print(f"   {vid}: Failure imminent!")

            if critical_vehicles:
                print(f"\n Found {len(critical_vehicles)} vehicles with imminent failures")
                for vid in critical_vehicles:
                    print(f"\nProcessing critical vehicle: {vid}")
                    run_single_vehicle_demo(data_loader, vid)

                    cont = input("\nContinue to next critical vehicle? (y/n): ").strip().lower()
                    if cont != 'y':
                        break
            else:
                print("\n No vehicles with imminent failures found")

        print("\n Demo completed successfully!")

    except FileNotFoundError as e:
        print(f"\n Error: CSV file not found")
        print(f"\nPlease ensure these files exist in the same directory:")
        print(f"  1. {TELEMETRY_CSV}")
        print(f"  2. {MAINTENANCE_CSV}")
        print(f"  3. {CUSTOMER_CSV}")
        print(f"\nError details: {e}")

    except Exception as e:
        print(f"\n Error: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()